In [1]:
from importlib import reload
#reload(Utilities)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [2]:
def build_sample_dfs(
    distinct_ids,
    save_dir, 
    years = [2020, 2021, 2022, 2023], 
    limit_per_month=10000, 
    conn_db=Utilities.get_athena_prod_aws_connection()
):
    r"""
    """
    #-------------------------
    general_sql = r"""
    SELECT
        EDE.issuertracking_id,
        EDE.serialnumber,
        EDE.enddeviceeventtypeid,
        EDE.valuesinterval,
        EDE.aep_premise_nb,
        EDE.reason,
        EDE.event_type,
        EDE.aep_opco,
        EDE.aep_event_dt
    FROM meter_events.end_device_event EDE
    WHERE EDE.aep_event_dt BETWEEN '{dt_0}' AND '{dt_1}'
    AND   EDE.enddeviceeventtypeid = '{ede_typeid}'
    LIMIT {limit}
    """
    #-------------------------
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    #-------------------------
    for i,distinct_id in enumerate(distinct_ids):
        print(f"distinct_id = {distinct_id} ({i+1}/{len(distinct_ids)})")
        dfs_for_id = []
        save_name_for_id = 'df_'+distinct_id.replace('.', '_')+'.pkl'
        #-----
        for year in years:
            # No harm in assuming all months have 31 days
            for month in range(1,13):
                dt_0 = "{year}-{month:02d}-01".format(year=year, month=month)
                dt_1 = "{year}-{month:02d}-31".format(year=year, month=month)
                sql_i = general_sql.format(
                    dt_0=dt_0, 
                    dt_1=dt_1, 
                    ede_typeid=distinct_id, 
                    limit=limit_per_month
                )
                df_i = pd.read_sql_query(sql_i, conn_db)
                dfs_for_id.append(df_i)
        assert(len(dfs_for_id) == 12*len(years))
        df_for_id = pd.concat(dfs_for_id)
        df_for_id.to_pickle((os.path.join(save_dir, save_name_for_id)))

In [3]:
def get_std_patterns_to_replace_by_typeid_EXPLICIT_V0():
    r"""
    Returns the standard patterns_to_replace_by_typeid.
    Function exists essentially to keep reduce_end_event_reasons_in_df from being too cluttered
    
    NOTE: Multi-line strings have more parentheses than absolutely necessary for clarity.
    NOTE: In (r'\s{2,}', ' '), the second element ' ' has a space between the quotes! So, multiple white-space characters
            are to be replaced with a single-space.
          This is different from many other cases, where ther is no space between the quotes, and in which case the match
            is to be removed (by replacing it with an empty character, e.g., (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''))  
    """
    #-------------------------
    patterns_to_replace_by_typeid = {
        #-------------------------
        # Examples
        # 'Power Loss cleared for meter 00:13:50:05:ff:0d:dd:17'
        '3.2.0.28':    [
            #-----
            (r'(Power Loss cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1')
#             #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],  
        
        #-------------------------
        # Examples
        # 'Power loss detected on meter 00:13:50:05:ff:0d:dd:17'
        '3.2.0.85':    [
            #-----
            (r'(Power loss detected)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Low Battery cleared for meter 00:13:50:02:00:0a:9e:1e.'
        '3.2.22.28':    [
            #-----
            (r'(Low Battery cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:02:00:0a:9e:1e.'
        '3.2.22.150':    [
            #-----
            (r'(Low Battery\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Test Mode Started occurred for meter 00:13:50:05:ff:1d:2d:4a.'
        # 'Meter event Test Mode Started  Time event occurred on meter = 01/20/2020 12:56:50  Sequence number = 56  User id = 1  Event argument = 00-00'
        # SEEMS RARE, BUT HAVE SEEN:
        # 'Meter is currently in test mode.'
        '3.7.19.242':    [
            #-----
            (r'(Test Mode Started)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'(Meter event Test Mode Started)\s*Time event occurred on meter = .* Sequence number = .* User id = .*  Event argument = .*', r'\1'), 
            #-----
            (r'(Meter event Test Mode Started).*', 'Test Mode Started'), 
            #-----
            (r'(Meter is currently in test mode)[\s\.]*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Test Mode Stopped occurred for meter 00:13:50:03:ff:06:20:96.'
        # 'Meter event Test Mode Stopped  Time event occurred on meter = 01/02/2020 08:25:04  Sequence number = 573  User id = 1  Event argument = 00-00'
        '3.7.19.243':    [
            #-----
            (r'(Test Mode Stopped)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'(Meter event Test Mode Stopped)\s*Time event occurred on meter = .* Sequence number = .* User id = .*  Event argument = .*', r'\1'), 
            #-----
            (r'(Meter event Test Mode Stopped).*', 'Test Mode Stopped')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'NIC operating on backup battery: 00:13:50:ff:fe:70:96:ac, Reboot Count: 6, NIC timestamp: 2021-01-14T10:17:54.000-05:00, Received timestamp: 2021-01-14T10:17:55.728-05:00'
        '3.7.22.4':    [
            #-----
            (r'(NIC operating on backup battery)\:(\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?), Reboot Count: .*, NIC timestamp: .*, Received timestamp: .*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # All seem to be: 'NIC backup battery inactive'
        # So, don't need to run any regex, but should still so no flags raised
        '3.7.22.19':    [
            #-----
            (r'(NIC backup battery inactive).*', r'\1')
            #-----
        ],      
        
        #-------------------------
        # Examples
        # 'Demand Reset occurred for meter 00:13:50:05:ff:18:b3:c1.'
        '3.8.0.215':    [
            #-----
            (r'(Demand Reset)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1') 
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter Program Seal mismatch for Device [Device ID, MAC Id] = [1ND785727916NMD06, 00:13:50:05:ff:1d:16:96] - program seal = 0bda0569c8e6ad375375eaa177034d3f61a80478 vs. UIQ program seal = e2dbe1ccbeef1f2df38adffeeafdb86fbd8fb006 at 2021-01-08T06:16:49.116-05:00.'
        '3.9.83.159':    [
            #-----
            (r'(Meter Program Seal mismatch for Device)\s*\[Device ID, MAC Id\]\s*=\s*.*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Requested operation JOB_OP_TYPE_ARB_METER_COMMAND could not be applied to the given device type and firmware version. Device: 00:13:50:05:ff:2d:ff:34, DeviceType: DID_SUBTYPE_I210CRD_HAN, Firmware Version: 3.12.5c'
        '3.11.63.161':  [
            #-----
            # Below, Device sometimes blank, sometime MAC-esque
            (r'(Requested operation)\s*(.*?)\s*(could not be applied) to the given device type and firmware version.\s*Device\:?(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)?,\s*DeviceType\:?.*,\s*Firmware Version\:?.*', r'\1 \3: \2')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 01/10/2020 10:03:37  Sequence number = 1189  User id = 0  Event argument = 00-00'
        # 'Meter detected a Tamper Attempt.'
        # 'Tamper attempt detected.'
        '3.12.0.257':   [
            #-----
            (r'(Meter event Tamper Attempt Suspected)\s*Time event occurred on meter = .* Sequence number = .* User id = .*  Event argument = .*', r'\1'), 
            #-----
            (r'(Meter event Tamper Attempt Suspected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a Tamper Attempt|Tamper attempt detected).*', 'Meter detected a Tamper Attempt'), 
            #-----
        ],    
        
        #-------------------------
        # Examples:
        # 'Tamper (Meter Inversion) detected on meter 00:13:50:05:ff:27:3f:a0.'
        '3.12.17.257':  [
            #-----
            (r'(Tamper\s*(?:\(.*\))?\s*detected)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\.?', '') 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Diag1 Condition cleared for meter 00:13:50:05:ff:04:55:a1.'
        # SEEMS RARE, BUT HAVE SEEN:
        # Reverse energy cleared for meter 00:13:50:05:ff:0d:dd:17.
        '3.12.48.28':    [
            #-----
            (r'((?:Diag1 Condition|Reverse energy) cleared)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Diag1: Polarity, Cross Phase, Reverse Energy Flow occurred for meter 00:13:50:03:00:34:b7:e9. Angle out of tolerance [Voltage - Phase  B].'
        # 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase A Current'
        # 'Service disconnect operation occurred with status: Close operation started'
        # 'Reverse energy (C1219 Table 3) occurred for meter 00:13:50:05:ff:0d:dd:17.'
        '3.12.48.219':  [
            #-----
            (r'(Diag1: Polarity, Cross Phase, Reverse Energy Flow)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)\s*(Angle out of tolerance) \[.*\]', r'\1: \2'), 
            #-----
            (r'(KV2c meter event Polarity, Cross Phase, Reverse Energy Flow) Diagnostic flags:.*', r'\1'), 
            #-----
            (r'(Service disconnect operation occurred with status:) (Open|Close) operation (started|succeeded)\s*', r'\1 \2 \3'), 
            #-----
            (r'((?:Diag1 Condition|Reverse energy)\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
        ],
        
        #-------------------------
        # Examples
        # 'Cleared: Meter00:13:50:05:ff:07:88:4b, cleared tamper detection (C1219 Table 3)'
        '3.12.93.28': [
            #-----
            (r'(Cleared: Meter)(?:\s*(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?), (cleared tamper detection.*)', r'\1 \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Meter event Reverse Rotation Detected  Time event occurred on meter = 01/28/2020 15:31:34  Sequence number = 854  User id = 0  Event argument = 00-00'
        # 'Meter detected a Reverse Rotation.'
        # 'Meter 00:13:50:05:ff:0b:ae:84, detected tampering (C1219 Table 3)'
        # SEEMS RARE, BUT HAVE SEEN:
        # Meter event Demand Reset Occurred  Time event occurred on meter = 10/25/2020 10:07:33  Sequence number = 1256  User id = 1  Event argument = 00-00
        '3.12.93.219':    [
            #-----
            (r'(Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\.,]?)\s*(detected tampering\s*(?:\(.*\))?)\s*', r'\1 \2'), 
            #-----
            (r'(Meter event (?:Reverse Rotation Detected|Demand Reset Occurred))\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'),
            #-----
            (r'(Meter event Reverse Rotation Detected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a Reverse Rotation).*', 'Meter event Reverse Rotation Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'A NET_MGMT command was sent from fdc9:ccbe:52c0:cbc0:250:56ff:feb5:91c7 with a key that has insufficient privileges to execute it. ID: 53 READ SUBID: 65535 ASSOC_ID: 8448'
        # 'A NET_MGMT command was sent from fdc9:ccbe:52c0:cbc0:a6ba:99ff:fe12:1b0e with a key that has insufficient privileges to execute it. ID: 207 WRITE SUBID: 65535 ASSOC_ID: 768'
        '3.12.136.38':  [
            #-----
            # BELOW, ID seemed meter-specific (or, at least, not general), as READ/WRITE SUBID and ASSOC_ID seem to only take on a few different values
            # Therefore, group 3, for ID, is not included in output
            (r'A?\s+?(.*command was sent) from(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(with a key that has insufficient privileges) to execute it.*(ID:\s*[0-9]+)\s*((?:READ|WRITE)\s*SUBID:\s*[0-9]+)\s*(ASSOC_ID:\s*[0-9]+).*', r'\1 \2: \4 \5'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'NET_MGMT command failed consecutively for 1 times for fd37:ec90:20c2:5c58:250:56ff:feb5:1010. WRITE'
        # 'Secure association operation failed consecutively for 1 times for fdc9:ccbe:52c0:d3a0:250:56ff:feb5:ec3c. 16270'
        # 'failed consecutively for 1 times for 0:0:0:0:0:0:0:0.'
        # 'N/A failed consecutively for 1 times for 0:0:0:0:0:0:0:0. N/A'
        '3.12.136.85':  [
            #-----
            (r'(.*) failed consecutively for [0-9]+ times?(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1 failed consecutively for 1 or more times'), 
            #-----
            (r'^failed consecutively for [0-9]+ times? for 0:0:0:0:0:0:0:0.$', 'N/A failed consecutively for 1 or more times'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Meter detected a RAM failure.'
        # So, don't need to run any regex, but should still so no flags raised
        '3.18.1.199':    [
            #-----
            (r'(Meter detected a RAM failure).*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # All seem to be: 'Meter detected a ROM failure.'
        # So, don't need to run any regex, but should still so no flags raised
        '3.18.1.220':    [
            #-----
            (r'(Meter detected a ROM failure).*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'NVRAM Error cleared for meter 00:13:50:05:ff:0d:7e:64.'
        '3.18.72.28':   [
            #-----
            (r'(NVRAM Error )(cleared?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'NVRAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:20:91:6c.'
        '3.18.72.79':   [
            #-----
            (r'(NVRAM Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 01/21/2020 09:45:01  Sequence number = 3335  User id = 0  Event argument = 00-00'
        # 'null'????????????
        # SEEMS RARE, BUT HAVE SEEN:
        # 'Meter event Demand Reset Occurred  Time event occurred on meter = 01/29/2023  Sequence number = 711  User id = 1  Event argument = 00-00 '
        # 'Meter event Reset List Pointers  Time event occurred on meter = 09/07/2021  Sequence number = 227  User id = 0  Event argument = 03-00'
        # 'KV2c meter event Received kWh Caution Diagnostic flags:Phase A Voltage'
        # 'Universal event Unsupported Event(41) with priority Alarm. '
        '3.18.72.85':   [
            #-----
#             (r'(Meter event (?:Nonvolatile Memory Failure Detected|Demand Reset Occurred|Reset List Pointers))\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'),
            (r'(Meter event.*?)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            # FAILSAFE in case Sequence number etc. not found above
            # NOTE: Cannot do (Meter event.*).*, as that will reduce everything altered above donw to simply 'Meter event'
            #       Also, if any additional info (e.g, Sequence number) is to be extracted using above, this must be removed
            (r'(Meter event (?:Nonvolatile Memory Failure Detected|Demand Reset Occurred|Reset List Pointers)).*', r'\1'),
            #-----
            (r'(Meter detected a nonvolatile memory failure).*', 'Meter event Nonvolatile Memory Failure Detected'), 
            #-----
            (r'(KV2c meter event .*?)\s*(Diagnostic flags)\s*\:\s*(.*?)\s*$', r'\1 (\2 = \3)'),
            #-----
            (r'(Universal event Unsupported Event\s*(?:\(.*\))?\s*with priority Alarm)[\s*\.]*', r'\1'), 
            #-----
            ('null', 'Reason is null, ID=3.18.72.85')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'RAM Error cleared for meter 00:13:50:05:ff:0d:dd:17.'
        '3.18.85.28':    [
            #-----
            (r'(RAM Error cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'RAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:0d:dd:17.'
        '3.18.85.79':   [
            #-----
            (r'(RAM Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a RAM failure.'
        # 'Meter event Ram Failure Detected  Time event occurred on meter = 7/27/2021 12:45:53 PM  Sequence number = 89  User id = 34891  Event argument = 7D-75'
        '3.18.85.85':   [
            #-----
            (r'(Meter event Ram Failure Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Ram Failure Detected).*', r'\1'),
            #-----
            (r'(Meter detected a RAM failure).*', 'Meter event Ram Failure Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1f:ba:d0.'
        '3.18.92.28':   [
            #-----
            (r'(ROM Error cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1f:ba:d0.'
        '3.18.92.79':   [
            #-----
            (r'(ROM Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Rom Failure Detected  Time event occurred on meter = 03/16/2020 07:00:51  Sequence number = 1983  User id = 1024  Event argument = 00-00'
        # 'Meter detected a ROM failure.'
        # 'ROM failure detected.'
        '3.18.92.85':    [
            #-----
            (r'(Meter event Rom Failure Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Rom Failure Detected).*', r'\1'),
            #-----
            (r'^(?:Meter detected a ROM failure|ROM failure detected).*$', 'Meter event Rom Failure Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Measurement Error Detected  Time event occurred on meter = 08/26/2021 08:27:25  Sequence number = 307  User id = 0  Event argument = 00-00 '
        '3.21.1.79':    [
            #-----
            (r'(Meter event Measurement Error Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Measurement Error Detected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a measurement error).*', 'Meter event Measurement Error Detected')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 08/30/2021 04:00:11  Sequence number = 3256  User id = 0  Event argument = 00-00 '
        '3.21.1.173':    [
            #-----
            (r'(Meter event Nonvolatile Memory Failure Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Nonvolatile Memory Failure Detected).*', r'\1'),
            #-----
            (r'(Meter detected a nonvolatile memory failure).*', 'Meter event Nonvolatile Memory Failure Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'System Error cleared for meter 00:13:50:05:ff:11:2d:23.'
        '3.21.3.28':    [
            #-----
            (r'(System Error cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:0f:17:ba.'
        '3.21.3.79':    [
            #-----
            (r'(System Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a self check error.'
        '3.21.17.28':    [
            #-----
            (r'(Meter detected a self check error).', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a self check error.'
        '3.21.18.79':    [
            #-----
            (r'(Meter detected a self check error).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Detected end of voltage sag on meter 00:13:50:05:ff:32:66:22 on one or several phases. Duration: 21 cycles (less than a second), Min RMS Voltage: Phase A 118.1 V, Phase B 95.4 V, Phase C 120.3 V, RMS Current (at min voltage): Phase A 1.0 A, Phase B 0.2 A, Phase C 1.2 A'
        '3.21.38.223':  [
            #-----
            (r'(Detected end of voltage sag)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*Duration:.*, Min RMS Voltage: Phase A .*, Phase B .*, Phase C .*, RMS Current \(at min voltage\): Phase A .*, Phase B .*, Phase C .*', r'\1'), 
            #-----
            # Failsafe
            (r'(Detected end of voltage sag)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Detected end of voltage swell on meter 00:13:50:05:ff:1b:49:b0 on one or several phases. Duration: 5 cycles (less than a second), Max RMS Voltage: Phase A 111.2 V, Phase B 133.2 V, Phase C 216.1 V, RMS Current (at max voltage): Phase A 2.8 A, Phase B 1.3 A, Phase C 0.8 A'
        '3.21.38.248':  [
            #-----
            (r'(Detected end of voltage swell)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*Duration: .*, Min RMS Voltage: .*, RMS Current \(at min voltage\): .*', r'\1'), 
            #-----
            # Failsafe
            (r'(Detected end of voltage swell)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1')  
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Detected end of voltage sag on meter 00:13:50:03:ff:01:a6:36. Duration: 2 seconds, Min RMS Voltage: Phase A 227.9 V, RMS Current (at min voltage): Phase A 0.0 A'
        '3.21.43.223':  [
            #-----
            (r'(Detected end of voltage sag)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*Duration: .*, Min RMS Voltage: .*, RMS Current \(at min voltage\): .*', r'\1'), 
            #-----
            (r'(Detected end of voltage sag)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1'), # failsafe  
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Detected end of voltage swell on meter 00:13:50:05:ff:2d:2c:25. Duration: 195 seconds, Max RMS Voltage: Phase A 265.7 V, RMS Current (at max voltage): Phase A 0.0 A'
        '3.21.43.248':  [
            #-----
            (r'(Detected end of voltage swell)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*Duration: .*, Min RMS Voltage: .*, RMS Current \(at min voltage\): .*', r'\1'), 
            #-----
            (r'(Detected end of voltage swell)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1'), # failsafe  
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Measurement Error cleared for meter 00:13:50:05:ff:07:c0:55.'
        '3.21.67.28':    [
            #-----
            (r'(Measurement Error.*cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Measurement Error Detected  Time event occurred on meter = 01/26/2020 19:47:00  Sequence number = 284  User id = 0  Event argument = 00-00'
        # 'Measurement Error (C1219 Table 3) occurred for meter 00:13:50:03:ff:06:d6:d5.'
        '3.21.67.79':    [
            #-----
            (r'(Measurement Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'(Meter event Measurement Error Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Measurement Error Detected).*', r'\1'),
            #-----
            (r'(Meter detected a measurement error).*', 'Meter event Measurement Error Detected')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'DSP Error cleared for meter 00:13:50:05:ff:0b:87:fc.'
        '3.21.82.28':    [
            #-----
            (r'(DSP Error cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:05:ff:2c:f3:e2.'
        '3.21.82.79':    [
            #-----
            (r'(DSP Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1') 
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Test Mode Stopped  Time event occurred on meter = 08/27/2021 10:04:04  Sequence number = 23834  User id = 1  Event argument = 00-00 '
        '3.22.12.243':   [
            #-----
            (r'(Meter event Test Mode Stopped)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Test Mode Stopped).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Test Mode Started  Time event occurred on meter = 08/27/2021 07:48:19  Sequence number = 765  User id = 1  Event argument = 00-00 '
        '3.22.19.242':   [
            #-----
            (r'(Meter event Test Mode Started)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Test Mode Started).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Device 00:13:50:05:ff:2e:17:7e has been determined to have exceeded the max allowable trap threshold, 20, within a certain time limit, 3600 seconds.'
        '3.23.17.139':   [
            #-----
            (r'(Device)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+).*(exceeded the max allowable trap threshold).*', r'\1 \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Access Point 00:13:50:08:ff:00:06:8e has lost connectivity with FHSS 900 MHz band.'
        '3.23.136.47':  [
            #-----
            (r'(Access Point)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(has lost connectivity with FHSS 900 MHz band.)', r'\1 \2') 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # Device: 00:13:50:05:ff:20:f5:ef Time: 2021-01-20T06:03:47.000-05:00 Failed Device: 00:13:50:05:ff:20:f5:ef Reason: Security public key mismatch Reboot Counter: 44 Refresh Counter: 0 Seconds since last reboot 6872803
        # 'NIC Link Layer Handshake Failed: Device: 00:13:50:05:ff:19:4e:6b, Rejected neighbor Mac ID: 00:13:50:05:ff:18:a3:52, Rejection Cause: invalid eblob signature'
        '3.23.136.85':  [
            #-----
            # Below, Device and Failed Device sometimes blank, sometime MAC-esque
            (r'Device\:?(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)? Time: .* Failed Device\:?(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)? (Reason: .*) Reboot Counter: .* Refresh Counter: .*', r'Device Failed: \1'), 
            #-----
            (r'(NIC Link Layer Handshake Failed): Device:(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+), Rejected neighbor Mac ID:(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+), (Rejection Cause: .*)', r'\1: \2')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter generated an energy polarity gyrbox call in event.'
        '3.25.17.3':   [
            #-----
            (r'(Meter generated an energy polarity gyrbox call in event).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Primary Power Down occurred for meter 00:13:50:05:ff:16:5f:61.'
        # 'Meter had a power outage. Unsafe power fail count = 32714'
        # 'Meter event Primary Power Down  Time event occurred on meter = 08/05/2021 16:59:27  Sequence number = 691  User id = 0  Event argument = 00-00'
        '3.26.0.47': [
            #-----
            (r'(Primary Power Down)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'(Meter had a power outage)\.?\s*(Unsafe power fail)\s*count\s*=\s*[0-9]+', r'\1 (\2)'), 
            #-----
            (r'^(Meter had a power outage).*$', r'\1'), #Fail proof, in case Unsafe power fail, count, etc., not found 
            #-----
            (r'(Meter event Primary Power Down)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Primary Power Down).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
        ],         
        
        #-------------------------
        # Examples
        # 'Primary Power Up occurred for meter 00:13:50:05:ff:22:2a:0c.'
        # 'Meter event Primary Power Up Time event occurred on meter = 08/14/2020 Sequence number = 343 User id = 0 Event argument = 00-00'
        '3.26.0.216':   [
            #-----
            (r'(Primary Power Up)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'Meter event (Primary Power Up)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'Meter event (Primary Power Up).*', r'\1'),  #Fail proof, in case time, sequence, etc., not found 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter had a power outage. Unsafe power fail count = 13050'
        # Meter event Primary Power Down  Time event occurred on meter = 08/31/2021 20:07:18  Sequence number = 103  User id = 0  Event argument = 00-00 
        '3.26.17.185':  [
            #-----
            (r'(Meter had a power outage).\s*(Unsafe power fail) count = [0-9]+', r'\1 (\2)'), 
            #-----
            (r'(Meter event Primary Power Down)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Primary Power Down).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Primary Power Up  Time event occurred on meter = 08/31/2021 20:07:44  Sequence number = 104  User id = 0  Event argument = 00-00 '
        '3.26.17.216':  [
            #-----
            (r'Meter event (Primary Power Up)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'Meter event (Primary Power Up).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Under Voltage cleared (CA000400) for meter 00:13:50:05:ff:0b:88:ec.'
        # 'Under Voltage (CA000400) cleared for meter 00:13:50:05:ff:15:e8:3b.'
        # 'Under Voltage (Diagnostic 6) cleared for meter 00:13:50:05:ff:3f:87:07N/A.'
        '3.26.38.37':   [
            #-----
            # NOTE: Due to annoying N/A at end of some MAC-esque IDs, I have to specify length of numerical entries
            #       as [0-9a-zA-Z]{1,2}, instead of the more general [0-9a-zA-Z]+ found elsewhere!
            (r'(Under Voltage|Low Potential|Diag6 Condition)\s*(\scleared)?\s*(\s\(.*\))?\s*(\scleared)?\s*(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+(?:N/A)?\.?)', r'\1\3\2\4')
            #-----
        ],
        
        #-------------------------
        # Examples
        # 'Low Potential (C1219 Table 3) occurred for meter 00:13:50:05:ff:1b:67:3b.'
        '3.26.38.47': [
            #-----
            (r'(Low Potential\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1') 
            #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Diag7 Condition cleared for meter 00:13:50:03:00:4d:41:fa.'
        # 'Over Voltage (Diagnostic 7) cleared for meter 00:13:50:05:ff:37:87:3dN/A.'
        '3.26.38.73':   [
            #-----
            # NOTE: Due to annoying N/A at end of some MAC-esque IDs, I have to specify length of numerical entries
            #       as [0-9a-zA-Z]{1,2}, instead of the more general [0-9a-zA-Z]+ found elsewhere!
            # NOTE: Also, the use of non-greedy (.*?) at beginning.
            #       If (.*) were used instead, results would be, e.g.
            #          'Over Voltage (Diagnostic 7) cleared for meter' instead of 'Over Voltage (Diagnostic 7) cleared'
            (r'(.*?)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+(?:N/A)?\.?)', r'\1')
            #-----
            #-----
#             # COULD ALSO DO
#             # NOTE: Due to annoying N/A at end of some MAC-esque IDs, I have to specify length of numerical entries
#             #       as [0-9a-zA-Z]{1,2}, instead of the more general [0-9a-zA-Z]+ found elsewhere!
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+(?:N/A)?\.?)', '')
        ],         
        
        #-------------------------
        # Examples
        # 'Diag7: Over Voltage, Element A occurred for meter 03.34.0.400:13:50:02:00:0a:d7:98.'
        # 'Over Voltage (Diagnostic 7) occurred for meter 00:13:50:05:ff:41:8a:8f: Phase A.'
        # SEEMS RARE, BUT HAVE SEEN:
        # 'KV2c meter event Over Voltage Diagnostic flags:Phase A Voltage'
        # 'KV2c meter event Under Voltage Caution Diagnostic flags:Phase B Voltage'
        # 'KV2c meter event Received kWh Caution Diagnostic flags:Phase A Voltage '
        # 'KV2c meter event Received kWh Caution Cleared '
        '3.26.38.93':   [
            #-----
            (r'(KV2c meter event (?:(?:Over|Under) Voltage)|(?:Voltage Out of Tolerance)|(?:(?:Received|Delivered) kWh))(?:\s*Caution)?\s*(Diagnostic flags)\s*\:\s*(.*?)\s*$', r'\1 (\2 = \3)'), 
            #-----
            (r'(KV2c meter event (?:(?:Over|Under) Voltage)|(?:Voltage Out of Tolerance)|(?:(?:Received|Delivered) kWh))(?:\s*Caution)?\s*Cleared\s*$', r'\1 Cleared'), 
            #-----
            (r'(.*Over Voltage.*?)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)(.*?)\s*$', r'\1\2') 
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Under Voltage (CA000400) for meter 00:13:50:05:ff:0b:88:ec. Phase  C Voltage out of tolerance.'
        # 'Under Voltage (CA000400) occurred for meter 00:13:50:05:ff:15:e8:3b: Phase A.'
        # 'KV2c meter event Under Voltage Diagnostic flags:Phase A Voltage'
        # 'Diag6: Under Voltage, Element A occurred for meter 00:13:50:05:ff:0b:88:ec.'
        '3.26.38.150':   [
            #-----
            # NOTE: Due to annoying N/A at end of some MAC-esque IDs, I have to specify length of numerical entries
            #       as [0-9a-zA-Z]{1,2}, instead of the more general [0-9a-zA-Z]+ found elsewhere!
            # NOTE: Also, the use of non-greedy (.*?) at beginning.
            #       If (.*) were used instead, results would be, e.g.
            #          'Diag6: Under Voltage, Element A occurred for meter'
            #       instead of 
            #          'Diag6: Under Voltage, Element A'
            (r'(Under Voltage\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\.\:]?)(\s?.*?)', r'\1\2'), 
            #-----
            (r'(Diag6.*?)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\.\:]?)(\s?.*?)', r'\1\2'), 
            #-----
            (r'(KV2c meter event .*?)\s*(Diagnostic flags)\s*\:\s*(.*?)\s*$', r'\1 (\2 = \3)')
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Last Gasp - NIC power lost for device: 00:13:50:05:ff:0b:83:e4, Reboot Count: 12264, NIC timestamp: 2020-01-19T22:21:02.000-05:00, Received timestamp: 2020-01-19T22:21:04.353-05:00, Fail Reason: [0x49] LG_ZERO_X_DETECTOR ,LG_DIRECT_NOTIFICATION'
        '3.26.136.47':  [
            #-----
            (r'(Last Gasp - NIC power lost for device):(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+).*(Fail Reason: .*)$', r'\1, \2')
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Device 00:13:50:05:ff:0b:8a:18, Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_DISABLED, Detector State: EL_EVENT_POWER_FAIL_DETECT_NIC_ZX_DISABLED, Reboot Count: 108'
        '3.26.136.66':  [
            #-----
            (r'Device(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+), (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'NIC Power Restore Trap Received from device: 00:13:50:05:ff:0c:7e:93, Reboot Count: 61, NIC timestamp: 2020-01-27T09:19:56.000-05:00, Received Timestamp: 2020-01-27T09:19:57.204-05:00, Power Restoration Timestamp: 2020-01-27T09:19:42.000-05:00, State: 4,p'
        '3.26.136.216': [
            #-----
            (r'(NIC Power Restore Trap Received from device):(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter assumed to be disconnected has reported Load side voltage indicating a potential case of tamper.'
        '3.31.1.143':   [
            #-----
            (r'(Meter assumed to be disconnected has reported Load side voltage indicating a potential case of tamper).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Reverse Rotation Detected  Time event occurred on meter = 08/30/2021 04:25:38  Sequence number = 13124  User id = 0  Event argument = 00-00 '
        '3.33.1.219':   [
            #-----
            (r'(Meter event Reverse Rotation Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Reverse Rotation Detected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a Reverse Rotation).*', 'Meter event Reverse Rotation Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a Tamper Attempt.'
        # 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 08/28/2021 08:45:05  Sequence number = 72  User id = 0  Event argument = 00-00 '
        '3.33.1.257':   [
            #-----
            (r'(Meter event Tamper Attempt Suspected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Tamper Attempt Suspected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a Tamper Attempt).*', 'Meter event Tamper Attempt Suspected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Cleared: Meter 00:13:50:05:ff:11:69:bb detected a high temperature condition. (C1219 Table 3)'
        '3.35.0.28': [
            #-----
            (r'(Cleared: Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(detected a high temperature condition)\.\s*(.*)', r'\1 \2 \3') 
            #-----
            #-----
#             # COULD ALSO DO
#             # NOTE: Here, the MAC-esque code occurs in middle of string, not the end as is common elsewhere.
#             #       This is why the pattern here excludes [\s*\.]* at the end
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', '')
        ],         
        
        #-------------------------
        # Examples
        # 'Meter 00:13:50:05:ff:11:69:bb detected a high temperature condition. (C1219 Table 3)'
        # 'Meter's temperature threshold exceeded.'
        # 'Meter event AX Temp Threshold Exceeded  Time event occurred on meter = 02/19/2021 13:33:50  Sequence number = 5115  User id = 0  Event argument = 00-00 '
        # 'Meter event S4TemperatureThreshold  Time event occurred on meter = 05/25/2023 17:12:50  Sequence number = 45  User id = 0  Event argument = 00-00-00-00-00-00 '
        '3.35.0.40': [
            #-----
            (r'(Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(detected a high temperature condition)\.\s*(.*)', r'\1 \2 \3'), 
            #-----
            (r"(Meter's temperature threshold exceeded).", r'\1'), 
            #-----
            (r'(Meter event .*?)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event (?:AX Temp Threshold Exceeded|S4TemperatureThreshold)).*', r'\1'),
        ],  
        
        #-------------------------
        # Examples
        # 'Meter detected a clock error.'
        # 'Meter event Clock Error Detected  Time event occurred on meter = 04/18/2020 10:00:10  Sequence number = 332  User id = 0  Event argument = 00-00'
        '3.36.0.79': [
            #-----
            (r'(Meter event Clock Error Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Clock Error Detected).*', r'\1'),
            #-----
            (r'(Meter detected a clock error).*', 'Meter event Clock Error Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a clock error.'
        '3.36.1.29': [
            #-----
            ('(Meter detected a clock error).', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter 00:13:50:05:ff:2e:13:de, detected loss of time (C1219 Table 3)'
        '3.36.114.73':  [
            #-----
            (r'(Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+),\s*(detected loss of time .*)', r'\1 \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Ignoring Interval Read data for device 00:13:50:05:ff:1e:fa:a4 as it has time in the future 2069-05-02 18:51:00.0'
        '3.36.114.159': [
            #-----
            (r'(Ignoring (?:Interval|Register) Read data for device)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(as it has time in the future).*', r'\1 \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter 00:13:50:05:ff:14:c9:61 needs explicit time sync. Drift: 14391 s, Encountered Problems:  TS_ERR_LP_BX, TS_ERR_BIG_DRIFT [0x44], Meter_Time: 06:33:41'
        '3.36.136.73':  [
            #-----
            (r'(Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time.*', r'\1 \2 \3'), 
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Error occurred when attempting to synch meter time with NIC time for device 00:13:50:05:ff:29:e1:20'
        # So, standard beg/end patterns should suffice
        '3.36.136.79':    [
            #-----
            (r'(Error occurred when attempting to synch meter time with NIC time for device)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)', r'\1')
            #-----
#             #-----
#             # COULD ALSO DO
#             (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
        ],         
        
        #-------------------------
        # Examples
        # 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase B Voltage '
        # 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase B Voltage, Phase C Voltage '
        '3.38.1.139':    [
            #-----
            (r'(KV2c meter event Polarity, Cross Phase, Reverse Energy Flow) Diagnostic flags:.*', r'\1'), 
            #-----
        ]
    }
    #-------------------------
    return patterns_to_replace_by_typeid

In [4]:
def get_std_patterns_to_replace_by_typeid_EXPLICIT():
    r"""
    Returns the standard patterns_to_replace_by_typeid.
    Function exists essentially to keep reduce_end_event_reasons_in_df from being too cluttered 
    """
    #-------------------------
    return {
        #-------------------------
        # Examples
        # 'Power Loss cleared for meter 00:13:50:05:ff:0d:dd:17'
        '3.2.0.28':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],  
        
        #-------------------------
        # Examples
        # 'Power loss detected on meter 00:13:50:05:ff:0d:dd:17'
        '3.2.0.85':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Low Battery cleared for meter 00:13:50:02:00:0a:9e:1e.'
        '3.2.22.28':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:02:00:0a:9e:1e.'
        '3.2.22.150':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Test Mode Started occurred for meter 00:13:50:05:ff:1d:2d:4a.'
        # 'Meter event Test Mode Started  Time event occurred on meter = 01/20/2020 12:56:50  Sequence number = 56  User id = 1  Event argument = 00-00'
        # SEEMS RARE, BUT HAVE SEEN:
        # 'Meter is currently in test mode.'
        '3.7.19.242':    [
            #-----
            (r'(Test Mode Started)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'(Meter event Test Mode Started)\s*Time event occurred on meter = .* Sequence number = .* User id = .*  Event argument = .*', r'\1'), 
            #-----
            (r'(Meter event Test Mode Started).*', 'Test Mode Started'), 
            #-----
            (r'(Meter is currently in test mode)[\s\.]*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Test Mode Stopped occurred for meter 00:13:50:03:ff:06:20:96.'
        # 'Meter event Test Mode Stopped  Time event occurred on meter = 01/02/2020 08:25:04  Sequence number = 573  User id = 1  Event argument = 00-00'
        '3.7.19.243':    [
            #-----
            (r'(Test Mode Stopped)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'(Meter event Test Mode Stopped)\s*Time event occurred on meter = .* Sequence number = .* User id = .*  Event argument = .*', r'\1'), 
            #-----
            (r'(Meter event Test Mode Stopped).*', 'Test Mode Stopped')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'NIC operating on backup battery: 00:13:50:ff:fe:70:96:ac, Reboot Count: 6, NIC timestamp: 2021-01-14T10:17:54.000-05:00, Received timestamp: 2021-01-14T10:17:55.728-05:00'
        '3.7.22.4':    [
            #-----
            (r'(NIC operating on backup battery)\:(\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?), Reboot Count: .*, NIC timestamp: .*, Received timestamp: .*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # All seem to be: 'NIC backup battery inactive'
        # So, don't need to run any regex, but should still so no flags raised
        '3.7.22.19':    [
            #-----
            (r'(NIC backup battery inactive).*', r'\1')
            #-----
        ],      
        
        #-------------------------
        # Examples
        # 'Demand Reset occurred for meter 00:13:50:05:ff:18:b3:c1.'
        '3.8.0.215':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter Program Seal mismatch for Device [Device ID, MAC Id] = [1ND785727916NMD06, 00:13:50:05:ff:1d:16:96] - program seal = 0bda0569c8e6ad375375eaa177034d3f61a80478 vs. UIQ program seal = e2dbe1ccbeef1f2df38adffeeafdb86fbd8fb006 at 2021-01-08T06:16:49.116-05:00.'
        '3.9.83.159':    [
            #-----
            (r'(Meter Program Seal mismatch for Device)\s*\[Device ID, MAC Id\]\s*=\s*.*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Requested operation JOB_OP_TYPE_ARB_METER_COMMAND could not be applied to the given device type and firmware version. Device: 00:13:50:05:ff:2d:ff:34, DeviceType: DID_SUBTYPE_I210CRD_HAN, Firmware Version: 3.12.5c'
        '3.11.63.161':  [
            #-----
            # Below, Device sometimes blank, sometime MAC-esque
            (r'(Requested operation)\s*(.*?)\s*(could not be applied) to the given device type and firmware version.\s*Device\:?(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)?,\s*DeviceType\:?.*,\s*Firmware Version\:?.*', r'\1 \3: \2')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 01/10/2020 10:03:37  Sequence number = 1189  User id = 0  Event argument = 00-00'
        # 'Meter detected a Tamper Attempt.'
        # 'Tamper attempt detected.'
        '3.12.0.257':   [
            #-----
            (r'(Meter event Tamper Attempt Suspected)\s*Time event occurred on meter = .* Sequence number = .* User id = .*  Event argument = .*', r'\1'), 
            #-----
            (r'(Meter event Tamper Attempt Suspected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a Tamper Attempt|Tamper attempt detected).*', 'Meter detected a Tamper Attempt'), 
            #-----
        ],    
        
        #-------------------------
        # Examples:
        # 'Tamper (Meter Inversion) detected on meter 00:13:50:05:ff:27:3f:a0.'
        '3.12.17.257':  [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\.?', '') 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Diag1 Condition cleared for meter 00:13:50:05:ff:04:55:a1.'
        # SEEMS RARE, BUT HAVE SEEN:
        # Reverse energy cleared for meter 00:13:50:05:ff:0d:dd:17.
        '3.12.48.28':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Diag1: Polarity, Cross Phase, Reverse Energy Flow occurred for meter 00:13:50:03:00:34:b7:e9. Angle out of tolerance [Voltage - Phase  B].'
        # 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase A Current'
        # 'Service disconnect operation occurred with status: Close operation started'
        # 'Reverse energy (C1219 Table 3) occurred for meter 00:13:50:05:ff:0d:dd:17.'
        '3.12.48.219':  [
            #-----
            (r'(Diag1: Polarity, Cross Phase, Reverse Energy Flow)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)\s*(Angle out of tolerance) \[.*\]', r'\1: \2'), 
            #-----
            (r'(KV2c meter event Polarity, Cross Phase, Reverse Energy Flow) Diagnostic flags:.*', r'\1'), 
            #-----
            (r'(Service disconnect operation occurred with status:) (Open|Close) operation (started|succeeded)\s*', r'\1 \2 \3'), 
            #-----
            (r'((?:Diag1 Condition|Reverse energy)\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
        ],
        
        #-------------------------
        # Examples
        # 'Cleared: Meter00:13:50:05:ff:07:88:4b, cleared tamper detection (C1219 Table 3)'
        '3.12.93.28': [
            #-----
            (r'(Cleared: Meter)(?:\s*(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?), (cleared tamper detection.*)', r'\1 \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Meter event Reverse Rotation Detected.  Time event occurred on meter = 01/28/2020 15:31:34  Sequence number = 854  User id = 0  Event argument = 00-00'
        # 'Meter detected a Reverse Rotation.'
        # 'Meter 00:13:50:05:ff:0b:ae:84, detected tampering (C1219 Table 3)'
        # SEEMS RARE, BUT HAVE SEEN:
        # Meter event Demand Reset Occurred  Time event occurred on meter = 10/25/2020 10:07:33  Sequence number = 1256  User id = 1  Event argument = 00-00
        '3.12.93.219':    [
            #-----
            (r'(Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\.,]?)\s*(detected tampering\s*(?:\(.*\))?)\s*', r'\1 \2'), 
            #-----
            (r'(Meter event (?:Reverse Rotation Detected|Demand Reset Occurred))\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'),
            #-----
            (r'(Meter event Reverse Rotation Detected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a Reverse Rotation).*', 'Meter event Reverse Rotation Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'A NET_MGMT command was sent from fdc9:ccbe:52c0:cbc0:250:56ff:feb5:91c7 with a key that has insufficient privileges to execute it. ID: 53 READ SUBID: 65535 ASSOC_ID: 8448'
        # 'A NET_MGMT command was sent from fdc9:ccbe:52c0:cbc0:a6ba:99ff:fe12:1b0e with a key that has insufficient privileges to execute it. ID: 207 WRITE SUBID: 65535 ASSOC_ID: 768'
        '3.12.136.38':  [
            #-----
            # BELOW, ID seemed meter-specific (or, at least, not general), as READ/WRITE SUBID and ASSOC_ID seem to only take on a few different values
            # Therefore, group 3, for ID, is not included in output
            (r'A?\s+?(.*command was sent) from(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(with a key that has insufficient privileges) to execute it.*(ID:\s*[0-9]+)\s*((?:READ|WRITE)\s*SUBID:\s*[0-9]+)\s*(ASSOC_ID:\s*[0-9]+).*', r'\1 \2: \4 \5'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'NET_MGMT command failed consecutively for 1 times for fd37:ec90:20c2:5c58:250:56ff:feb5:1010. WRITE'
        # 'Secure association operation failed consecutively for 1 times for fdc9:ccbe:52c0:d3a0:250:56ff:feb5:ec3c. 16270'
        # 'failed consecutively for 1 times for 0:0:0:0:0:0:0:0.'
        # 'N/A failed consecutively for 1 times for 0:0:0:0:0:0:0:0. N/A'
        '3.12.136.85':  [
            #-----
            (r'(.*) failed consecutively for [0-9]+ times?(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1 failed consecutively for 1 or more times'), 
            #-----
            (r'^failed consecutively for [0-9]+ times? for 0:0:0:0:0:0:0:0.$', 'N/A failed consecutively for 1 or more times'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Meter detected a RAM failure.'
        # So, don't need to run any regex, but should still so no flags raised
        '3.18.1.199':    [
            #-----
            (r'(Meter detected a RAM failure).*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # All seem to be: 'Meter detected a ROM failure.'
        # So, don't need to run any regex, but should still so no flags raised
        '3.18.1.220':    [
            #-----
            (r'(Meter detected a ROM failure).*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'NVRAM Error cleared for meter 00:13:50:05:ff:0d:7e:64.'
        '3.18.72.28':   [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'NVRAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:20:91:6c.'
        '3.18.72.79':   [
            #-----
            (r'(NVRAM Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 01/21/2020 09:45:01  Sequence number = 3335  User id = 0  Event argument = 00-00'
        # 'null'????????????
        # SEEMS RARE, BUT HAVE SEEN:
        # 'Meter event Demand Reset Occurred  Time event occurred on meter = 01/29/2023  Sequence number = 711  User id = 1  Event argument = 00-00 '
        # 'Meter event Reset List Pointers  Time event occurred on meter = 09/07/2021  Sequence number = 227  User id = 0  Event argument = 03-00'
        # 'KV2c meter event Received kWh Caution Diagnostic flags:Phase A Voltage'
        # 'Universal event Unsupported Event(41) with priority Alarm. '
        '3.18.72.85':   [
            #-----
#             (r'(Meter event (?:Nonvolatile Memory Failure Detected|Demand Reset Occurred|Reset List Pointers))\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'),
            (r'(Meter event.*?)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            # FAILSAFE in case Sequence number etc. not found above
            # NOTE: Cannot do (Meter event.*).*, as that will reduce everything altered above donw to simply 'Meter event'
            #       Also, if any additional info (e.g, Sequence number) is to be extracted using above, this must be removed
            (r'(Meter event (?:Nonvolatile Memory Failure Detected|Demand Reset Occurred|Reset List Pointers)).*', r'\1'),
            #-----
            (r'(Meter detected a nonvolatile memory failure).*', 'Meter event Nonvolatile Memory Failure Detected'), 
            #-----
            (r'(KV2c meter event .*?)\s*(Diagnostic flags)\s*\:\s*(.*?)\s*$', r'\1 (\2 = \3)'),
            #-----
            (r'(Universal event Unsupported Event\s*(?:\(.*\))?\s*with priority Alarm)[\s*\.]*', r'\1'), 
            #-----
            ('null', 'Reason is null, ID=3.18.72.85')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'RAM Error cleared for meter 00:13:50:05:ff:0d:dd:17.'
        '3.18.85.28':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'RAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:0d:dd:17.'
        '3.18.85.79':   [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a RAM failure.'
        # 'Meter event Ram Failure Detected  Time event occurred on meter = 7/27/2021 12:45:53 PM  Sequence number = 89  User id = 34891  Event argument = 7D-75'
        '3.18.85.85':   [
            #-----
            (r'(Meter event Ram Failure Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Ram Failure Detected).*', r'\1'),
            #-----
            (r'(Meter detected a RAM failure).*', 'Meter event Ram Failure Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1f:ba:d0.'
        '3.18.92.28':   [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1f:ba:d0.'
        '3.18.92.79':   [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Rom Failure Detected  Time event occurred on meter = 03/16/2020 07:00:51  Sequence number = 1983  User id = 1024  Event argument = 00-00'
        # 'Meter detected a ROM failure.'
        # 'ROM failure detected.'
        '3.18.92.85':    [
            #-----
            (r'(Meter event Rom Failure Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Rom Failure Detected).*', r'\1'),
            #-----
            (r'^(?:Meter detected a ROM failure|ROM failure detected).*$', 'Meter event Rom Failure Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Measurement Error Detected  Time event occurred on meter = 08/26/2021 08:27:25  Sequence number = 307  User id = 0  Event argument = 00-00 '
        '3.21.1.79':    [
            #-----
            (r'(Meter event Measurement Error Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Measurement Error Detected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a measurement error).*', 'Meter event Measurement Error Detected')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 08/30/2021 04:00:11  Sequence number = 3256  User id = 0  Event argument = 00-00 '
        '3.21.1.173':    [
            #-----
            (r'(Meter event Nonvolatile Memory Failure Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Nonvolatile Memory Failure Detected).*', r'\1'),
            #-----
            (r'(Meter detected a nonvolatile memory failure).*', 'Meter event Nonvolatile Memory Failure Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'System Error cleared for meter 00:13:50:05:ff:11:2d:23.'
        '3.21.3.28':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:0f:17:ba.'
        '3.21.3.79':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a self check error.'
        '3.21.17.28':    [
            #-----
            (r'(Meter detected a self check error).', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a self check error.'
        '3.21.18.79':    [
            #-----
            (r'(Meter detected a self check error).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Detected end of voltage sag on meter 00:13:50:05:ff:32:66:22 on one or several phases. Duration: 21 cycles (less than a second), Min RMS Voltage: Phase A 118.1 V, Phase B 95.4 V, Phase C 120.3 V, RMS Current (at min voltage): Phase A 1.0 A, Phase B 0.2 A, Phase C 1.2 A'
        '3.21.38.223':  [
            #-----
            (r'(Detected end of voltage sag)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*Duration:.*, Min RMS Voltage: Phase A .*, Phase B .*, Phase C .*, RMS Current \(at min voltage\): Phase A .*, Phase B .*, Phase C .*', r'\1'), 
            #-----
            # Failsafe
            (r'(Detected end of voltage sag)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Detected end of voltage swell on meter 00:13:50:05:ff:1b:49:b0 on one or several phases. Duration: 5 cycles (less than a second), Max RMS Voltage: Phase A 111.2 V, Phase B 133.2 V, Phase C 216.1 V, RMS Current (at max voltage): Phase A 2.8 A, Phase B 1.3 A, Phase C 0.8 A'
        '3.21.38.248':  [
            #-----
            (r'(Detected end of voltage swell)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*Duration: .*, Min RMS Voltage: .*, RMS Current \(at min voltage\): .*', r'\1'), 
            #-----
            # Failsafe
            (r'(Detected end of voltage swell)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1')  
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Detected end of voltage sag on meter 00:13:50:03:ff:01:a6:36. Duration: 2 seconds, Min RMS Voltage: Phase A 227.9 V, RMS Current (at min voltage): Phase A 0.0 A'
        '3.21.43.223':  [
            #-----
            (r'(Detected end of voltage sag)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*Duration: .*, Min RMS Voltage: .*, RMS Current \(at min voltage\): .*', r'\1'), 
            #-----
            (r'(Detected end of voltage sag)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1'), # failsafe  
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Detected end of voltage swell on meter 00:13:50:05:ff:2d:2c:25. Duration: 195 seconds, Max RMS Voltage: Phase A 265.7 V, RMS Current (at max voltage): Phase A 0.0 A'
        '3.21.43.248':  [
            #-----
            (r'(Detected end of voltage swell)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*Duration: .*, Min RMS Voltage: .*, RMS Current \(at min voltage\): .*', r'\1'), 
            #-----
            (r'(Detected end of voltage swell)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1'), # failsafe  
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Measurement Error cleared for meter 00:13:50:05:ff:07:c0:55.'
        '3.21.67.28':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Measurement Error Detected  Time event occurred on meter = 01/26/2020 19:47:00  Sequence number = 284  User id = 0  Event argument = 00-00'
        # 'Measurement Error (C1219 Table 3) occurred for meter 00:13:50:03:ff:06:d6:d5.'
        '3.21.67.79':    [
            #-----
            (r'(Measurement Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'(Meter event Measurement Error Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Measurement Error Detected).*', r'\1'),
            #-----
            (r'(Meter detected a measurement error).*', 'Meter event Measurement Error Detected')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'DSP Error cleared for meter 00:13:50:05:ff:0b:87:fc.'
        '3.21.82.28':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:05:ff:2c:f3:e2.'
        '3.21.82.79':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Test Mode Stopped  Time event occurred on meter = 08/27/2021 10:04:04  Sequence number = 23834  User id = 1  Event argument = 00-00 '
        '3.22.12.243':   [
            #-----
            (r'(Meter event Test Mode Stopped)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Test Mode Stopped).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Test Mode Started  Time event occurred on meter = 08/27/2021 07:48:19  Sequence number = 765  User id = 1  Event argument = 00-00 '
        '3.22.19.242':   [
            #-----
            (r'(Meter event Test Mode Started)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Test Mode Started).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Device 00:13:50:05:ff:2e:17:7e has been determined to have exceeded the max allowable trap threshold, 20, within a certain time limit, 3600 seconds.'
        '3.23.17.139':   [
            #-----
            (r'(Device)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+).*(exceeded the max allowable trap threshold).*', r'\1 \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Access Point 00:13:50:08:ff:00:06:8e has lost connectivity with FHSS 900 MHz band.'
        '3.23.136.47':  [
            #-----
            (r'(Access Point)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(has lost connectivity with FHSS 900 MHz band.)', r'\1 \2') 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # Device: 00:13:50:05:ff:20:f5:ef Time: 2021-01-20T06:03:47.000-05:00 Failed Device: 00:13:50:05:ff:20:f5:ef Reason: Security public key mismatch Reboot Counter: 44 Refresh Counter: 0 Seconds since last reboot 6872803
        # 'NIC Link Layer Handshake Failed: Device: 00:13:50:05:ff:19:4e:6b, Rejected neighbor Mac ID: 00:13:50:05:ff:18:a3:52, Rejection Cause: invalid eblob signature'
        '3.23.136.85':  [
            #-----
            # Below, Device and Failed Device sometimes blank, sometime MAC-esque
            (r'Device\:?(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)? Time: .* Failed Device\:?(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)? (Reason: .*) Reboot Counter: .* Refresh Counter: .*', r'Device Failed: \1'), 
            #-----
            (r'(NIC Link Layer Handshake Failed): Device:(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+), Rejected neighbor Mac ID:(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+), (Rejection Cause: .*)', r'\1: \2')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter generated an energy polarity gyrbox call in event.'
        '3.25.17.3':   [
            #-----
            (r'(Meter generated an energy polarity gyrbox call in event).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Primary Power Down occurred for meter 00:13:50:05:ff:16:5f:61.'
        # 'Meter had a power outage. Unsafe power fail count = 32714'
        # 'Meter event Primary Power Down  Time event occurred on meter = 08/05/2021 16:59:27  Sequence number = 691  User id = 0  Event argument = 00-00'
        '3.26.0.47': [
            #-----
            (r'(Primary Power Down)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'(Meter had a power outage)\.?\s*(Unsafe power fail)\s*count\s*=\s*[0-9]+', r'\1 (\2)'), 
            #-----
            (r'^(Meter had a power outage).*$', r'\1'), #Fail proof, in case Unsafe power fail, count, etc., not found 
            #-----
            (r'(Meter event Primary Power Down)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Primary Power Down).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
        ],         
        
        #-------------------------
        # Examples
        # 'Primary Power Up occurred for meter 00:13:50:05:ff:22:2a:0c.'
        # 'Meter event Primary Power Up Time event occurred on meter = 08/14/2020 Sequence number = 343 User id = 0 Event argument = 00-00'
        '3.26.0.216':   [
            #-----
            (r'(Primary Power Up)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'Meter event (Primary Power Up)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'Meter event (Primary Power Up).*', r'\1'),  #Fail proof, in case time, sequence, etc., not found 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter had a power outage. Unsafe power fail count = 13050'
        # Meter event Primary Power Down  Time event occurred on meter = 08/31/2021 20:07:18  Sequence number = 103  User id = 0  Event argument = 00-00 
        '3.26.17.185':  [
            #-----
            (r'(Meter had a power outage).\s*(Unsafe power fail) count = [0-9]+', r'\1 (\2)'), 
            #-----
            (r'(Meter event Primary Power Down)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Primary Power Down).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Primary Power Up  Time event occurred on meter = 08/31/2021 20:07:44  Sequence number = 104  User id = 0  Event argument = 00-00 '
        '3.26.17.216':  [
            #-----
            (r'Meter event (Primary Power Up)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'Meter event (Primary Power Up).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Under Voltage cleared (CA000400) for meter 00:13:50:05:ff:0b:88:ec.'
        # 'Under Voltage (CA000400) cleared for meter 00:13:50:05:ff:15:e8:3b.'
        # 'Under Voltage (Diagnostic 6) cleared for meter 00:13:50:05:ff:3f:87:07N/A.'
        '3.26.38.37':   [
            #-----
            # NOTE: Due to annoying N/A at end of some MAC-esque IDs, I have to specify length of numerical entries
            #       as [0-9a-zA-Z]{1,2}, instead of the more general [0-9a-zA-Z]+ found elsewhere!
            (r'(Under Voltage|Low Potential|Diag6 Condition)\s*(\scleared)?\s*(\s\(.*\))?\s*(\scleared)?\s*(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+(?:N/A)?\.?)', r'\1\3\2\4')
            #-----
        ],       
        
        #-------------------------
        # Examples
        # 'Low Potential (C1219 Table 3) occurred for meter 00:13:50:05:ff:1b:67:3b.'
        '3.26.38.47': [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Diag7 Condition cleared for meter 00:13:50:03:00:4d:41:fa.'
        # 'Over Voltage (Diagnostic 7) cleared for meter 00:13:50:05:ff:37:87:3dN/A.'
        '3.26.38.73':   [
            #-----
            # NOTE: Due to annoying N/A at end of some MAC-esque IDs, I have to specify length of numerical entries
            #       as [0-9a-zA-Z]{1,2}, instead of the more general [0-9a-zA-Z]+ found elsewhere!
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+(?:N/A)?\.?)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Diag7: Over Voltage, Element A occurred for meter 03.34.0.400:13:50:02:00:0a:d7:98.'
        # 'Over Voltage (Diagnostic 7) occurred for meter 00:13:50:05:ff:41:8a:8f: Phase A.'
        # SEEMS RARE, BUT HAVE SEEN:
        # 'KV2c meter event Over Voltage Diagnostic flags:Phase A Voltage'
        # 'KV2c meter event Under Voltage Caution Diagnostic flags:Phase B Voltage'
        # 'KV2c meter event Received kWh Caution Diagnostic flags:Phase A Voltage '
        # 'KV2c meter event Received kWh Caution Cleared '
        '3.26.38.93':   [
            #-----
            (r'(KV2c meter event (?:(?:Over|Under) Voltage)|(?:Voltage Out of Tolerance)|(?:(?:Received|Delivered) kWh))(?:\s*Caution)?\s*(Diagnostic flags)\s*\:\s*(.*?)\s*$', r'\1 (\2 = \3)'), 
            #-----
            (r'(KV2c meter event (?:(?:Over|Under) Voltage)|(?:Voltage Out of Tolerance)|(?:(?:Received|Delivered) kWh))(?:\s*Caution)?\s*Cleared\s*$', r'\1 Cleared'), 
            #-----
            (r'(.*Over Voltage.*?)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)(.*?)\s*$', r'\1\2') 
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Under Voltage (CA000400) for meter 00:13:50:05:ff:0b:88:ec. Phase  C Voltage out of tolerance.'
        # 'Under Voltage (CA000400) occurred for meter 00:13:50:05:ff:15:e8:3b: Phase A.'
        # 'KV2c meter event Under Voltage Diagnostic flags:Phase A Voltage'
        # 'Diag6: Under Voltage, Element A occurred for meter 00:13:50:05:ff:0b:88:ec.'
        '3.26.38.150':   [
            #-----
            # NOTE: Due to annoying N/A at end of some MAC-esque IDs, I have to specify length of numerical entries
            #       as [0-9a-zA-Z]{1,2}, instead of the more general [0-9a-zA-Z]+ found elsewhere!
            # NOTE: Also, the use of non-greedy (.*?) at beginning.
            #       If (.*) were used instead, results would be, e.g.
            #          'Diag6: Under Voltage, Element A occurred for meter'
            #       instead of 
            #          'Diag6: Under Voltage, Element A'
            (r'(Under Voltage\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\.\:]?)(\s?.*?)', r'\1\2'), 
            #-----
            (r'(Diag6.*?)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\.\:]?)(\s?.*?)', r'\1\2'), 
            #-----
            (r'(KV2c meter event .*?)\s*(Diagnostic flags)\s*\:\s*(.*?)\s*$', r'\1 (\2 = \3)')
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Last Gasp - NIC power lost for device: 00:13:50:05:ff:0b:83:e4, Reboot Count: 12264, NIC timestamp: 2020-01-19T22:21:02.000-05:00, Received timestamp: 2020-01-19T22:21:04.353-05:00, Fail Reason: [0x49] LG_ZERO_X_DETECTOR ,LG_DIRECT_NOTIFICATION'
        '3.26.136.47':  [
            #-----
            (r'(Last Gasp - NIC power lost for device):(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+).*(Fail Reason: .*)$', r'\1, \2')
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Device 00:13:50:05:ff:0b:8a:18, Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_DISABLED, Detector State: EL_EVENT_POWER_FAIL_DETECT_NIC_ZX_DISABLED, Reboot Count: 108'
        '3.26.136.66':  [
            #-----
            (r'Device(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+), (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'NIC Power Restore Trap Received from device: 00:13:50:05:ff:0c:7e:93, Reboot Count: 61, NIC timestamp: 2020-01-27T09:19:56.000-05:00, Received Timestamp: 2020-01-27T09:19:57.204-05:00, Power Restoration Timestamp: 2020-01-27T09:19:42.000-05:00, State: 4,p'
        '3.26.136.216': [
            #-----
            (r'(NIC Power Restore Trap Received from device):(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter assumed to be disconnected has reported Load side voltage indicating a potential case of tamper.'
        '3.31.1.143':   [
            #-----
            (r'(Meter assumed to be disconnected has reported Load side voltage indicating a potential case of tamper).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Reverse Rotation Detected  Time event occurred on meter = 08/30/2021 04:25:38  Sequence number = 13124  User id = 0  Event argument = 00-00 '
        '3.33.1.219':   [
            #-----
            (r'(Meter event Reverse Rotation Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Reverse Rotation Detected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a Reverse Rotation).*', 'Meter event Reverse Rotation Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a Tamper Attempt.'
        # 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 08/28/2021 08:45:05  Sequence number = 72  User id = 0  Event argument = 00-00 '
        '3.33.1.257':   [
            #-----
            (r'(Meter event Tamper Attempt Suspected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Tamper Attempt Suspected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a Tamper Attempt).*', 'Meter event Tamper Attempt Suspected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Cleared: Meter 00:13:50:05:ff:11:69:bb detected a high temperature condition. (C1219 Table 3)'
        '3.35.0.28': [
            #-----
            # NOTE: Here, the MAC-esque code occurs in middle of string, not the end as is common elsewhere.
            #       This is why the pattern here excludes [\s*\.]* at the end
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter 00:13:50:05:ff:11:69:bb detected a high temperature condition. (C1219 Table 3)'
        # 'Meter's temperature threshold exceeded.'
        # 'Meter event AX Temp Threshold Exceeded  Time event occurred on meter = 02/19/2021 13:33:50  Sequence number = 5115  User id = 0  Event argument = 00-00 '
        # 'Meter event S4TemperatureThreshold  Time event occurred on meter = 05/25/2023 17:12:50  Sequence number = 45  User id = 0  Event argument = 00-00-00-00-00-00 '
        '3.35.0.40': [
            #-----
            (r'(Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(detected a high temperature condition)\.\s*(.*)', r'\1 \2 \3'), 
            #-----
            (r"(Meter's temperature threshold exceeded).", r'\1'), 
            #-----
            (r'(Meter event .*?)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event (?:AX Temp Threshold Exceeded|S4TemperatureThreshold)).*', r'\1'),
        ],  
        
        #-------------------------
        # Examples
        # 'Meter detected a clock error.'
        # 'Meter event Clock Error Detected  Time event occurred on meter = 04/18/2020 10:00:10  Sequence number = 332  User id = 0  Event argument = 00-00'
        '3.36.0.79': [
            #-----
            (r'(Meter event Clock Error Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Clock Error Detected).*', r'\1'),
            #-----
            (r'(Meter detected a clock error).*', 'Meter event Clock Error Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a clock error.'
        '3.36.1.29': [
            #-----
            ('(Meter detected a clock error).', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter 00:13:50:05:ff:2e:13:de, detected loss of time (C1219 Table 3)'
        '3.36.114.73':  [
            #-----
            (r'(Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+),\s*(detected loss of time .*)', r'\1 \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Ignoring Interval Read data for device 00:13:50:05:ff:1e:fa:a4 as it has time in the future 2069-05-02 18:51:00.0'
        '3.36.114.159': [
            #-----
            (r'(Ignoring (?:Interval|Register) Read data for device)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(as it has time in the future).*', r'\1 \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter 00:13:50:05:ff:14:c9:61 needs explicit time sync. Drift: 14391 s, Encountered Problems:  TS_ERR_LP_BX, TS_ERR_BIG_DRIFT [0x44], Meter_Time: 06:33:41'
        '3.36.136.73':  [
            #-----
            (r'(Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time.*', r'\1 \2 \3'), 
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Error occurred when attempting to synch meter time with NIC time for device 00:13:50:05:ff:29:e1:20'
        # So, standard beg/end patterns should suffice
        '3.36.136.79':    [
            #-----
            (r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\s*\.]*)', '')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase B Voltage '
        # 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase B Voltage, Phase C Voltage '
        '3.38.1.139':    [
            #-----
            (r'(KV2c meter event Polarity, Cross Phase, Reverse Energy Flow) Diagnostic flags:.*', r'\1'), 
            #-----
        ]
    }

In [5]:
def get_std_patterns_to_replace_by_typeid_EXPLICIT_v2():
    r"""
    Returns the standard patterns_to_replace_by_typeid.
    Function exists essentially to keep reduce_end_event_reasons_in_df from being too cluttered
    
    NOTE: Multi-line strings have more parentheses than absolutely necessary for clarity.
    NOTE: In (r'\s{2,}', ' '), the second element ' ' has a space between the quotes! So, multiple white-space characters
            are to be replaced with a single-space.
          This is different from many other cases, where ther is no space between the quotes, and in which case the match
            is to be removed (by replacing it with an empty character, e.g., (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''))  
    """
    #-------------------------
    patterns_to_replace_by_typeid = {
        #-------------------------
        # Examples
        # 'Power Loss cleared for meter 00:13:50:05:ff:0d:dd:17'
        '3.2.0.28':    [
            #-----
            (r'(Power Loss cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1')
            #-----
        ],  
        
        #-------------------------
        # Examples
        # 'Power loss detected on meter 00:13:50:05:ff:0d:dd:17'
        '3.2.0.85':    [
            #-----
            (r'(Power loss detected)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Low Battery cleared for meter 00:13:50:02:00:0a:9e:1e.'
        '3.2.22.28':    [
            #-----
            (r'(Low Battery cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:02:00:0a:9e:1e.'
        '3.2.22.150':    [
            #-----
            (r'(Low Battery\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Test Mode Started occurred for meter 00:13:50:05:ff:1d:2d:4a.'
        # 'Meter event Test Mode Started  Time event occurred on meter = 01/20/2020 12:56:50  Sequence number = 56  User id = 1  Event argument = 00-00'
        # SEEMS RARE, BUT HAVE SEEN:
        # 'Meter is currently in test mode.'
        '3.7.19.242':    [
            #-----
            (r'(Test Mode Started)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'(Meter event Test Mode Started)\s*Time event occurred on meter = .* Sequence number = .* User id = .*  Event argument = .*', r'\1'), 
            #-----
            (r'(Meter event Test Mode Started).*', 'Test Mode Started'), 
            #-----
            (r'(Meter is currently in test mode)[\s\.]*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Test Mode Stopped occurred for meter 00:13:50:03:ff:06:20:96.'
        # 'Meter event Test Mode Stopped  Time event occurred on meter = 01/02/2020 08:25:04  Sequence number = 573  User id = 1  Event argument = 00-00'
        '3.7.19.243':    [
            #-----
            (r'(Test Mode Stopped)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'(Meter event Test Mode Stopped)\s*Time event occurred on meter = .* Sequence number = .* User id = .*  Event argument = .*', r'\1'), 
            #-----
            (r'(Meter event Test Mode Stopped).*', 'Test Mode Stopped')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'NIC operating on backup battery: 00:13:50:ff:fe:70:96:ac, Reboot Count: 6, NIC timestamp: 2021-01-14T10:17:54.000-05:00, Received timestamp: 2021-01-14T10:17:55.728-05:00'
        '3.7.22.4':    [
            #-----
            (r'(NIC operating on backup battery)\:(\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?), Reboot Count: .*, NIC timestamp: .*, Received timestamp: .*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # All seem to be: 'NIC backup battery inactive'
        # So, don't need to run any regex, but should still so no flags raised
        '3.7.22.19':    [
            #-----
            (r'(NIC backup battery inactive).*', r'\1')
            #-----
        ],      
        
        #-------------------------
        # Examples
        # 'Demand Reset occurred for meter 00:13:50:05:ff:18:b3:c1.'
        '3.8.0.215':    [
            #-----
            (r'(Demand Reset)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1') 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter Program Seal mismatch for Device [Device ID, MAC Id] = [1ND785727916NMD06, 00:13:50:05:ff:1d:16:96] - program seal = 0bda0569c8e6ad375375eaa177034d3f61a80478 vs. UIQ program seal = e2dbe1ccbeef1f2df38adffeeafdb86fbd8fb006 at 2021-01-08T06:16:49.116-05:00.'
        '3.9.83.159':    [
            #-----
            (r'(Meter Program Seal mismatch for Device)\s*\[Device ID, MAC Id\]\s*=\s*.*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Requested operation JOB_OP_TYPE_ARB_METER_COMMAND could not be applied to the given device type and firmware version. Device: 00:13:50:05:ff:2d:ff:34, DeviceType: DID_SUBTYPE_I210CRD_HAN, Firmware Version: 3.12.5c'
        '3.11.63.161':  [
            #-----
            # Below, Device sometimes blank, sometime MAC-esque
            (r'(Requested operation)\s*(.*?)\s*(could not be applied) to the given device type and firmware version.\s*Device\:?(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)?,\s*DeviceType\:?.*,\s*Firmware Version\:?.*', r'\1 \3: \2')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 01/10/2020 10:03:37  Sequence number = 1189  User id = 0  Event argument = 00-00'
        # 'Meter detected a Tamper Attempt.'
        # 'Tamper attempt detected.'
        '3.12.0.257':   [
            #-----
            (r'(Meter event Tamper Attempt Suspected)\s*Time event occurred on meter = .* Sequence number = .* User id = .*  Event argument = .*', r'\1'), 
            #-----
            (r'(Meter event Tamper Attempt Suspected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a Tamper Attempt|Tamper attempt detected).*', 'Meter detected a Tamper Attempt'), 
            #-----
        ],    
        
        #-------------------------
        # Examples:
        # 'Tamper (Meter Inversion) detected on meter 00:13:50:05:ff:27:3f:a0.'
        '3.12.17.257':  [
            #-----
            (r'(Tamper\s*(?:\(.*\))?\s*detected)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Diag1 Condition cleared for meter 00:13:50:05:ff:04:55:a1.'
        # SEEMS RARE, BUT HAVE SEEN:
        # Reverse energy cleared for meter 00:13:50:05:ff:0d:dd:17.
        '3.12.48.28':    [
            #-----
            (r'((?:Diag1 Condition|Reverse energy) cleared)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Diag1: Polarity, Cross Phase, Reverse Energy Flow occurred for meter 00:13:50:03:00:34:b7:e9. Angle out of tolerance [Voltage - Phase  B].'
        # 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase A Current'
        # 'Service disconnect operation occurred with status: Close operation started'
        # 'Reverse energy (C1219 Table 3) occurred for meter 00:13:50:05:ff:0d:dd:17.'
        '3.12.48.219':  [
            #-----
            (r'(Diag1: Polarity, Cross Phase, Reverse Energy Flow)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)\s*(Angle out of tolerance) \[.*\]', r'\1: \2'), 
            #-----
            (r'(KV2c meter event Polarity, Cross Phase, Reverse Energy Flow) Diagnostic flags:.*', r'\1'), 
            #-----
            (r'(Service disconnect operation occurred with status:) (Open|Close) operation (started|succeeded)\s*', r'\1 \2 \3'), 
            #-----
            (r'((?:Diag1 Condition|Reverse energy)\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
        ],
        
        #-------------------------
        # Examples
        # 'Cleared: Meter00:13:50:05:ff:07:88:4b, cleared tamper detection (C1219 Table 3)'
        '3.12.93.28': [
            #-----
            (r'(Cleared: Meter)(?:\s*(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?), (cleared tamper detection.*)', r'\1 \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Meter event Reverse Rotation Detected  Time event occurred on meter = 01/28/2020 15:31:34  Sequence number = 854  User id = 0  Event argument = 00-00'
        # 'Meter detected a Reverse Rotation.'
        # 'Meter 00:13:50:05:ff:0b:ae:84, detected tampering (C1219 Table 3)'
        # SEEMS RARE, BUT HAVE SEEN:
        # Meter event Demand Reset Occurred  Time event occurred on meter = 10/25/2020 10:07:33  Sequence number = 1256  User id = 1  Event argument = 00-00
        '3.12.93.219':    [
            #-----
            (r'(Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\.,]?)\s*(detected tampering\s*(?:\(.*\))?)\s*', r'\1 \2'), 
            #-----
            (r'(Meter event (?:Reverse Rotation Detected|Demand Reset Occurred))\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'),
            #-----
            (r'(Meter event Reverse Rotation Detected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a Reverse Rotation).*', 'Meter event Reverse Rotation Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'A NET_MGMT command was sent from fdc9:ccbe:52c0:cbc0:250:56ff:feb5:91c7 with a key that has insufficient privileges to execute it. ID: 53 READ SUBID: 65535 ASSOC_ID: 8448'
        # 'A NET_MGMT command was sent from fdc9:ccbe:52c0:cbc0:a6ba:99ff:fe12:1b0e with a key that has insufficient privileges to execute it. ID: 207 WRITE SUBID: 65535 ASSOC_ID: 768'
        '3.12.136.38':  [
            #-----
            # BELOW, ID seemed meter-specific (or, at least, not general), as READ/WRITE SUBID and ASSOC_ID seem to only take on a few different values
            # Therefore, group 3, for ID, is not included in output
            (r'A?\s+?(.*command was sent) from(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(with a key that has insufficient privileges) to execute it.*(ID:\s*[0-9]+)\s*((?:READ|WRITE)\s*SUBID:\s*[0-9]+)\s*(ASSOC_ID:\s*[0-9]+).*', r'\1 \2: \4 \5'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'NET_MGMT command failed consecutively for 1 times for fd37:ec90:20c2:5c58:250:56ff:feb5:1010. WRITE'
        # 'Secure association operation failed consecutively for 1 times for fdc9:ccbe:52c0:d3a0:250:56ff:feb5:ec3c. 16270'
        # 'failed consecutively for 1 times for 0:0:0:0:0:0:0:0.'
        # 'N/A failed consecutively for 1 times for 0:0:0:0:0:0:0:0. N/A'
        '3.12.136.85':  [
            #-----
            (r'(.*) failed consecutively for [0-9]+ times?(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1 failed consecutively for 1 or more times'), 
            #-----
            (r'^failed consecutively for [0-9]+ times? for 0:0:0:0:0:0:0:0.$', 'N/A failed consecutively for 1 or more times'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Meter detected a RAM failure.'
        # So, don't need to run any regex, but should still so no flags raised
        '3.18.1.199':    [
            #-----
            (r'(Meter detected a RAM failure).*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # All seem to be: 'Meter detected a ROM failure.'
        # So, don't need to run any regex, but should still so no flags raised
        '3.18.1.220':    [
            #-----
            (r'(Meter detected a ROM failure).*', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'NVRAM Error cleared for meter 00:13:50:05:ff:0d:7e:64.'
        '3.18.72.28':   [
            #-----
            (r'(NVRAM Error cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'NVRAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:20:91:6c.'
        '3.18.72.79':   [
            #-----
            (r'(NVRAM Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 01/21/2020 09:45:01  Sequence number = 3335  User id = 0  Event argument = 00-00'
        # 'null'????????????
        # SEEMS RARE, BUT HAVE SEEN:
        # 'Meter event Demand Reset Occurred  Time event occurred on meter = 01/29/2023  Sequence number = 711  User id = 1  Event argument = 00-00 '
        # 'Meter event Reset List Pointers  Time event occurred on meter = 09/07/2021  Sequence number = 227  User id = 0  Event argument = 03-00'
        # 'KV2c meter event Received kWh Caution Diagnostic flags:Phase A Voltage'
        # 'Universal event Unsupported Event(41) with priority Alarm. '
        '3.18.72.85':   [
            #-----
#             (r'(Meter event (?:Nonvolatile Memory Failure Detected|Demand Reset Occurred|Reset List Pointers))\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'),
            (r'(Meter event.*?)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            # FAILSAFE in case Sequence number etc. not found above
            # NOTE: Cannot do (Meter event.*).*, as that will reduce everything altered above donw to simply 'Meter event'
            #       Also, if any additional info (e.g, Sequence number) is to be extracted using above, this must be removed
            (r'(Meter event (?:Nonvolatile Memory Failure Detected|Demand Reset Occurred|Reset List Pointers)).*', r'\1'),
            #-----
            (r'(Meter detected a nonvolatile memory failure).*', 'Meter event Nonvolatile Memory Failure Detected'), 
            #-----
            (r'(KV2c meter event .*?)\s*(Diagnostic flags)\s*\:\s*(.*?)\s*$', r'\1 (\2 = \3)'),
            #-----
            (r'(Universal event Unsupported Event\s*(?:\(.*\))?\s*with priority Alarm)[\s*\.]*', r'\1'), 
            #-----
            ('null', 'Reason is null, ID=3.18.72.85')
            #-----
        ],         
        
        #-------------------------
        # Examples:
        # 'RAM Error cleared for meter 00:13:50:05:ff:0d:dd:17.'
        '3.18.85.28':    [
            #-----
            (r'(RAM Error cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'RAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:0d:dd:17.'
        '3.18.85.79':   [
            #-----
            (r'(RAM Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a RAM failure.'
        # 'Meter event Ram Failure Detected  Time event occurred on meter = 7/27/2021 12:45:53 PM  Sequence number = 89  User id = 34891  Event argument = 7D-75'
        '3.18.85.85':   [
            #-----
            (r'(Meter event Ram Failure Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Ram Failure Detected).*', r'\1'),
            #-----
            (r'(Meter detected a RAM failure).*', 'Meter event Ram Failure Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1f:ba:d0.'
        '3.18.92.28':   [
            #-----
            (r'(ROM Error cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1f:ba:d0.'
        '3.18.92.79':   [
            #-----
            (r'(ROM Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Rom Failure Detected  Time event occurred on meter = 03/16/2020 07:00:51  Sequence number = 1983  User id = 1024  Event argument = 00-00'
        # 'Meter detected a ROM failure.'
        # 'ROM failure detected.'
        '3.18.92.85':    [
            #-----
            (r'(Meter event Rom Failure Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Rom Failure Detected).*', r'\1'),
            #-----
            (r'^(?:Meter detected a ROM failure|ROM failure detected).*$', 'Meter event Rom Failure Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Measurement Error Detected  Time event occurred on meter = 08/26/2021 08:27:25  Sequence number = 307  User id = 0  Event argument = 00-00 '
        '3.21.1.79':    [
            #-----
            (r'(Meter event Measurement Error Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Measurement Error Detected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a measurement error).*', 'Meter event Measurement Error Detected')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 08/30/2021 04:00:11  Sequence number = 3256  User id = 0  Event argument = 00-00 '
        '3.21.1.173':    [
            #-----
            (r'(Meter event Nonvolatile Memory Failure Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Nonvolatile Memory Failure Detected).*', r'\1'),
            #-----
            (r'(Meter detected a nonvolatile memory failure).*', 'Meter event Nonvolatile Memory Failure Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'System Error cleared for meter 00:13:50:05:ff:11:2d:23.'
        '3.21.3.28':    [
            #-----
            (r'(System Error cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:0f:17:ba.'
        '3.21.3.79':    [
            #-----
            (r'(System Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a self check error.'
        '3.21.17.28':    [
            #-----
            (r'(Meter detected a self check error).', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a self check error.'
        '3.21.18.79':    [
            #-----
            (r'(Meter detected a self check error).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Detected end of voltage sag on meter 00:13:50:05:ff:32:66:22 on one or several phases. Duration: 21 cycles (less than a second), Min RMS Voltage: Phase A 118.1 V, Phase B 95.4 V, Phase C 120.3 V, RMS Current (at min voltage): Phase A 1.0 A, Phase B 0.2 A, Phase C 1.2 A'
        '3.21.38.223':  [
            #-----
            (r'(Detected end of voltage sag)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*Duration:.*, Min RMS Voltage: Phase A .*, Phase B .*, Phase C .*, RMS Current \(at min voltage\): Phase A .*, Phase B .*, Phase C .*', r'\1'), 
            #-----
            # Failsafe
            (r'(Detected end of voltage sag)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Detected end of voltage swell on meter 00:13:50:05:ff:1b:49:b0 on one or several phases. Duration: 5 cycles (less than a second), Max RMS Voltage: Phase A 111.2 V, Phase B 133.2 V, Phase C 216.1 V, RMS Current (at max voltage): Phase A 2.8 A, Phase B 1.3 A, Phase C 0.8 A'
        '3.21.38.248':  [
            #-----
            (r'(Detected end of voltage swell)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*Duration: .*, Min RMS Voltage: .*, RMS Current \(at min voltage\): .*', r'\1'), 
            #-----
            # Failsafe
            (r'(Detected end of voltage swell)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1')  
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Detected end of voltage sag on meter 00:13:50:03:ff:01:a6:36. Duration: 2 seconds, Min RMS Voltage: Phase A 227.9 V, RMS Current (at min voltage): Phase A 0.0 A'
        '3.21.43.223':  [
            #-----
            (r'(Detected end of voltage sag)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*Duration: .*, Min RMS Voltage: .*, RMS Current \(at min voltage\): .*', r'\1'), 
            #-----
            (r'(Detected end of voltage sag)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1'), # failsafe  
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Detected end of voltage swell on meter 00:13:50:05:ff:2d:2c:25. Duration: 195 seconds, Max RMS Voltage: Phase A 265.7 V, RMS Current (at max voltage): Phase A 0.0 A'
        '3.21.43.248':  [
            #-----
            (r'(Detected end of voltage swell)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*Duration: .*, Min RMS Voltage: .*, RMS Current \(at min voltage\): .*', r'\1'), 
            #-----
            (r'(Detected end of voltage swell)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?).*', r'\1'), # failsafe  
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Measurement Error cleared for meter 00:13:50:05:ff:07:c0:55.'
        '3.21.67.28':    [
            #-----
            (r'(Measurement Error.*cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Measurement Error Detected  Time event occurred on meter = 01/26/2020 19:47:00  Sequence number = 284  User id = 0  Event argument = 00-00'
        # 'Measurement Error (C1219 Table 3) occurred for meter 00:13:50:03:ff:06:d6:d5.'
        '3.21.67.79':    [
            #-----
            (r'(Measurement Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'(Meter event Measurement Error Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Measurement Error Detected).*', r'\1'),
            #-----
            (r'(Meter detected a measurement error).*', 'Meter event Measurement Error Detected')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'DSP Error cleared for meter 00:13:50:05:ff:0b:87:fc.'
        '3.21.82.28':    [
            #-----
            (r'(DSP Error cleared)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:05:ff:2c:f3:e2.'
        '3.21.82.79':    [
            #-----
            (r'(DSP Error\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1') 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Test Mode Stopped  Time event occurred on meter = 08/27/2021 10:04:04  Sequence number = 23834  User id = 1  Event argument = 00-00 '
        '3.22.12.243':   [
            #-----
            (r'(Meter event Test Mode Stopped)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Test Mode Stopped).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Test Mode Started  Time event occurred on meter = 08/27/2021 07:48:19  Sequence number = 765  User id = 1  Event argument = 00-00 '
        '3.22.19.242':   [
            #-----
            (r'(Meter event Test Mode Started)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Test Mode Started).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Device 00:13:50:05:ff:2e:17:7e has been determined to have exceeded the max allowable trap threshold, 20, within a certain time limit, 3600 seconds.'
        '3.23.17.139':   [
            #-----
            (r'(Device)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+).*(exceeded the max allowable trap threshold).*', r'\1 \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Access Point 00:13:50:08:ff:00:06:8e has lost connectivity with FHSS 900 MHz band.'
        '3.23.136.47':  [
            #-----
            (r'(Access Point)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(has lost connectivity with FHSS 900 MHz band.)', r'\1 \2') 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # Device: 00:13:50:05:ff:20:f5:ef Time: 2021-01-20T06:03:47.000-05:00 Failed Device: 00:13:50:05:ff:20:f5:ef Reason: Security public key mismatch Reboot Counter: 44 Refresh Counter: 0 Seconds since last reboot 6872803
        # 'NIC Link Layer Handshake Failed: Device: 00:13:50:05:ff:19:4e:6b, Rejected neighbor Mac ID: 00:13:50:05:ff:18:a3:52, Rejection Cause: invalid eblob signature'
        '3.23.136.85':  [
            #-----
            # Below, Device and Failed Device sometimes blank, sometime MAC-esque
            (r'Device\:?(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)? Time: .* Failed Device\:?(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)? (Reason: .*) Reboot Counter: .* Refresh Counter: .*', r'Device Failed: \1'), 
            #-----
            (r'(NIC Link Layer Handshake Failed): Device:(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+), Rejected neighbor Mac ID:(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+), (Rejection Cause: .*)', r'\1: \2')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter generated an energy polarity gyrbox call in event.'
        '3.25.17.3':   [
            #-----
            (r'(Meter generated an energy polarity gyrbox call in event).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Primary Power Down occurred for meter 00:13:50:05:ff:16:5f:61.'
        # 'Meter had a power outage. Unsafe power fail count = 32714'
        # 'Meter event Primary Power Down  Time event occurred on meter = 08/05/2021 16:59:27  Sequence number = 691  User id = 0  Event argument = 00-00'
        '3.26.0.47': [
            #-----
            (r'(Primary Power Down)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'(Meter had a power outage)\.?\s*(Unsafe power fail)\s*count\s*=\s*[0-9]+', r'\1 (\2)'), 
            #-----
            (r'^(Meter had a power outage).*$', r'\1'), #Fail proof, in case Unsafe power fail, count, etc., not found 
            #-----
            (r'(Meter event Primary Power Down)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Primary Power Down).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
        ],         
        
        #-------------------------
        # Examples
        # 'Primary Power Up occurred for meter 00:13:50:05:ff:22:2a:0c.'
        # 'Meter event Primary Power Up Time event occurred on meter = 08/14/2020 Sequence number = 343 User id = 0 Event argument = 00-00'
        '3.26.0.216':   [
            #-----
            (r'(Primary Power Up)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1'), 
            #-----
            (r'Meter event (Primary Power Up)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'Meter event (Primary Power Up).*', r'\1'),  #Fail proof, in case time, sequence, etc., not found 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter had a power outage. Unsafe power fail count = 13050'
        # Meter event Primary Power Down  Time event occurred on meter = 08/31/2021 20:07:18  Sequence number = 103  User id = 0  Event argument = 00-00 
        '3.26.17.185':  [
            #-----
            (r'(Meter had a power outage).\s*(Unsafe power fail) count = [0-9]+', r'\1 (\2)'), 
            #-----
            (r'(Meter event Primary Power Down)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Primary Power Down).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Primary Power Up  Time event occurred on meter = 08/31/2021 20:07:44  Sequence number = 104  User id = 0  Event argument = 00-00 '
        '3.26.17.216':  [
            #-----
            (r'Meter event (Primary Power Up)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'Meter event (Primary Power Up).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Under Voltage cleared (CA000400) for meter 00:13:50:05:ff:0b:88:ec.'
        # 'Under Voltage (CA000400) cleared for meter 00:13:50:05:ff:15:e8:3b.'
        # 'Under Voltage (Diagnostic 6) cleared for meter 00:13:50:05:ff:3f:87:07N/A.'
        '3.26.38.37':   [
            #-----
            # NOTE: Due to annoying N/A at end of some MAC-esque IDs, I have to specify length of numerical entries
            #       as [0-9a-zA-Z]{1,2}, instead of the more general [0-9a-zA-Z]+ found elsewhere!
            (r'(Under Voltage|Low Potential|Diag6 Condition)\s*(\scleared)?\s*(\s\(.*\))?\s*(\scleared)?\s*(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+(?:N/A)?\.?)', r'\1\3\2\4')
            #-----
        ],        
        
        #-------------------------
        # Examples
        # 'Low Potential (C1219 Table 3) occurred for meter 00:13:50:05:ff:1b:67:3b.'
        '3.26.38.47': [
            #-----
            (r'(Low Potential\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)', r'\1') 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Diag7 Condition cleared for meter 00:13:50:03:00:4d:41:fa.'
        # 'Over Voltage (Diagnostic 7) cleared for meter 00:13:50:05:ff:37:87:3dN/A.'
        '3.26.38.73':   [
            #-----
            # NOTE: Due to annoying N/A at end of some MAC-esque IDs, I have to specify length of numerical entries
            #       as [0-9a-zA-Z]{1,2}, instead of the more general [0-9a-zA-Z]+ found elsewhere!
            # NOTE: Also, the use of non-greedy (.*?) at beginning.
            #       If (.*) were used instead, results would be, e.g.
            #          'Over Voltage (Diagnostic 7) cleared for meter' instead of 'Over Voltage (Diagnostic 7) cleared'
            (r'(.*?)(?:(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+(?:N/A)?\.?)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Diag7: Over Voltage, Element A occurred for meter 03.34.0.400:13:50:02:00:0a:d7:98.'
        # 'Over Voltage (Diagnostic 7) occurred for meter 00:13:50:05:ff:41:8a:8f: Phase A.'
        # SEEMS RARE, BUT HAVE SEEN:
        # 'KV2c meter event Over Voltage Diagnostic flags:Phase A Voltage'
        # 'KV2c meter event Under Voltage Caution Diagnostic flags:Phase B Voltage'
        # 'KV2c meter event Received kWh Caution Diagnostic flags:Phase A Voltage '
        # 'KV2c meter event Received kWh Caution Cleared '
        '3.26.38.93':   [
            #-----
            (r'(KV2c meter event (?:(?:Over|Under) Voltage)|(?:Voltage Out of Tolerance)|(?:(?:Received|Delivered) kWh))(?:\s*Caution)?\s*(Diagnostic flags)\s*\:\s*(.*?)\s*$', r'\1 (\2 = \3)'), 
            #-----
            (r'(KV2c meter event (?:(?:Over|Under) Voltage)|(?:Voltage Out of Tolerance)|(?:(?:Received|Delivered) kWh))(?:\s*Caution)?\s*Cleared\s*$', r'\1 Cleared'), 
            #-----
            (r'(.*Over Voltage.*?)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)(.*?)\s*$', r'\1\2') 
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Under Voltage (CA000400) for meter 00:13:50:05:ff:0b:88:ec. Phase  C Voltage out of tolerance.'
        # 'Under Voltage (CA000400) occurred for meter 00:13:50:05:ff:15:e8:3b: Phase A.'
        # 'KV2c meter event Under Voltage Diagnostic flags:Phase A Voltage'
        # 'Diag6: Under Voltage, Element A occurred for meter 00:13:50:05:ff:0b:88:ec.'
        '3.26.38.150':   [
            #-----
            # NOTE: Due to annoying N/A at end of some MAC-esque IDs, I have to specify length of numerical entries
            #       as [0-9a-zA-Z]{1,2}, instead of the more general [0-9a-zA-Z]+ found elsewhere!
            # NOTE: Also, the use of non-greedy (.*?) at beginning.
            #       If (.*) were used instead, results would be, e.g.
            #          'Diag6: Under Voltage, Element A occurred for meter'
            #       instead of 
            #          'Diag6: Under Voltage, Element A'
            (r'(Under Voltage\s*(?:\(.*\))?)\s*(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\.\:]?)(\s?.*?)', r'\1\2'), 
            #-----
            (r'(Diag6.*?)(?:(?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+[\.\:]?)(\s?.*?)', r'\1\2'), 
            #-----
            (r'(KV2c meter event .*?)\s*(Diagnostic flags)\s*\:\s*(.*?)\s*$', r'\1 (\2 = \3)')
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Last Gasp - NIC power lost for device: 00:13:50:05:ff:0b:83:e4, Reboot Count: 12264, NIC timestamp: 2020-01-19T22:21:02.000-05:00, Received timestamp: 2020-01-19T22:21:04.353-05:00, Fail Reason: [0x49] LG_ZERO_X_DETECTOR ,LG_DIRECT_NOTIFICATION'
        '3.26.136.47':  [
            #-----
            (r'(Last Gasp - NIC power lost for device):(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+).*(Fail Reason: .*)$', r'\1, \2')
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Device 00:13:50:05:ff:0b:8a:18, Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_DISABLED, Detector State: EL_EVENT_POWER_FAIL_DETECT_NIC_ZX_DISABLED, Reboot Count: 108'
        '3.26.136.66':  [
            #-----
            (r'Device(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+), (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'NIC Power Restore Trap Received from device: 00:13:50:05:ff:0c:7e:93, Reboot Count: 61, NIC timestamp: 2020-01-27T09:19:56.000-05:00, Received Timestamp: 2020-01-27T09:19:57.204-05:00, Power Restoration Timestamp: 2020-01-27T09:19:42.000-05:00, State: 4,p'
        '3.26.136.216': [
            #-----
            (r'(NIC Power Restore Trap Received from device):(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter assumed to be disconnected has reported Load side voltage indicating a potential case of tamper.'
        '3.31.1.143':   [
            #-----
            (r'(Meter assumed to be disconnected has reported Load side voltage indicating a potential case of tamper).*', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter event Reverse Rotation Detected  Time event occurred on meter = 08/30/2021 04:25:38  Sequence number = 13124  User id = 0  Event argument = 00-00 '
        '3.33.1.219':   [
            #-----
            (r'(Meter event Reverse Rotation Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Reverse Rotation Detected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a Reverse Rotation).*', 'Meter event Reverse Rotation Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a Tamper Attempt.'
        # 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 08/28/2021 08:45:05  Sequence number = 72  User id = 0  Event argument = 00-00 '
        '3.33.1.257':   [
            #-----
            (r'(Meter event Tamper Attempt Suspected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Tamper Attempt Suspected).*', r'\1'), #Fail proof, in case time, sequence, etc., not found 
            #-----
            (r'(Meter detected a Tamper Attempt).*', 'Meter event Tamper Attempt Suspected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Cleared: Meter 00:13:50:05:ff:11:69:bb detected a high temperature condition. (C1219 Table 3)'
        '3.35.0.28': [
            #-----
            (r'(Cleared: Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(detected a high temperature condition)\.\s*(.*)', r'\1 \2 \3') 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter 00:13:50:05:ff:11:69:bb detected a high temperature condition. (C1219 Table 3)'
        # 'Meter's temperature threshold exceeded.'
        # 'Meter event AX Temp Threshold Exceeded  Time event occurred on meter = 02/19/2021 13:33:50  Sequence number = 5115  User id = 0  Event argument = 00-00 '
        # 'Meter event S4TemperatureThreshold  Time event occurred on meter = 05/25/2023 17:12:50  Sequence number = 45  User id = 0  Event argument = 00-00-00-00-00-00 '
        '3.35.0.40': [
            #-----
            (r'(Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(detected a high temperature condition)\.\s*(.*)', r'\1 \2 \3'), 
            #-----
            (r"(Meter's temperature threshold exceeded).", r'\1'), 
            #-----
            (r'(Meter event .*?)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event (?:AX Temp Threshold Exceeded|S4TemperatureThreshold)).*', r'\1'),
        ],  
        
        #-------------------------
        # Examples
        # 'Meter detected a clock error.'
        # 'Meter event Clock Error Detected  Time event occurred on meter = 04/18/2020 10:00:10  Sequence number = 332  User id = 0  Event argument = 00-00'
        '3.36.0.79': [
            #-----
            (r'(Meter event Clock Error Detected)\s*Time event occurred on meter\s*=.*\s*Sequence number\s*=.*\s*User id\s*=.*\s*Event argument\s*=.*', r'\1'), 
            #-----
            (r'(Meter event Clock Error Detected).*', r'\1'),
            #-----
            (r'(Meter detected a clock error).*', 'Meter event Clock Error Detected'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter detected a clock error.'
        '3.36.1.29': [
            #-----
            ('(Meter detected a clock error).', r'\1'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter 00:13:50:05:ff:2e:13:de, detected loss of time (C1219 Table 3)'
        '3.36.114.73':  [
            #-----
            (r'(Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+),\s*(detected loss of time .*)', r'\1 \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Ignoring Interval Read data for device 00:13:50:05:ff:1e:fa:a4 as it has time in the future 2069-05-02 18:51:00.0'
        '3.36.114.159': [
            #-----
            (r'(Ignoring (?:Interval|Register) Read data for device)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(as it has time in the future).*', r'\1 \2'), 
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'Meter 00:13:50:05:ff:14:c9:61 needs explicit time sync. Drift: 14391 s, Encountered Problems:  TS_ERR_LP_BX, TS_ERR_BIG_DRIFT [0x44], Meter_Time: 06:33:41'
        '3.36.136.73':  [
            #-----
            (r'(Meter)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)\s*(needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time.*', r'\1 \2 \3'), 
            #-----
        ], 
        
        #-------------------------
        # Examples
        # 'Error occurred when attempting to synch meter time with NIC time for device 00:13:50:05:ff:29:e1:20'
        # So, standard beg/end patterns should suffice
        '3.36.136.79':    [
            #-----
            (r'(Error occurred when attempting to synch meter time with NIC time for device)(?:\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+)', r'\1')
            #-----
        ],         
        
        #-------------------------
        # Examples
        # 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase B Voltage '
        # 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase B Voltage, Phase C Voltage '
        '3.38.1.139':    [
            #-----
            (r'(KV2c meter event Polarity, Cross Phase, Reverse Energy Flow) Diagnostic flags:.*', r'\1'), 
            #-----
        ]
    }
    #-------------------------
    return patterns_to_replace_by_typeid

In [6]:
save_dir_base = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex'
build_samples = False

In [7]:
# distinct_ids = AMIEndEvents.get_end_event_distinct_fields(
#     '2020-01-01', 
#     '2023-03-21', 
#     fields=['enddeviceeventtypeid']
# )
# distinct_ids.to_pickle(os.path.join(save_dir_base, 'distinct_ids.pkl'))
#-----
distinct_ids = pd.read_pickle(os.path.join(save_dir_base, 'distinct_ids.pkl'))
#-----
distinct_ids = natsorted(distinct_ids['enddeviceeventtypeid'].tolist())

In [8]:
distinct_ids

[None,
 '3.2.0.28',
 '3.2.0.85',
 '3.2.22.28',
 '3.2.22.150',
 '3.7.19.242',
 '3.7.19.243',
 '3.7.22.4',
 '3.7.22.19',
 '3.8.0.215',
 '3.9.83.159',
 '3.11.63.161',
 '3.12.0.257',
 '3.12.17.257',
 '3.12.48.28',
 '3.12.48.219',
 '3.12.93.28',
 '3.12.93.219',
 '3.12.136.38',
 '3.12.136.85',
 '3.18.1.199',
 '3.18.1.220',
 '3.18.72.28',
 '3.18.72.79',
 '3.18.72.85',
 '3.18.85.28',
 '3.18.85.79',
 '3.18.85.85',
 '3.18.92.28',
 '3.18.92.79',
 '3.18.92.85',
 '3.21.1.79',
 '3.21.1.173',
 '3.21.3.28',
 '3.21.3.79',
 '3.21.17.28',
 '3.21.18.79',
 '3.21.38.223',
 '3.21.38.248',
 '3.21.43.223',
 '3.21.43.248',
 '3.21.67.28',
 '3.21.67.79',
 '3.21.82.28',
 '3.21.82.79',
 '3.22.12.243',
 '3.22.19.242',
 '3.23.17.139',
 '3.23.136.47',
 '3.23.136.85',
 '3.25.17.3',
 '3.26.0.47',
 '3.26.0.216',
 '3.26.17.185',
 '3.26.17.216',
 '3.26.38.37',
 '3.26.38.47',
 '3.26.38.73',
 '3.26.38.93',
 '3.26.38.150',
 '3.26.136.47',
 '3.26.136.66',
 '3.26.136.216',
 '3.31.1.143',
 '3.33.1.219',
 '3.33.1.257',
 '3.35.0.2

In [9]:
if build_samples:
    build_sample_dfs(
        distinct_ids=distinct_ids,
        save_dir=os.path.join(save_dir_base, 'sample_dfs'), 
        years = [2020, 2021, 2022, 2023], 
        limit_per_month=10000, 
        conn_db=Utilities.get_athena_prod_aws_connection()
    )

In [10]:
sample_df_paths = Utilities.find_all_paths(
    base_dir=os.path.join(save_dir_base, 'sample_dfs'), 
    glob_pattern=r'df_*.pkl'
)
#-----
ids_with_paths_dict = {}
for path_i in sample_df_paths:
    name_i = Path(path_i).name
    name_i = re.sub(r'df_(.*).pkl', r'\1', name_i)
    name_i = name_i.replace('_', '.')
    assert(name_i not in ids_with_paths_dict.keys())
    ids_with_paths_dict[name_i] = path_i

In [11]:
# natsorted(ids_with_paths_dict.keys())

# ---------------------------------------------------------------------------------------------------

In [12]:
std_patterns_to_replace = get_std_patterns_to_replace_by_typeid_EXPLICIT()

# ---------------------------------------------------------------------------------------------------

# '3.2.0.28'

In [13]:
df_3_2_0_28 = pd.read_pickle(ids_with_paths_dict['3.2.0.28'])
#-----
print(f"shape[0] = {df_3_2_0_28.shape[0]}")
print(f"# Unique reasons = {df_3_2_0_28['reason'].nunique()}")
print(df_3_2_0_28['reason'].unique())

shape[0] = 1
# Unique reasons = 1
['Power Loss cleared for meter 00:13:50:05:ff:0d:dd:17']


In [14]:
curated_reasons = []
for reason_i in df_3_2_0_28['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.2.0.28']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Power Loss cleared


# '3.2.0.85'

In [15]:
df_3_2_0_85 = pd.read_pickle(ids_with_paths_dict['3.2.0.85'])
#-----
print(f"shape[0] = {df_3_2_0_85.shape[0]}")
print(f"# Unique reasons = {df_3_2_0_85['reason'].nunique()}")
print(df_3_2_0_85['reason'].unique())

shape[0] = 1
# Unique reasons = 1
['Power loss detected on meter 00:13:50:05:ff:0d:dd:17']


In [16]:
curated_reasons = []
for reason_i in df_3_2_0_85['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.2.0.85']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Power loss detected


# '3.2.22.28'

In [17]:
df_3_2_22_28 = pd.read_pickle(ids_with_paths_dict['3.2.22.28'])
#-----
print(f"shape[0] = {df_3_2_22_28.shape[0]}")
print(f"# Unique reasons = {df_3_2_22_28['reason'].nunique()}")
print(df_3_2_22_28['reason'].unique())

shape[0] = 82
# Unique reasons = 41
['Low Battery cleared for meter 00:13:50:02:00:0a:9e:1e.'
 'Low Battery cleared for meter 00:13:50:02:00:0a:a2:c6.'
 'Low Battery cleared for meter 00:13:50:05:ff:06:67:48.'
 'Low Battery cleared for meter 00:13:50:05:ff:1b:6d:70.'
 'Low Battery cleared for meter 00:13:50:05:ff:23:ad:78.'
 'Low Battery cleared for meter 00:13:50:02:00:0a:a8:e3.'
 'Low Battery cleared for meter 00:13:50:02:00:0a:a8:a6.'
 'Low Battery cleared for meter 00:13:50:05:ff:1e:72:94.'
 'Low Battery cleared for meter 00:13:50:05:ff:1f:f9:bf.'
 'Low Battery cleared for meter 00:13:50:02:00:0a:bd:72.'
 'Low Battery cleared for meter 00:13:50:01:01:60:ed:1e.'
 'Low Battery cleared for meter 00:13:50:02:00:0b:36:e4.'
 'Low Battery cleared for meter 00:13:50:ff:fe:03:4f:11.'
 'Low Battery cleared for meter 00:13:50:01:01:60:f0:f1.'
 'Low Battery cleared for meter 00:13:50:05:ff:1a:7f:93.'
 'Low Battery cleared for meter 00:13:50:05:ff:0b:98:cb.'
 'Low Battery cleared for meter 00:1

In [18]:
curated_reasons = []
for reason_i in df_3_2_22_28['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.2.22.28']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Low Battery cleared


# '3.2.22.150'

In [19]:
df_3_2_22_150 = pd.read_pickle(ids_with_paths_dict['3.2.22.150'])
#-----
print(f"shape[0] = {df_3_2_22_150.shape[0]}")
print(f"# Unique reasons = {df_3_2_22_150['reason'].nunique()}")
print(df_3_2_22_150['reason'].unique())

shape[0] = 92
# Unique reasons = 48
['Low Battery (C1219 Table 3) occurred for meter 00:13:50:02:00:0a:9e:1e.'
 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:05:ff:02:72:01.'
 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:02:00:0a:a2:c6.'
 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:05:ff:06:67:48.'
 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:05:ff:1b:6d:70.'
 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:02:00:0a:a8:e3.'
 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:02:00:0a:a8:a6.'
 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:01:01:5a:59:20.'
 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:01:01:60:ed:1e.'
 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:05:ff:1f:f9:bf.'
 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:05:ff:1e:61:86.'
 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:02:00:0a:bd:72.'
 'Low Battery (C1219 Table 3) occurred for meter 00:13:50:02:00:

In [20]:
curated_reasons = []
for reason_i in df_3_2_22_150['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.2.22.150']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Low Battery (C1219 Table 3)


# '3.7.19.242'

In [21]:
df_3_7_19_242 = pd.read_pickle(ids_with_paths_dict['3.7.19.242'])
#-----
print(f"shape[0] = {df_3_7_19_242.shape[0]}")
print(f"# Unique reasons = {df_3_7_19_242['reason'].nunique()}")
print(df_3_7_19_242['reason'].unique())

shape[0] = 1449426
# Unique reasons = 10010
['Meter event Test Mode Started  Time event occurred on meter = 01/03/2020 10:52:59  Sequence number = 10  User id = 1  Event argument = 00-00'
 'Meter event Test Mode Started  Time event occurred on meter = 01/03/2020 19:38:56  Sequence number = 119  User id = 1  Event argument = 00-00'
 'Meter event Test Mode Started  Time event occurred on meter = 01/03/2020 16:43:05  Sequence number = 3025  User id = 1  Event argument = 00-00'
 ... 'Test Mode Started occurred for meter 00:13:50:05:ff:02:6e:cc.'
 'Test Mode Started occurred for meter 00:13:50:05:ff:02:88:29.'
 'Test Mode Started occurred for meter 00:13:50:03:ff:06:d1:d5.']


In [22]:
curated_reasons = []
for reason_i in df_3_7_19_242['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.7.19.242']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter is currently in test mode
Test Mode Started
Meter event Test Mode Started


# '3.7.19.243'

In [23]:
df_3_7_19_243 = pd.read_pickle(ids_with_paths_dict['3.7.19.243'])
#-----
print(f"shape[0] = {df_3_7_19_243.shape[0]}")
print(f"# Unique reasons = {df_3_7_19_243['reason'].nunique()}")
print(df_3_7_19_243['reason'].unique())

shape[0] = 1447615
# Unique reasons = 8809
['Meter event Test Mode Stopped  Time event occurred on meter = 01/20/2020 14:05:56  Sequence number = 61  User id = 1  Event argument = 00-00'
 'Meter event Test Mode Stopped  Time event occurred on meter = 01/30/2020 13:56:48  Sequence number = 148  User id = 1  Event argument = 00-00'
 'Meter event Test Mode Stopped  Time event occurred on meter = 01/04/2020 16:45:31  Sequence number = 3043  User id = 1  Event argument = 00-00'
 ... 'Test Mode Stopped occurred for meter 00:13:50:03:ff:06:3f:51.'
 'Test Mode Stopped occurred for meter 00:13:50:03:ff:04:65:d8.'
 'Test Mode Stopped occurred for meter 00:13:50:03:ff:06:70:47.']


In [24]:
curated_reasons = []
for reason_i in df_3_7_19_243['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.7.19.243']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter event Test Mode Stopped
Test Mode Stopped


# '3.7.22.4'

In [25]:
df_3_7_22_4 = pd.read_pickle(ids_with_paths_dict['3.7.22.4'])
#-----
print(f"shape[0] = {df_3_7_22_4.shape[0]}")
print(f"# Unique reasons = {df_3_7_22_4['reason'].nunique()}")
print(df_3_7_22_4['reason'].unique())

shape[0] = 15881
# Unique reasons = 15869
['NIC operating on backup battery: 00:13:50:ff:fe:70:96:b7, Reboot Count: 6, NIC timestamp: 2021-01-09T23:51:00.000-05:00, Received timestamp: 2021-01-09T23:51:01.816-05:00'
 'NIC operating on backup battery: 00:13:50:ff:fe:70:96:63, Reboot Count: 8, NIC timestamp: 2021-01-09T08:10:15.000-05:00, Received timestamp: 2021-01-09T08:10:17.475-05:00'
 'NIC operating on backup battery: 00:13:50:ff:fe:70:96:ac, Reboot Count: 6, NIC timestamp: 2021-01-14T10:17:54.000-05:00, Received timestamp: 2021-01-14T10:17:55.728-05:00'
 ...
 'NIC operating on backup battery: 00:13:50:ff:fe:12:1a:0e, Reboot Count: 25, NIC timestamp: 2023-03-02T18:42:55.000-06:00, Received timestamp: 2023-03-02T18:42:56.419-06:00'
 'NIC operating on backup battery: 00:13:50:ff:fe:12:1a:0e, Reboot Count: 25, NIC timestamp: 2023-03-01T06:41:24.000-06:00, Received timestamp: 2023-03-01T06:41:26.031-06:00'
 'NIC operating on backup battery: 00:13:50:ff:fe:12:1a:0e, Reboot Count: 25, NIC

In [26]:
curated_reasons = []
for reason_i in df_3_7_22_4['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.7.22.4']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

NIC operating on backup battery


# '3.7.22.19'

In [27]:
df_3_7_22_19 = pd.read_pickle(ids_with_paths_dict['3.7.22.19'])
#-----
print(f"shape[0] = {df_3_7_22_19.shape[0]}")
print(f"# Unique reasons = {df_3_7_22_19['reason'].nunique()}")
print(df_3_7_22_19['reason'].unique())

shape[0] = 59047
# Unique reasons = 1
['NIC backup battery inactive']


In [28]:
curated_reasons = []
for reason_i in df_3_7_22_19['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.7.22.19']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

NIC backup battery inactive


# '3.8.0.215'

In [29]:
df_3_8_0_215 = pd.read_pickle(ids_with_paths_dict['3.8.0.215'])
#-----
print(f"shape[0] = {df_3_8_0_215.shape[0]}")
print(f"# Unique reasons = {df_3_8_0_215['reason'].nunique()}")
print(df_3_8_0_215['reason'].unique())

shape[0] = 2700000
# Unique reasons = 262388
['Demand Reset occurred for meter 00:13:50:01:00:0d:a3:2d.'
 'Demand Reset occurred for meter 00:13:50:01:00:b7:b3:77.'
 'Demand Reset occurred for meter 00:13:50:02:00:97:f3:d0.' ...
 'Demand Reset occurred for meter 00:13:50:05:ff:3a:6d:8e.'
 'Demand Reset occurred for meter 00:13:50:05:ff:3a:6d:7b.'
 'Demand Reset occurred for meter 00:13:50:05:ff:4b:7b:98.']


In [30]:
curated_reasons = []
for reason_i in df_3_8_0_215['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.8.0.215']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Demand Reset


# '3.9.83.159'

In [31]:
df_3_9_83_159 = pd.read_pickle(ids_with_paths_dict['3.9.83.159'])
#-----
print(f"shape[0] = {df_3_9_83_159.shape[0]}")
print(f"# Unique reasons = {df_3_9_83_159['reason'].nunique()}")
print(df_3_9_83_159['reason'].unique())

shape[0] = 27497
# Unique reasons = 27007
['Meter Program Seal mismatch for Device [Device ID, MAC Id] = [V9D585888649PMNA1, 00:13:50:05:ff:26:2f:5f] - program seal = 6a622d2cb1b04fb4b869b90103eac1842a0e2169 vs. UIQ program seal = 7298b94285241f59f4bc491277f5a0df61365ac7 at 2021-01-16T13:36:22.904-05:00.'
 'Meter Program Seal mismatch for Device [Device ID, MAC Id] = [V9D585888649PMNA1, 00:13:50:05:ff:26:2f:5f] - program seal = 6a622d2cb1b04fb4b869b90103eac1842a0e2169 vs. UIQ program seal = 7298b94285241f59f4bc491277f5a0df61365ac7 at 2021-01-16T13:36:03.750-05:00.'
 'Meter Program Seal mismatch for Device [Device ID, MAC Id] = [1ND785901154NMD06, 00:13:50:05:ff:25:52:90] - program seal = d503282c96d067dab2104cb4742107161c4c69b1 vs. UIQ program seal = 05e157fd3041d59efceb489a9aba694ecf23f259 at 2021-01-13T16:14:19.468-05:00.'
 ...
 'Meter Program Seal mismatch for Device [Device ID, MAC Id] = [3KD784133129NM008, 00:13:50:05:ff:20:ea:e9] - program seal = eda212e784aaa25b1f0a76705f1d62164

In [32]:
curated_reasons = []
for reason_i in df_3_9_83_159['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.9.83.159']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter Program Seal mismatch for Device


# '3.11.63.161'

In [33]:
df_3_11_63_161 = pd.read_pickle(ids_with_paths_dict['3.11.63.161'])
#-----
print(f"shape[0] = {df_3_11_63_161.shape[0]}")
print(f"# Unique reasons = {df_3_11_63_161['reason'].nunique()}")
print(df_3_11_63_161['reason'].unique())

shape[0] = 72980
# Unique reasons = 62937
['Requested operation JOB_OP_TYPE_ARB_METER_COMMAND could not be applied to the given device type and firmware version. Device: 00:13:50:05:ff:2d:e0:5f, DeviceType: DID_SUBTYPE_I210CRD_HAN, Firmware Version: 3.12.5c'
 'Requested operation JOB_OP_TYPE_ARB_METER_COMMAND could not be applied to the given device type and firmware version. Device: 00:13:50:05:ff:2d:e3:b7, DeviceType: DID_SUBTYPE_I210CRD_HAN, Firmware Version: 3.12.5c'
 'Requested operation JOB_OP_TYPE_ARB_METER_COMMAND could not be applied to the given device type and firmware version. Device: 00:13:50:05:ff:2e:28:ab, DeviceType: DID_SUBTYPE_I210CRD_HAN, Firmware Version: 3.12.5c'
 ...
 'Requested operation JOB_OP_NEW_DATA_READ could not be applied to the given device type and firmware version. Device: 00:13:50:05:ff:3e:15:38, DeviceType: DID_SUBTYPE_I210CRD_HAN, Firmware Version: 3.12.5c'
 'Requested operation JOB_OP_PROVISION_GET_STATUS could not be applied to the given device typ

In [34]:
curated_reasons = []
for reason_i in df_3_11_63_161['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.11.63.161']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Requested operation could not be applied: JOB_OP_TYPE_DEMAND_RESET 
Requested operation could not be applied: JOB_OP_TYPE_ARB_METER_COMMAND 
Requested operation could not be applied: JOB_OP_PROVISION_CONNECT 
Requested operation could not be applied: JOB_OP_LP_READ 
Requested operation could not be applied: JOB_OP_NEW_DATA_READ 
Requested operation could not be applied: JOB_OP_REGISTER_SELF_READ 
Requested operation could not be applied: JOB_OP_REGISTER_CURR_READ 
Requested operation could not be applied: JOB_OP_TYPE_DEV_PROG_READ 
Requested operation could not be applied: JOB_OP_PROVISION_GET_STATUS 
Requested operation could not be applied: JOB_OP_PROVISION_DISCONNECT 
Requested operation could not be applied: JOB_OP_NIC_NEW_DATA_READ 


# '3.12.0.257'

In [35]:
df_3_12_0_257_1 = pd.read_pickle(ids_with_paths_dict['3.12.0.257'])
df_3_12_0_257_2 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_TX\sample_dfs\df_3_12_0_257.pkl')
df_3_12_0_257 = pd.concat([df_3_12_0_257_1, df_3_12_0_257_2])
#-----
print(f"shape[0] = {df_3_12_0_257.shape[0]}")
print(f"# Unique reasons = {df_3_12_0_257['reason'].nunique()}")
print(df_3_12_0_257['reason'].unique())

shape[0] = 1021729
# Unique reasons = 36463
['Meter event Tamper Attempt Suspected  Time event occurred on meter = 01/20/2020 10:11:26  Sequence number = 152  User id = 0  Event argument = 00-00'
 'Meter detected a Tamper Attempt.'
 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 01/20/2020 12:24:49  Sequence number = 158  User id = 0  Event argument = 00-00'
 ...
 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 05/03/2023 16:09:59  Sequence number = 161  User id = 0  Event argument = 00-00 '
 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 05/03/2023 16:42:02  Sequence number = 549  User id = 0  Event argument = 00-00 '
 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 05/03/2023 17:19:37  Sequence number = 7275  User id = 0  Event argument = 00-00 ']


In [36]:
curated_reasons = []
for reason_i in df_3_12_0_257['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.12.0.257']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter detected a Tamper Attempt
Meter event Tamper Attempt Suspected


# '3.12.17.257'

In [37]:
df_3_12_17_257 = pd.read_pickle(ids_with_paths_dict['3.12.17.257'])
#-----
print(f"shape[0] = {df_3_12_17_257.shape[0]}")
print(f"# Unique reasons = {df_3_12_17_257['reason'].nunique()}")
print(df_3_12_17_257['reason'].unique())

shape[0] = 19389
# Unique reasons = 7183
['Tamper (Meter Inversion) detected on meter 00:13:50:05:ff:10:96:90.'
 'Tamper (Meter Inversion) detected on meter 00:13:50:05:ff:18:cb:30.'
 'Tamper (Meter Inversion) detected on meter 00:13:50:05:ff:0e:a4:8f.' ...
 'Tamper (Meter Inversion) detected on meter 00:13:50:05:ff:4e:40:56.'
 'Tamper (Meter Inversion) detected on meter 00:13:50:05:ff:22:02:d7.'
 'Tamper (Meter Inversion) detected on meter 00:13:50:05:ff:3c:9d:69.']


In [38]:
curated_reasons = []
for reason_i in df_3_12_17_257['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.12.17.257']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Tamper (Meter Inversion) detected


# '3.12.48.28'

In [39]:
df_3_12_48_28 = pd.read_pickle(ids_with_paths_dict['3.12.48.28'])
#-----
print(f"shape[0] = {df_3_12_48_28.shape[0]}")
print(f"# Unique reasons = {df_3_12_48_28['reason'].nunique()}")
print(df_3_12_48_28['reason'].unique())

shape[0] = 2432609
# Unique reasons = 20338
['Diag1 Condition cleared for meter 00:13:50:05:ff:04:55:a1.'
 'Diag1 Condition cleared for meter 00:13:50:05:ff:1b:3f:03.'
 'Diag1 Condition cleared for meter 00:13:50:05:ff:0f:74:ac.' ...
 'Diag1 Condition cleared for meter 00:13:50:05:ff:0b:83:22.'
 'Diag1 Condition cleared for meter 00:13:50:05:ff:24:a1:85.'
 'Diag1 Condition cleared for meter 00:13:50:05:ff:2c:ee:e8.']


In [40]:
curated_reasons = []
for reason_i in df_3_12_48_28['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.12.48.28']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Reverse energy cleared
Diag1 Condition cleared


# '3.12.48.219'

In [41]:
df_3_12_48_219 = pd.read_pickle(ids_with_paths_dict['3.12.48.219'])
#-----
print(f"shape[0] = {df_3_12_48_219.shape[0]}")
print(f"# Unique reasons = {df_3_12_48_219['reason'].nunique()}")
print(df_3_12_48_219['reason'].unique())

shape[0] = 2459882
# Unique reasons = 27622
['KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase C Voltage'
 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase A Current'
 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase B Voltage, Phase C Voltage'
 ...
 'Diag1: Polarity, Cross Phase, Reverse Energy Flow occurred for meter 00:13:50:05:ff:2c:f6:ef. Angle out of tolerance [Voltage - Phase  C].'
 'Diag1: Polarity, Cross Phase, Reverse Energy Flow occurred for meter 00:13:50:05:ff:24:a6:88. Angle out of tolerance [Voltage - Phase  C].'
 'Diag1: Polarity, Cross Phase, Reverse Energy Flow occurred for meter 00:13:50:05:ff:34:8d:e9. Angle out of tolerance [Voltage - Phase  B][Current - Phase  B].']


In [42]:
curated_reasons = []
for reason_i in df_3_12_48_219['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.12.48.219']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Service disconnect operation occurred with status: Open succeeded
Service disconnect operation occurred with status: Close succeeded 
Service disconnect operation occurred with status: Open started 
Service disconnect operation occurred with status: Close started 
Diag1: Polarity, Cross Phase, Reverse Energy Flow: Angle out of tolerance.
Reverse energy (C1219 Table 3) 
Service disconnect operation occurred with status: Close started
KV2c meter event Polarity, Cross Phase, Reverse Energy Flow


# '3.12.93.28'

In [43]:
df_3_12_93_28 = pd.read_pickle(ids_with_paths_dict['3.12.93.28'])
#-----
print(f"shape[0] = {df_3_12_93_28.shape[0]}")
print(f"# Unique reasons = {df_3_12_93_28['reason'].nunique()}")
print(df_3_12_93_28['reason'].unique())

shape[0] = 3436
# Unique reasons = 3172
['Cleared: Meter00:13:50:05:ff:12:b4:8d, cleared tamper detection (C1219 Table 3)'
 'Cleared: Meter00:13:50:05:ff:13:63:20, cleared tamper detection (C1219 Table 3)'
 'Cleared: Meter00:13:50:05:ff:07:88:4b, cleared tamper detection (C1219 Table 3)'
 ...
 'Cleared: Meter00:13:50:05:ff:09:33:03, cleared tamper detection (C1219 Table 3)'
 'Cleared: Meter00:13:50:03:ff:01:ba:28, cleared tamper detection (C1219 Table 3)'
 'Cleared: Meter00:13:50:03:ff:03:58:7f, cleared tamper detection (C1219 Table 3)']


In [44]:
curated_reasons = []
for reason_i in df_3_12_93_28['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.12.93.28']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Cleared: Meter cleared tamper detection (C1219 Table 3)


# '3.12.93.219'

In [45]:
df_3_12_93_219 = pd.read_pickle(ids_with_paths_dict['3.12.93.219'])
#-----
print(f"shape[0] = {df_3_12_93_219.shape[0]}")
print(f"# Unique reasons = {df_3_12_93_219['reason'].nunique()}")
print(df_3_12_93_219['reason'].unique())

shape[0] = 3900000
# Unique reasons = 426463
['Meter event Reverse Rotation Detected  Time event occurred on meter = 01/31/2020 15:58:24  Sequence number = 16908  User id = 0  Event argument = 00-00'
 'Meter detected a Reverse Rotation.'
 'Meter event Reverse Rotation Detected  Time event occurred on meter = 01/31/2020 12:55:31  Sequence number = 36052  User id = 0  Event argument = 00-00'
 ...
 'Meter event Reverse Rotation Detected  Time event occurred on meter = 03/09/2023 09:23:03  Sequence number = 1965  User id = 0  Event argument = 00-00 '
 'Meter event Reverse Rotation Detected  Time event occurred on meter = 03/09/2023 09:24:39  Sequence number = 3636  User id = 0  Event argument = 00-00 '
 'Meter event Reverse Rotation Detected  Time event occurred on meter = 03/09/2023 10:06:33  Sequence number = 596  User id = 0  Event argument = 00-00 ']


In [46]:
curated_reasons = []
for reason_i in df_3_12_93_219['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.12.93.219']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter detected tampering (C1219 Table 3)
Meter event Reverse Rotation Detected
Meter event Demand Reset Occurred


# '3.12.136.38'

In [47]:
df_3_12_136_38 = pd.read_pickle(ids_with_paths_dict['3.12.136.38'])
#-----
print(f"shape[0] = {df_3_12_136_38.shape[0]}")
print(f"# Unique reasons = {df_3_12_136_38['reason'].nunique()}")
print(df_3_12_136_38['reason'].unique())

shape[0] = 85383
# Unique reasons = 58
['A NET_MGMT command was sent from fdc9:ccbe:52c0:cbc0:a6ba:99ff:fe12:1b0e with a key that has insufficient privileges to execute it. ID: 207 WRITE SUBID: 65535 ASSOC_ID: 768'
 'A NET_MGMT command was sent from fdc9:ccbe:52c0:cbc0:250:56ff:feb5:91c7 with a key that has insufficient privileges to execute it. ID: 53 READ SUBID: 65535 ASSOC_ID: 8448'
 'A NET_MGMT command was sent from fdc9:ccbe:52c0:cbc0:a6ba:99ff:fe12:1b0e with a key that has insufficient privileges to execute it. ID: 208 WRITE SUBID: 65535 ASSOC_ID: 768'
 'A NET_MGMT command was sent from fd37:ec90:20c2:5c58:250:56ff:feb5:2d1f with a key that has insufficient privileges to execute it. ID: 53 READ SUBID: 65535 ASSOC_ID: 8448'
 'A NET_MGMT command was sent from fd37:ec90:20c2:5c58:226:55ff:fe2b:fd3e with a key that has insufficient privileges to execute it. ID: 312 WRITE SUBID: 65535 ASSOC_ID: 768'
 'A NET_MGMT command was sent from fd37:ec90:20c2:5c58:226:55ff:fe2b:fd3e with a key t

In [48]:
curated_reasons = []
for reason_i in df_3_12_136_38['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.12.136.38']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

NET_MGMT command was sent with a key that has insufficient privileges: WRITE SUBID: 65535 ASSOC_ID: 8960
NET_MGMT command was sent with a key that has insufficient privileges: WRITE SUBID: 1 ASSOC_ID: 52894
NET_MGMT command was sent with a key that has insufficient privileges: WRITE SUBID: 1 ASSOC_ID: 27367
NET_MGMT command was sent with a key that has insufficient privileges: READ SUBID: 65535 ASSOC_ID: 256
NET_MGMT command was sent with a key that has insufficient privileges: READ SUBID: 65535 ASSOC_ID: 20036
NET_MGMT command was sent with a key that has insufficient privileges: WRITE SUBID: 65535 ASSOC_ID: 768
NET_MGMT command was sent with a key that has insufficient privileges: WRITE SUBID: 38769 ASSOC_ID: 1542
NET_MGMT command was sent with a key that has insufficient privileges: READ SUBID: 65535 ASSOC_ID: 8448
NET_MGMT command was sent with a key that has insufficient privileges: READ SUBID: 1164 ASSOC_ID: 27412
NET_MGMT command was sent with a key that has insufficient privile

# '3.12.136.85'

In [49]:
df_3_12_136_85 = pd.read_pickle(ids_with_paths_dict['3.12.136.85'])
#-----
print(f"shape[0] = {df_3_12_136_85.shape[0]}")
print(f"# Unique reasons = {df_3_12_136_85['reason'].nunique()}")
print(df_3_12_136_85['reason'].unique())

shape[0] = 2700000
# Unique reasons = 105
['NET_MGMT command failed consecutively for 1 times for fd37:ec90:20c2:5c58:250:56ff:feb5:1010. WRITE'
 'NET_MGMT command failed consecutively for 1 times for fdc9:ccbe:52c0:d3a0:250:56ff:feb5:ec3c. WRITE'
 'failed consecutively for 1 times for 0:0:0:0:0:0:0:0.'
 'Secure association operation failed consecutively for 1 times for fdc9:ccbe:52c0:d3a0:250:56ff:feb5:ec3c. 58753'
 'NET_MGMT command failed consecutively for 1 times for fdc9:ccbe:52c0:cbe0:250:56ff:feb5:5b32. WRITE'
 'Secure association operation failed consecutively for 1 times for fdc9:ccbe:52c0:d3a0:250:56ff:feb5:ec3c. 40991'
 'Secure association operation failed consecutively for 1 times for fdc9:ccbe:52c0:cbe0:250:56ff:feb5:5b32. 26035'
 'NET_MGMT command failed consecutively for 1 times for fdc9:ccbe:52c0:d3a0:250:56ff:feb5:ad6b. WRITE'
 'Secure association operation failed consecutively for 1 times for fd37:ec90:20c2:5c58:250:56ff:feb5:1010. 57781'
 'Certificate "insert" operat

In [50]:
curated_reasons = []
for reason_i in df_3_12_136_85['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.12.136.85']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Secure association operation failed consecutively for 1 or more times
NET_MGMT command failed consecutively for 1 or more times
N/A failed consecutively for 1 or more times
Certificate "insert" operation failed consecutively for 1 or more times


# '3.18.1.199'

In [51]:
df_3_18_1_199 = pd.read_pickle(ids_with_paths_dict['3.18.1.199'])
#-----
print(f"shape[0] = {df_3_18_1_199.shape[0]}")
print(f"# Unique reasons = {df_3_18_1_199['reason'].nunique()}")
print(df_3_18_1_199['reason'].unique())

shape[0] = 1
# Unique reasons = 1
['Meter detected a RAM failure.']


In [52]:
curated_reasons = []
for reason_i in df_3_18_1_199['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.18.1.199']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter detected a RAM failure


# '3.18.1.220'

In [53]:
df_3_18_1_220 = pd.read_pickle(ids_with_paths_dict['3.18.1.220'])
#-----
print(f"shape[0] = {df_3_18_1_220.shape[0]}")
print(f"# Unique reasons = {df_3_18_1_220['reason'].nunique()}")
print(df_3_18_1_220['reason'].unique())

shape[0] = 21
# Unique reasons = 1
['Meter detected a ROM failure.']


In [54]:
curated_reasons = []
for reason_i in df_3_18_1_220['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.18.1.220']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter detected a ROM failure


# '3.18.72.28'

In [55]:
df_3_18_72_28 = pd.read_pickle(ids_with_paths_dict['3.18.72.28'])
#-----
print(f"shape[0] = {df_3_18_72_28.shape[0]}")
print(f"# Unique reasons = {df_3_18_72_28['reason'].nunique()}")
print(df_3_18_72_28['reason'].unique())

shape[0] = 3942
# Unique reasons = 1728
['NVRAM Error cleared for meter 00:13:50:05:ff:1b:b6:d0.'
 'NVRAM Error cleared for meter 00:13:50:05:ff:1c:5c:8b.'
 'NVRAM Error cleared for meter 00:13:50:05:ff:24:d7:d7.' ...
 'NVRAM Error cleared for meter 00:13:50:05:ff:41:40:4e.'
 'NVRAM Error cleared for meter 00:13:50:05:ff:00:46:a7.'
 'NVRAM Error cleared for meter 00:13:50:03:ff:02:a8:3a.']


In [56]:
curated_reasons = []
for reason_i in df_3_18_72_28['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.18.72.28']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

NVRAM Error cleared


# '3.18.72.79'

In [57]:
df_3_18_72_79 = pd.read_pickle(ids_with_paths_dict['3.18.72.79'])
#-----
print(f"shape[0] = {df_3_18_72_79.shape[0]}")
print(f"# Unique reasons = {df_3_18_72_79['reason'].nunique()}")
print(df_3_18_72_79['reason'].unique())

shape[0] = 6143
# Unique reasons = 3524
['NVRAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:11:10:cc.'
 'NVRAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:10:fd:3b.'
 'NVRAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1d:77:d9.'
 ...
 'NVRAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:04:8d:47.'
 'NVRAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:04:8d:64.'
 'NVRAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:00:46:a7.']


In [58]:
curated_reasons = []
for reason_i in df_3_18_72_79['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.18.72.79']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

NVRAM Error (C1219 Table 3) 
NVRAM Error (C1219 Table 3: Er000200) 


# '3.18.72.85'

In [59]:
df_3_18_72_85 = pd.read_pickle(ids_with_paths_dict['3.18.72.85'])
#-----
print(f"shape[0] = {df_3_18_72_85.shape[0]}")
print(f"# Unique reasons = {df_3_18_72_85['reason'].nunique()}")
print(df_3_18_72_85['reason'].unique())

shape[0] = 1815480
# Unique reasons = 1638510
['Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 01/16/2020 15:00:11  Sequence number = 1339  User id = 0  Event argument = 00-00'
 'Meter detected a nonvolatile memory failure.'
 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 01/16/2020 15:52:15  Sequence number = 33574  User id = 0  Event argument = 00-00'
 ...
 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 07/03/2023 18:37:09  Sequence number = 31521  User id = 0  Event argument = 00-00'
 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 9/19/2023 4:21:16 PM  Sequence number = 16663  User id = 0  Event argument = 00-00'
 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 12/11/2023 01:17:41  Sequence number = 17951  User id = 44624  Event argument = FF-00']


In [60]:
curated_reasons = []
for reason_i in df_3_18_72_85['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.18.72.85']
    )
#     if curated_reason_i=='Meter event ':
#         print(reason_i)
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*sorted(curated_reasons), sep='\n')

KV2c meter event Received kWh Caution  (Diagnostic flags = Phase A Voltage)
Meter event Demand Reset Occurred  
Meter event Nonvolatile Memory Failure Detected
Meter event Nonvolatile Memory Failure Detected  
Meter event Reset List Pointers  
Reason is null, ID=3.18.72.85
Universal event Unsupported Event(41) with priority Alarm


# '3.18.85.28'

In [61]:
df_3_18_85_28 = pd.read_pickle(ids_with_paths_dict['3.18.85.28'])
#-----
print(f"shape[0] = {df_3_18_85_28.shape[0]}")
print(f"# Unique reasons = {df_3_18_85_28['reason'].nunique()}")
print(df_3_18_85_28['reason'].unique())

shape[0] = 1
# Unique reasons = 1
['RAM Error cleared for meter 00:13:50:05:ff:0d:dd:17.']


In [62]:
curated_reasons = []
for reason_i in df_3_18_85_28['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.18.85.28']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

RAM Error cleared


# '3.18.85.79'

In [63]:
df_3_18_85_79 = pd.read_pickle(ids_with_paths_dict['3.18.85.79'])
#-----
print(f"shape[0] = {df_3_18_85_79.shape[0]}")
print(f"# Unique reasons = {df_3_18_85_79['reason'].nunique()}")
print(df_3_18_85_79['reason'].unique())

shape[0] = 1
# Unique reasons = 1
['RAM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:0d:dd:17.']


In [64]:
curated_reasons = []
for reason_i in df_3_18_85_79['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.18.85.79']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

RAM Error (C1219 Table 3)


# '3.18.85.85'

In [65]:
df_3_18_85_85 = pd.read_pickle(ids_with_paths_dict['3.18.85.85'])
#-----
print(f"shape[0] = {df_3_18_85_85.shape[0]}")
print(f"# Unique reasons = {df_3_18_85_85['reason'].nunique()}")
print(df_3_18_85_85['reason'].unique())

shape[0] = 361
# Unique reasons = 5
['Meter detected a RAM failure.'
 'Meter event Ram Failure Detected  Time event occurred on meter = 7/27/2021 12:45:53 PM  Sequence number = 89  User id = 34891  Event argument = 7D-75'
 'Meter event Ram Failure Detected  Time event occurred on meter = 12/27/2021 07:04:06  Sequence number = 195  User id = 0  Event argument = 00-00 '
 'Meter event Ram Failure Detected  Time event occurred on meter = 2/6/2022 5:06:08 PM  Sequence number = 345  User id = 34891  Event argument = 7D-75'
 'Meter event Ram Failure Detected  Time event occurred on meter = 10/02/2022 20:51:19  Sequence number = 141  User id = 0  Event argument = 00-00 ']


In [66]:
curated_reasons = []
for reason_i in df_3_18_85_85['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.18.85.85']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter event Ram Failure Detected


# '3.18.92.28'

In [67]:
df_3_18_92_28 = pd.read_pickle(ids_with_paths_dict['3.18.92.28'])
#-----
print(f"shape[0] = {df_3_18_92_28.shape[0]}")
print(f"# Unique reasons = {df_3_18_92_28['reason'].nunique()}")
print(df_3_18_92_28['reason'].unique())

shape[0] = 21
# Unique reasons = 19
['ROM Error cleared for meter 00:13:50:05:ff:1f:ba:d0.'
 'ROM Error cleared for meter 00:13:50:05:ff:0f:50:01.'
 'ROM Error cleared for meter 00:13:50:05:ff:1b:6e:54.'
 'ROM Error cleared for meter 00:13:50:05:ff:0f:6e:8e.'
 'ROM Error cleared for meter 00:13:50:05:ff:0a:e3:83.'
 'ROM Error cleared for meter 00:13:50:05:ff:1b:35:ed.'
 'ROM Error cleared for meter 00:13:50:03:ff:06:22:d6.'
 'ROM Error cleared for meter 00:13:50:05:ff:1b:27:df.'
 'ROM Error cleared for meter 00:13:50:05:ff:0f:0e:34.'
 'ROM Error cleared for meter 00:13:50:05:ff:1f:c2:93.'
 'ROM Error cleared for meter 00:13:50:05:ff:0d:dd:17.'
 'ROM Error cleared for meter 00:13:50:05:ff:1b:1c:b1.'
 'ROM Error cleared for meter 00:13:50:05:ff:11:7c:3e.'
 'ROM Error cleared for meter 00:13:50:03:ff:06:0d:f6.'
 'ROM Error cleared for meter 00:13:50:05:ff:2c:f6:f9.'
 'ROM Error cleared for meter 00:13:50:03:ff:06:78:77.'
 'ROM Error cleared for meter 00:13:50:05:ff:02:7f:04.'
 'ROM Error 

In [68]:
curated_reasons = []
for reason_i in df_3_18_92_28['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.18.92.28']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

ROM Error cleared


# '3.18.92.79'

In [69]:
df_3_18_92_79 = pd.read_pickle(ids_with_paths_dict['3.18.92.79'])
#-----
print(f"shape[0] = {df_3_18_92_79.shape[0]}")
print(f"# Unique reasons = {df_3_18_92_79['reason'].nunique()}")
print(df_3_18_92_79['reason'].unique())

shape[0] = 29
# Unique reasons = 26
['ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1f:ba:d0.'
 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1b:35:ed.'
 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:0f:6e:8e.'
 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1b:3e:4e.'
 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1b:6e:54.'
 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:04:1a:66.'
 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1b:27:df.'
 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:0f:6e:d0.'
 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:13:c1:55.'
 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:0f:0e:34.'
 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1b:3c:9a.'
 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1f:c7:11.'
 'ROM Error (C1219 Table 3) occurred for meter 00:13:50:05:ff:1f:c2:93.'
 'ROM Error (C1

In [70]:
curated_reasons = []
for reason_i in df_3_18_92_79['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.18.92.79']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

ROM Error (C1219 Table 3)


# '3.18.92.85'

In [71]:
df_3_18_92_85_1 = pd.read_pickle(ids_with_paths_dict['3.18.92.85'])
df_3_18_92_85_2 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_TX\sample_dfs\df_3_18_92_85.pkl')
df_3_18_92_85 = pd.concat([df_3_18_92_85_1, df_3_18_92_85_2])
#-----
print(f"shape[0] = {df_3_18_92_85.shape[0]}")
print(f"# Unique reasons = {df_3_18_92_85['reason'].nunique()}")
print(df_3_18_92_85['reason'].unique())

shape[0] = 8277
# Unique reasons = 9
['Meter detected a ROM failure.'
 'Meter event Rom Failure Detected  Time event occurred on meter = 03/16/2020 07:00:51  Sequence number = 1983  User id = 1024  Event argument = 00-00'
 'Meter event Rom Failure Detected  Time event occurred on meter = 11/24/2020 17:00:11  Sequence number = 564  User id = 0  Event argument = 00-00 '
 'Meter event Rom Failure Detected  Time event occurred on meter = 12/04/2020 10:00:13  Sequence number = 69  User id = 0  Event argument = 00-00 '
 'Meter event Rom Failure Detected  Time event occurred on meter = 12/09/2020 03:23:51  Sequence number = 1688  User id = 0  Event argument = 00-00 '
 'Meter event Rom Failure Detected  Time event occurred on meter = 12/18/2021 13:47:16  Sequence number = 20510  User id = 0  Event argument = 00-00 '
 'Meter event Rom Failure Detected  Time event occurred on meter = 12/20/2021 14:40:16  Sequence number = 20510  User id = 0  Event argument = 00-00 '
 'Meter event Rom Failure Det

In [72]:
curated_reasons = []
for reason_i in df_3_18_92_85['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.18.92.85']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter event Rom Failure Detected


# '3.21.1.79'

In [73]:
df_3_21_1_79 = pd.read_pickle(ids_with_paths_dict['3.21.1.79'])
#-----
print(f"shape[0] = {df_3_21_1_79.shape[0]}")
print(f"# Unique reasons = {df_3_21_1_79['reason'].nunique()}")
print(df_3_21_1_79['reason'].unique())

shape[0] = 118
# Unique reasons = 61
['Meter event Measurement Error Detected  Time event occurred on meter = 08/22/2021 20:05:07  Sequence number = 27684  User id = 0  Event argument = 00-00 '
 'Meter event Measurement Error Detected  Time event occurred on meter = 08/26/2021 08:27:25  Sequence number = 307  User id = 0  Event argument = 00-00 '
 'Meter event Measurement Error Detected  Time event occurred on meter = 08/26/2021 08:51:07  Sequence number = 308  User id = 0  Event argument = 00-00 '
 'Meter detected a measurement error.'
 'Meter event Measurement Error Detected  Time event occurred on meter = 08/26/2021 22:13:49  Sequence number = 1104  User id = 0  Event argument = 00-00 '
 'Meter event Measurement Error Detected  Time event occurred on meter = 08/26/2021 05:54:09  Sequence number = 311  User id = 0  Event argument = 00-00 '
 'Meter event Measurement Error Detected  Time event occurred on meter = 08/26/2021 16:36:23  Sequence number = 55256  User id = 0  Event argument

In [74]:
curated_reasons = []
for reason_i in df_3_21_1_79['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.1.79']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter event Measurement Error Detected


# '3.21.1.173'

In [75]:
df_3_21_1_173 = pd.read_pickle(ids_with_paths_dict['3.21.1.173'])
#-----
print(f"shape[0] = {df_3_21_1_173.shape[0]}")
print(f"# Unique reasons = {df_3_21_1_173['reason'].nunique()}")
print(df_3_21_1_173['reason'].unique())

shape[0] = 10430
# Unique reasons = 9251
['Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 08/28/2021 04:00:11  Sequence number = 3232  User id = 0  Event argument = 00-00 '
 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 08/28/2021 04:22:00  Sequence number = 5476  User id = 0  Event argument = 00-00 '
 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 08/28/2021 04:00:01  Sequence number = 10175  User id = 0  Event argument = 00-00 '
 ...
 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 09/02/2021 03:41:33  Sequence number = 42906  User id = 0  Event argument = 00-00 '
 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 09/02/2021 03:41:34  Sequence number = 42907  User id = 0  Event argument = 00-00 '
 'Meter event Nonvolatile Memory Failure Detected  Time event occurred on meter = 09/02/2021 03:51:00  Sequence number = 22928

In [76]:
curated_reasons = []
for reason_i in df_3_21_1_173['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.1.173']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter event Nonvolatile Memory Failure Detected


# '3.21.3.28'

In [77]:
df_3_21_3_28 = pd.read_pickle(ids_with_paths_dict['3.21.3.28'])
#-----
print(f"shape[0] = {df_3_21_3_28.shape[0]}")
print(f"# Unique reasons = {df_3_21_3_28['reason'].nunique()}")
print(df_3_21_3_28['reason'].unique())

shape[0] = 122
# Unique reasons = 121
['System Error cleared for meter 00:13:50:05:ff:1b:65:b4.'
 'System Error cleared for meter 00:13:50:05:ff:0d:c6:bc.'
 'System Error cleared for meter 00:13:50:05:ff:10:14:0c.'
 'System Error cleared for meter 00:13:50:05:ff:11:07:96.'
 'System Error cleared for meter 00:13:50:05:ff:0d:d4:5c.'
 'System Error cleared for meter 00:13:50:05:ff:11:a8:ab.'
 'System Error cleared for meter 00:13:50:05:ff:0f:22:1a.'
 'System Error cleared for meter 00:13:50:05:ff:0f:13:d2.'
 'System Error cleared for meter 00:13:50:05:ff:0d:e9:95.'
 'System Error cleared for meter 00:13:50:05:ff:13:3d:f2.'
 'System Error cleared for meter 00:13:50:05:ff:0f:cf:29.'
 'System Error cleared for meter 00:13:50:05:ff:11:55:84.'
 'System Error cleared for meter 00:13:50:05:ff:0d:ce:32.'
 'System Error cleared for meter 00:13:50:05:ff:0d:d1:a0.'
 'System Error cleared for meter 00:13:50:05:ff:11:83:06.'
 'System Error cleared for meter 00:13:50:05:ff:11:7f:dd.'
 'System Error cle

In [78]:
curated_reasons = []
for reason_i in df_3_21_3_28['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.3.28']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

System Error cleared


# '3.21.3.79'

In [79]:
df_3_21_3_79 = pd.read_pickle(ids_with_paths_dict['3.21.3.79'])
#-----
print(f"shape[0] = {df_3_21_3_79.shape[0]}")
print(f"# Unique reasons = {df_3_21_3_79['reason'].nunique()}")
print(df_3_21_3_79['reason'].unique())

shape[0] = 209
# Unique reasons = 204
['System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:0f:17:ba.'
 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:07:6a:3f.'
 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:14:cb:05.'
 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:11:7f:dd.'
 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:11:8e:70.'
 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:14:cb:09.'
 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:25:b0:3d.'
 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:0f:0d:e1.'
 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:14:d2:a6.'
 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:0f:0d:e3.'
 'System Error (C1219 Table 3: Er000020) occurred for meter 00:13:50:05:ff:0f:03:18.'
 'System Error (

In [80]:
curated_reasons = []
for reason_i in df_3_21_3_79['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.3.79']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

System Error (C1219 Table 3: Er000020)


# '3.21.17.28'

In [81]:
df_3_21_17_28 = pd.read_pickle(ids_with_paths_dict['3.21.17.28'])
#-----
print(f"shape[0] = {df_3_21_17_28.shape[0]}")
print(f"# Unique reasons = {df_3_21_17_28['reason'].nunique()}")
print(df_3_21_17_28['reason'].unique())

shape[0] = 380484
# Unique reasons = 1
['Meter detected a self check error.']


In [82]:
curated_reasons = []
for reason_i in df_3_21_17_28['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.17.28']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter detected a self check error


# '3.21.18.79'

In [83]:
df_3_21_18_79 = pd.read_pickle(ids_with_paths_dict['3.21.18.79'])
#-----
print(f"shape[0] = {df_3_21_18_79.shape[0]}")
print(f"# Unique reasons = {df_3_21_18_79['reason'].nunique()}")
print(df_3_21_18_79['reason'].unique())

shape[0] = 4352
# Unique reasons = 1
['Meter detected a self check error.']


In [84]:
curated_reasons = []
for reason_i in df_3_21_18_79['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.18.79']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter detected a self check error


# '3.21.38.223'

In [85]:
df_3_21_38_223 = pd.read_pickle(ids_with_paths_dict['3.21.38.223'])
#-----
print(f"shape[0] = {df_3_21_38_223.shape[0]}")
print(f"# Unique reasons = {df_3_21_38_223['reason'].nunique()}")
print(df_3_21_38_223['reason'].unique())

shape[0] = 785056
# Unique reasons = 707588
['Detected end of voltage sag on meter 00:13:50:05:ff:32:66:22 on one or several phases. Duration: 21 cycles (less than a second), Min RMS Voltage: Phase A 118.1 V, Phase B 95.4 V, Phase C 120.3 V, RMS Current (at min voltage): Phase A 1.0 A, Phase B 0.2 A, Phase C 1.2 A'
 'Detected end of voltage sag on meter 00:13:50:05:ff:32:66:22 on one or several phases. Duration: 22 cycles (less than a second), Min RMS Voltage: Phase A 118.2 V, Phase B 96.3 V, Phase C 120.3 V, RMS Current (at min voltage): Phase A 1.0 A, Phase B 0.2 A, Phase C 1.2 A'
 'Detected end of voltage sag on meter 00:13:50:05:ff:32:66:22 on one or several phases. Duration: 22 cycles (less than a second), Min RMS Voltage: Phase A 119.6 V, Phase B 118.1 V, Phase C 103.3 V, RMS Current (at min voltage): Phase A 0.9 A, Phase B 1.2 A, Phase C 0.6 A'
 ...
 'Detected end of voltage sag on meter 00:13:50:05:ff:06:8d:d9 on one or several phases. Duration: 13 cycles (less than a second), 

In [86]:
curated_reasons = []
for reason_i in df_3_21_38_223['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.38.223']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Detected end of voltage sag


# '3.21.38.248'

In [87]:
df_3_21_38_248 = pd.read_pickle(ids_with_paths_dict['3.21.38.248'])
#-----
print(f"shape[0] = {df_3_21_38_248.shape[0]}")
print(f"# Unique reasons = {df_3_21_38_248['reason'].nunique()}")
print(df_3_21_38_248['reason'].unique())

shape[0] = 669559
# Unique reasons = 571198
['Detected end of voltage swell on meter 00:13:50:05:ff:15:90:5d on one or several phases. Duration: 61 cycles (0 minutes 1.0 seconds), Max RMS Voltage: Phase A 123.1 V, Phase B 164.0 V, Phase C 122.4 V, RMS Current (at max voltage): Phase A 0.5 A, Phase B 0.2 A, Phase C 0.4 A'
 'Detected end of voltage swell on meter 00:13:50:05:ff:15:90:5d on one or several phases. Duration: 134 cycles (0 minutes 2.2 seconds), Max RMS Voltage: Phase A 123.1 V, Phase B 163.8 V, Phase C 122.3 V, RMS Current (at max voltage): Phase A 0.4 A, Phase B 0.2 A, Phase C 0.4 A'
 'Detected end of voltage swell on meter 00:13:50:05:ff:15:90:5d on one or several phases. Duration: 102 cycles (0 minutes 1.7 seconds), Max RMS Voltage: Phase A 122.4 V, Phase B 163.2 V, Phase C 122.3 V, RMS Current (at max voltage): Phase A 0.4 A, Phase B 0.2 A, Phase C 0.4 A'
 ...
 'Detected end of voltage swell on meter 00:13:50:03:ff:06:11:dc on one or several phases. Duration: 5 cycles (l

In [88]:
curated_reasons = []
for reason_i in df_3_21_38_248['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.38.248']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Detected end of voltage swell


# '3.21.43.223'

In [89]:
df_3_21_43_223 = pd.read_pickle(ids_with_paths_dict['3.21.43.223'])
#-----
print(f"shape[0] = {df_3_21_43_223.shape[0]}")
print(f"# Unique reasons = {df_3_21_43_223['reason'].nunique()}")
print(df_3_21_43_223['reason'].unique())

shape[0] = 2682
# Unique reasons = 1430
['Detected end of voltage sag on meter 00:13:50:03:00:4f:77:74. Duration: 4 seconds, Min RMS Voltage: Phase A 227.7 V, RMS Current (at min voltage): Phase A 0.0 A'
 'Detected end of voltage sag on meter 00:13:50:03:00:4f:77:74. Duration: 2 seconds, Min RMS Voltage: Phase A 227.7 V, RMS Current (at min voltage): Phase A 0.0 A'
 'Detected end of voltage sag on meter 00:13:50:03:00:4f:77:74. Duration: 3 seconds, Min RMS Voltage: Phase A 227.0 V, RMS Current (at min voltage): Phase A 0.0 A'
 ...
 'Detected end of voltage sag on meter 00:13:50:05:ff:2b:e0:8d. Duration: 11 seconds, Min RMS Voltage: Phase A 227.7 V, RMS Current (at min voltage): Phase A 0.0 A'
 'Detected end of voltage sag on meter 00:13:50:05:ff:08:bc:99. Duration: 1 seconds, Min RMS Voltage: Phase A 213.8 V, RMS Current (at min voltage): Phase A 0.0 A'
 'Detected end of voltage sag on meter 00:13:50:05:ff:08:bc:99. Duration: 1 seconds, Min RMS Voltage: Phase A 200.6 V, RMS Current (at

In [90]:
curated_reasons = []
for reason_i in df_3_21_43_223['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.43.223']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Detected end of voltage sag


# '3.21.43.248'

In [91]:
df_3_21_43_248 = pd.read_pickle(ids_with_paths_dict['3.21.43.248'])
#-----
print(f"shape[0] = {df_3_21_43_248.shape[0]}")
print(f"# Unique reasons = {df_3_21_43_248['reason'].nunique()}")
print(df_3_21_43_248['reason'].unique())

shape[0] = 848
# Unique reasons = 333
['Detected end of voltage swell on meter 00:13:50:05:ff:2d:2c:25. Duration: 195 seconds, Max RMS Voltage: Phase A 265.7 V, RMS Current (at max voltage): Phase A 0.0 A'
 'Detected end of voltage swell on meter 00:13:50:05:ff:2d:2c:25. Duration: 6 seconds, Max RMS Voltage: Phase A 266.8 V, RMS Current (at max voltage): Phase A 0.0 A'
 'Detected end of voltage swell on meter 00:13:50:05:ff:12:f5:b1. Duration: 23166 seconds, Max RMS Voltage: Phase A 158.3 V, RMS Current (at max voltage): Phase A 0.0 A'
 'Detected end of voltage swell on meter 00:13:50:05:ff:12:f5:b1. Duration: 1 seconds, Max RMS Voltage: Phase A 132.4 V, RMS Current (at max voltage): Phase A 0.0 A'
 'Detected end of voltage swell on meter 00:13:50:05:ff:12:f5:b1. Duration: 32 seconds, Max RMS Voltage: Phase A 132.8 V, RMS Current (at max voltage): Phase A 0.0 A'
 'Detected end of voltage swell on meter 00:13:50:05:ff:12:f5:b1. Duration: 352 seconds, Max RMS Voltage: Phase A 146.7 V, RM

In [92]:
curated_reasons = []
for reason_i in df_3_21_43_248['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.43.248']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Detected end of voltage swell


# '3.21.67.28'

In [93]:
df_3_21_67_28 = pd.read_pickle(ids_with_paths_dict['3.21.67.28'])
#-----
print(f"shape[0] = {df_3_21_67_28.shape[0]}")
print(f"# Unique reasons = {df_3_21_67_28['reason'].nunique()}")
print(df_3_21_67_28['reason'].unique())

shape[0] = 1509
# Unique reasons = 36
['Measurement Error cleared for meter 00:13:50:03:ff:06:10:15.'
 'Measurement Error cleared for meter 00:13:50:05:ff:07:c0:55.'
 'Measurement Error cleared for meter 00:13:50:05:ff:0f:0c:b6.'
 'Measurement Error cleared for meter 00:13:50:05:ff:0b:87:fc.'
 'Measurement Error cleared for meter 00:13:50:05:ff:02:7a:f9.'
 'Measurement Error cleared for meter 00:13:50:05:ff:1b:35:ed.'
 'Measurement Error cleared for meter 00:13:50:02:00:03:69:e3.'
 'Measurement Error cleared for meter 00:13:50:05:ff:0f:74:8d.'
 'Measurement Error cleared for meter 00:13:50:05:ff:04:1a:66.'
 'Measurement Error cleared for meter 00:13:50:05:ff:07:ba:b9.'
 'Measurement Error cleared for meter 00:13:50:05:ff:0b:86:38.'
 'Measurement Error cleared for meter 00:13:50:05:ff:1f:c7:11.'
 'Measurement Error cleared for meter 00:13:50:05:ff:0b:7f:f7.'
 'Measurement Error cleared for meter 00:13:50:03:ff:02:2b:d0.'
 'Measurement Error cleared for meter 00:13:50:05:ff:02:91:b4.'
 '

In [94]:
curated_reasons = []
for reason_i in df_3_21_67_28['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.67.28']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Measurement Error cleared


# '3.21.67.79'

In [95]:
df_3_21_67_79 = pd.read_pickle(ids_with_paths_dict['3.21.67.79'])
#-----
print(f"shape[0] = {df_3_21_67_79.shape[0]}")
print(f"# Unique reasons = {df_3_21_67_79['reason'].nunique()}")
print(df_3_21_67_79['reason'].unique())

shape[0] = 45365
# Unique reasons = 24035
['Meter event Measurement Error Detected  Time event occurred on meter = 01/01/2020 05:02:43  Sequence number = 241  User id = 0  Event argument = 00-00'
 'Meter event Measurement Error Detected  Time event occurred on meter = 01/01/2020 20:51:18  Sequence number = 1878  User id = 0  Event argument = 00-00'
 'Meter event Measurement Error Detected  Time event occurred on meter = 01/01/2020 07:56:50  Sequence number = 242  User id = 0  Event argument = 00-00'
 ...
 'Meter event Measurement Error Detected  Time event occurred on meter = 03/14/2023 02:00:56  Sequence number = 3953  User id = 0  Event argument = 00-00 '
 'Meter event Measurement Error Detected  Time event occurred on meter = 03/14/2023 03:27:02  Sequence number = 3954  User id = 0  Event argument = 00-00 '
 'Meter event Measurement Error Detected  Time event occurred on meter = 9/12/2023 6:06:57 AM  Sequence number = 89  User id = 34891  Event argument = 7D-75']


In [96]:
curated_reasons = []
for reason_i in df_3_21_67_79['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.67.79']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Measurement Error (C1219 Table 3) 
Measurement Error (C1219 Table 3: Er100000) 
Meter event Measurement Error Detected


# '3.21.82.28'

In [97]:
df_3_21_82_28 = pd.read_pickle(ids_with_paths_dict['3.21.82.28'])
#-----
print(f"shape[0] = {df_3_21_82_28.shape[0]}")
print(f"# Unique reasons = {df_3_21_82_28['reason'].nunique()}")
print(df_3_21_82_28['reason'].unique())

shape[0] = 62
# Unique reasons = 19
['DSP Error cleared for meter 00:13:50:05:ff:0b:87:fc.'
 'DSP Error cleared for meter 00:13:50:05:ff:1b:35:ed.'
 'DSP Error cleared for meter 00:13:50:05:ff:0f:74:8d.'
 'DSP Error cleared for meter 00:13:50:02:00:03:69:e3.'
 'DSP Error cleared for meter 00:13:50:05:ff:15:af:91.'
 'DSP Error cleared for meter 00:13:50:05:ff:07:ba:b9.'
 'DSP Error cleared for meter 00:13:50:05:ff:13:c0:a6.'
 'DSP Error cleared for meter 00:13:50:05:ff:0f:0c:b6.'
 'DSP Error cleared for meter 00:13:50:05:ff:1f:c7:11.'
 'DSP Error cleared for meter 00:13:50:03:ff:02:2b:d0.'
 'DSP Error cleared for meter 00:13:50:05:ff:02:91:97.'
 'DSP Error cleared for meter 00:13:50:03:ff:06:3c:37.'
 'DSP Error cleared for meter 00:13:50:05:ff:23:a7:ab.'
 'DSP Error cleared for meter 00:13:50:03:ff:06:0d:f6.'
 'DSP Error cleared for meter 00:13:50:05:ff:11:7d:2f.'
 'DSP Error cleared for meter 00:13:50:05:ff:02:7f:04.'
 'DSP Error cleared for meter 00:13:50:05:ff:02:68:cb.'
 'DSP Error 

In [98]:
curated_reasons = []
for reason_i in df_3_21_82_28['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.82.28']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

DSP Error cleared


# '3.21.82.79'

In [99]:
df_3_21_82_79 = pd.read_pickle(ids_with_paths_dict['3.21.82.79'])
#-----
print(f"shape[0] = {df_3_21_82_79.shape[0]}")
print(f"# Unique reasons = {df_3_21_82_79['reason'].nunique()}")
print(df_3_21_82_79['reason'].unique())

shape[0] = 83
# Unique reasons = 32
['DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:05:ff:15:af:91.'
 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:05:ff:0b:87:fc.'
 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:05:ff:1b:35:ed.'
 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:02:00:03:69:e3.'
 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:05:ff:04:1a:66.'
 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:05:ff:13:c0:a6.'
 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:05:ff:07:ba:b9.'
 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:05:ff:13:bb:db.'
 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:05:ff:0f:0c:b6.'
 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:03:ff:02:1f:cb.'
 'DSP Error (C1219 Table 3: Er200000) occurred for meter 00:13:50:03:ff:06:78:70.'
 'DSP Error (C1219 Table 3: Er200000) occurred for 

In [100]:
curated_reasons = []
for reason_i in df_3_21_82_79['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.21.82.79']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

DSP Error (C1219 Table 3: Er200000)


# '3.22.12.243'

In [101]:
df_3_22_12_243 = pd.read_pickle(ids_with_paths_dict['3.22.12.243'])
#-----
print(f"shape[0] = {df_3_22_12_243.shape[0]}")
print(f"# Unique reasons = {df_3_22_12_243['reason'].nunique()}")
print(df_3_22_12_243['reason'].unique())

shape[0] = 45
# Unique reasons = 45
['Meter event Test Mode Stopped  Time event occurred on meter = 08/27/2021 10:04:04  Sequence number = 23834  User id = 1  Event argument = 00-00 '
 'Meter event Test Mode Stopped  Time event occurred on meter = 08/30/2021 09:27:50  Sequence number = 2442  User id = 1  Event argument = 00-00 '
 'Meter event Test Mode Stopped  Time event occurred on meter = 08/30/2021 10:57:32  Sequence number = 1465  User id = 1  Event argument = 00-00 '
 'Meter event Test Mode Stopped  Time event occurred on meter = 08/30/2021 19:49:10  Sequence number = 296  User id = 1  Event argument = 00-00 '
 'Meter event Test Mode Stopped  Time event occurred on meter = 08/30/2021 11:52:44  Sequence number = 1915  User id = 1  Event argument = 00-00 '
 'Meter event Test Mode Stopped  Time event occurred on meter = 08/24/2021 15:11:30  Sequence number = 2515  User id = 1  Event argument = 00-00 '
 'Meter event Test Mode Stopped  Time event occurred on meter = 08/24/2021 09:14:0

In [102]:
curated_reasons = []
for reason_i in df_3_22_12_243['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.22.12.243']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter event Test Mode Stopped


# '3.22.19.242'

In [103]:
df_3_22_19_242 = pd.read_pickle(ids_with_paths_dict['3.22.19.242'])
#-----
print(f"shape[0] = {df_3_22_19_242.shape[0]}")
print(f"# Unique reasons = {df_3_22_19_242['reason'].nunique()}")
print(df_3_22_19_242['reason'].unique())

shape[0] = 50
# Unique reasons = 50
['Meter event Test Mode Started  Time event occurred on meter = 08/30/2021 03:31:25  Sequence number = 119  User id = 1  Event argument = 00-00 '
 'Meter event Test Mode Started  Time event occurred on meter = 08/30/2021 03:33:55  Sequence number = 124  User id = 1  Event argument = 00-00 '
 'Meter event Test Mode Started  Time event occurred on meter = 08/30/2021 03:34:32  Sequence number = 129  User id = 1  Event argument = 00-00 '
 'Meter event Test Mode Started  Time event occurred on meter = 08/30/2021 03:34:34  Sequence number = 133  User id = 1  Event argument = 00-00 '
 'Meter event Test Mode Started  Time event occurred on meter = 08/30/2021 03:34:35  Sequence number = 137  User id = 1  Event argument = 00-00 '
 'Meter event Test Mode Started  Time event occurred on meter = 08/30/2021 19:49:09  Sequence number = 294  User id = 1  Event argument = 00-00 '
 'Meter event Test Mode Started  Time event occurred on meter = 08/30/2021 09:26:38  Seq

In [104]:
curated_reasons = []
for reason_i in df_3_22_19_242['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.22.19.242']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter event Test Mode Started


# '3.23.17.139'

In [105]:
df_3_23_17_139 = pd.read_pickle(ids_with_paths_dict['3.23.17.139'])
#-----
print(f"shape[0] = {df_3_23_17_139.shape[0]}")
print(f"# Unique reasons = {df_3_23_17_139['reason'].nunique()}")
print(df_3_23_17_139['reason'].unique())

shape[0] = 29863
# Unique reasons = 28303
['Device 00:13:50:05:ff:19:7b:be has been determined to have exceeded the max allowable trap threshold, 20, within a certain time limit, 3600 seconds.'
 'Device 00:13:50:05:ff:20:91:89 has been determined to have exceeded the max allowable trap threshold, 20, within a certain time limit, 3600 seconds.'
 'Device 00:13:50:05:ff:20:8f:67 has been determined to have exceeded the max allowable trap threshold, 20, within a certain time limit, 3600 seconds.'
 ...
 'Device 00:13:50:05:ff:2f:50:53 has been determined to have exceeded the max allowable trap threshold, 20, within a certain time limit, 3600 seconds.'
 'Device 00:13:50:05:ff:2b:30:b1 has been determined to have exceeded the max allowable trap threshold, 20, within a certain time limit, 3600 seconds.'
 'Device 00:13:50:05:ff:44:9c:d8 has been determined to have exceeded the max allowable trap threshold, 20, within a certain time limit, 3600 seconds.']


In [106]:
curated_reasons = []
for reason_i in df_3_23_17_139['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.23.17.139']
    )
#     if curated_reason_i != 'Primary Power Up occurred for meter':
#         print(curated_reason_i)
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Device exceeded the max allowable trap threshold


# '3.23.136.47'

In [107]:
df_3_23_136_47 = pd.read_pickle(ids_with_paths_dict['3.23.136.47'])
#-----
print(f"shape[0] = {df_3_23_136_47.shape[0]}")
print(f"# Unique reasons = {df_3_23_136_47['reason'].nunique()}")
print(df_3_23_136_47['reason'].unique())

shape[0] = 189812
# Unique reasons = 1506
['Access Point 00:13:50:09:ff:00:15:a9 has lost connectivity with FHSS 900 MHz band.'
 'Access Point 00:13:50:08:ff:00:06:f7 has lost connectivity with FHSS 900 MHz band.'
 'Access Point 00:13:50:08:ff:00:06:8f has lost connectivity with FHSS 900 MHz band.'
 ...
 'Access Point 00:13:50:09:ff:00:14:c0 has lost connectivity with FHSS 900 MHz band.'
 'Access Point 00:13:50:09:ff:00:1d:9b has lost connectivity with FHSS 900 MHz band.'
 'Access Point 00:13:50:09:ff:00:1d:cc has lost connectivity with FHSS 900 MHz band.']


# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [108]:
curated_reasons = []
for reason_i in df_3_23_136_47['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.23.136.47']
    )
#     if curated_reason_i != 'Primary Power Up occurred for meter':
#         print(curated_reason_i)
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Access Point has lost connectivity with FHSS 900 MHz band.


# '3.23.136.85'

In [109]:
df_3_23_136_85 = pd.read_pickle(ids_with_paths_dict['3.23.136.85'])
#-----
print(f"shape[0] = {df_3_23_136_85.shape[0]}")
print(f"# Unique reasons = {df_3_23_136_85['reason'].nunique()}")
print(df_3_23_136_85['reason'].unique())

shape[0] = 2595861
# Unique reasons = 178557
['NIC Link Layer Handshake Failed: Device: 00:13:50:ff:fe:70:9c:a3, Rejected neighbor Mac ID: 00:13:50:05:ff:17:77:e3, Rejection Cause: invalid birth certificate'
 'NIC Link Layer Handshake Failed: Device: 00:13:50:03:ff:01:e0:8a, Rejected neighbor Mac ID: 00:13:50:ff:fe:11:e6:bf, Rejection Cause: invalid eblob signature'
 'NIC Link Layer Handshake Failed: Device: 00:13:50:05:ff:1b:f5:1d, Rejected neighbor Mac ID: 00:13:50:ff:fe:11:b9:55, Rejection Cause: invalid eblob signature'
 ...
 'NIC Link Layer Handshake Failed: Device: 00:13:50:05:ff:01:97:87, Rejected neighbor Mac ID: 00:13:50:05:ff:05:a9:69, Rejection Cause: invalid eblob signature'
 'NIC Link Layer Handshake Failed: Device: 00:13:50:05:ff:1f:ad:27, Rejected neighbor Mac ID: 00:13:50:05:ff:0a:75:9d, Rejection Cause: invalid eblob signature'
 'NIC Link Layer Handshake Failed: Device: 00:13:50:05:ff:0d:0b:ce, Rejected neighbor Mac ID: 00:13:50:05:ff:04:60:74, Rejection Cause: invalid

In [110]:
curated_reasons = []
for reason_i in df_3_23_136_85['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.23.136.85']
    )
#     if curated_reason_i != 'Primary Power Up occurred for meter':
#         print(curated_reason_i)
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Device Failed: Reason: unknown (0x6)
Device Failed: Reason: Security public key mismatch
NIC Link Layer Handshake Failed: Rejection Cause: invalid eblob signature
NIC Link Layer Handshake Failed: Rejection Cause: invalid birth certificate
NIC Link Layer Handshake Failed: Rejection Cause: invalid dl chain cert
Device Failed: Reason: unknown (0x4)
NIC Link Layer Handshake Failed: Rejection Cause: invalid mfg cert


# '3.25.17.3'

In [111]:
df_3_25_17_3 = pd.read_pickle(ids_with_paths_dict['3.25.17.3'])
#-----
print(f"shape[0] = {df_3_25_17_3.shape[0]}")
print(f"# Unique reasons = {df_3_25_17_3['reason'].nunique()}")
print(df_3_25_17_3['reason'].unique())

shape[0] = 3331524
# Unique reasons = 1
['Meter generated an energy polarity gyrbox call in event.']


In [112]:
curated_reasons = []
for reason_i in df_3_25_17_3['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.25.17.3']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter generated an energy polarity gyrbox call in event


# '3.26.0.47'

In [13]:
df_3_26_0_47 = pd.read_pickle(ids_with_paths_dict['3.26.0.47'])
#-----
print(f"shape[0] = {df_3_26_0_47.shape[0]}")
print(f"# Unique reasons = {df_3_26_0_47['reason'].nunique()}")
print(df_3_26_0_47['reason'].unique())

shape[0] = 3900000
# Unique reasons = 1598399
['Primary Power Down occurred for meter 00:13:50:05:ff:24:1a:d2.'
 'Primary Power Down occurred for meter 00:13:50:05:ff:19:31:63.'
 'Primary Power Down occurred for meter 00:13:50:01:00:dd:58:40.' ...
 'Primary Power Down occurred for meter 00:13:50:05:ff:4a:32:bc.'
 'Primary Power Down occurred for meter 00:13:50:05:ff:48:24:47.'
 'Primary Power Down occurred for meter 00:13:50:05:ff:47:b2:1d.']


In [14]:
curated_reasons = []
for reason_i in df_3_26_0_47['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.26.0.47']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter had a power outage (Unsafe power fail)
Meter event Primary Power Down
Primary Power Down


# '3.26.0.216'

In [115]:
df_3_26_0_216 = pd.read_pickle(ids_with_paths_dict['3.26.0.216'])
#-----
print(f"shape[0] = {df_3_26_0_216.shape[0]}")
print(f"# Unique reasons = {df_3_26_0_216['reason'].nunique()}")
print(df_3_26_0_216['reason'].unique())

shape[0] = 3900000
# Unique reasons = 1651018
['Primary Power Up occurred for meter 00:13:50:05:ff:10:92:05.'
 'Primary Power Up occurred for meter 00:13:50:05:ff:15:bd:c8.'
 'Primary Power Up occurred for meter 00:13:50:05:ff:23:e4:bf.' ...
 'Primary Power Up occurred for meter 00:13:50:05:ff:41:42:d2.'
 'Primary Power Up occurred for meter 00:13:50:05:ff:4b:68:f2.'
 'Primary Power Up occurred for meter 00:13:50:05:ff:40:08:a8.']


In [116]:
curated_reasons = []
for reason_i in df_3_26_0_216['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.26.0.216']
    )
#     if curated_reason_i != 'Primary Power Up occurred for meter':
#         print(curated_reason_i)
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Primary Power Up


# '3.26.17.185'

In [117]:
df_3_26_17_185 = pd.read_pickle(ids_with_paths_dict['3.26.17.185'])
#-----
print(f"shape[0] = {df_3_26_17_185.shape[0]}")
print(f"# Unique reasons = {df_3_26_17_185['reason'].nunique()}")
print(df_3_26_17_185['reason'].unique())

shape[0] = 5854
# Unique reasons = 2412
['Meter had a power outage. Unsafe power fail count = 45969'
 'Meter had a power outage. Unsafe power fail count = 415'
 'Meter had a power outage. Unsafe power fail count = 1' ...
 'Meter had a power outage. Unsafe power fail count = 2306'
 'Meter had a power outage. Unsafe power fail count = 6726'
 'Meter had a power outage. Unsafe power fail count = 1450']


In [118]:
curated_reasons = []
for reason_i in df_3_26_17_185['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.26.17.185']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter event Primary Power Down
Meter had a power outage (Unsafe power fail)


# '3.26.17.216'

In [119]:
df_3_26_17_216 = pd.read_pickle(ids_with_paths_dict['3.26.17.216'])
#-----
print(f"shape[0] = {df_3_26_17_216.shape[0]}")
print(f"# Unique reasons = {df_3_26_17_216['reason'].nunique()}")
print(df_3_26_17_216['reason'].unique())

shape[0] = 1
# Unique reasons = 1
['Meter event Primary Power Up  Time event occurred on meter = 08/31/2021 20:07:44  Sequence number = 104  User id = 0  Event argument = 00-00 ']


In [120]:
curated_reasons = []
for reason_i in df_3_26_17_216['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.26.17.216']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Primary Power Up


# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# '3.26.38.37'

In [121]:
df_3_26_38_37 = pd.read_pickle(ids_with_paths_dict['3.26.38.37'])
#-----
print(f"shape[0] = {df_3_26_38_37.shape[0]}")
print(f"# Unique reasons = {df_3_26_38_37['reason'].nunique()}")
print(df_3_26_38_37['reason'].unique())

shape[0] = 3900000
# Unique reasons = 89431
['Under Voltage cleared (CA000400) for meter 00:13:50:05:ff:0b:88:ec.'
 'Under Voltage (CA000400) cleared for meter 00:13:50:05:ff:0c:79:88.'
 'Under Voltage (CA000400) cleared for meter 00:13:50:01:00:e3:db:2b.' ...
 'Low Potential cleared for meter 00:13:50:05:ff:1b:3a:81.'
 'Under Voltage cleared (CA000400) for meter 00:13:50:05:ff:1b:68:84.'
 'Low Potential cleared for meter 00:13:50:05:ff:1b:68:84.']


In [122]:
curated_reasons = []
for reason_i in df_3_26_38_37['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.26.38.37']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Low Potential cleared
Under Voltage (Diagnostic 6) cleared
Diag6 Condition cleared
Under Voltage (CA000400) cleared


# '3.26.38.47'

In [123]:
df_3_26_38_47 = pd.read_pickle(ids_with_paths_dict['3.26.38.47'])
#-----
print(f"shape[0] = {df_3_26_38_47.shape[0]}")
print(f"# Unique reasons = {df_3_26_38_47['reason'].nunique()}")
print(df_3_26_38_47['reason'].unique())

shape[0] = 1055446
# Unique reasons = 106912
['Low Potential (C1219 Table 3) occurred for meter 00:13:50:05:ff:1b:67:3b.'
 'Low Potential (C1219 Table 3) occurred for meter 00:13:50:05:ff:15:e8:3b.'
 'Low Potential (C1219 Table 3) occurred for meter 00:13:50:05:ff:26:2d:3f.'
 ...
 'Low Potential (C1219 Table 3) occurred for meter 00:13:50:03:ff:02:32:93.'
 'Low Potential (C1219 Table 3) occurred for meter 00:13:50:03:ff:03:0b:91.'
 'Low Potential (C1219 Table 3) occurred for meter 00:13:50:05:ff:2c:e5:72.']


In [124]:
curated_reasons = []
for reason_i in df_3_26_38_47['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.26.38.47']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Low Potential (C1219 Table 3)


# '3.26.38.73'

In [125]:
df_3_26_38_73 = pd.read_pickle(ids_with_paths_dict['3.26.38.73'])
#-----
print(f"shape[0] = {df_3_26_38_73.shape[0]}")
print(f"# Unique reasons = {df_3_26_38_73['reason'].nunique()}")
print(df_3_26_38_73['reason'].unique())

shape[0] = 3298659
# Unique reasons = 262681
['Diag7 Condition cleared for meter 00:13:50:02:00:0a:ff:36.'
 'Diag7 Condition cleared for meter 00:13:50:05:ff:1b:59:d4.'
 'Diag7 Condition cleared for meter 00:13:50:02:00:0b:01:e4.' ...
 'Over Voltage (Diagnostic 7) cleared for meter 00:13:50:05:ff:13:d6:97N/A.'
 'Over Voltage (Diagnostic 7) cleared for meter 00:13:50:05:ff:20:71:d0N/A.'
 'Over Voltage (Diagnostic 7) cleared for meter 00:13:50:05:ff:20:8a:73N/A.']


In [126]:
curated_reasons = []
for reason_i in df_3_26_38_73['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.26.38.73']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Diag7 Condition cleared
Over Voltage (Diagnostic 7) cleared


# '3.26.38.93'

In [127]:
df_3_26_38_93_1 = pd.read_pickle(ids_with_paths_dict['3.26.38.93'])
df_3_26_38_93_2 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_TX\sample_dfs\df_3_26_38_93.pkl')
df_3_26_38_93 = pd.concat([df_3_26_38_93_1, df_3_26_38_93_2])
#-----
print(f"shape[0] = {df_3_26_38_93.shape[0]}")
print(f"# Unique reasons = {df_3_26_38_93['reason'].nunique()}")
print(df_3_26_38_93['reason'].unique())

shape[0] = 4281627
# Unique reasons = 189383
['KV2c meter event Over Voltage Diagnostic flags:Phase A Voltage'
 'Diag7: Over Voltage, Element A occurred for meter 00:13:50:ff:fe:03:4e:12.'
 'Diag7: Over Voltage, Element A occurred for meter 00:13:50:05:ff:04:35:4a.'
 ...
 'Over Voltage (Diagnostic 7) occurred for meter 00:13:50:05:ff:1b:c3:31: Phase A.'
 'Over Voltage (Diagnostic 7) occurred for meter 00:13:50:05:ff:23:94:7d: Phase A.'
 'KV2c meter event Over Voltage Diagnostic flags: Phase A Voltage ']


In [128]:
curated_reasons = []
for reason_i in df_3_26_38_93['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.26.38.93']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*natsorted(curated_reasons), sep='\n')

Diag7: Over Voltage, Element A 
KV2c meter event Over Voltage (Diagnostic flags = Phase A Voltage)
KV2c meter event Received kWh (Diagnostic flags = Phase A Voltage)
KV2c meter event Received kWh Cleared
KV2c meter event Under Voltage (Diagnostic flags = Phase B Voltage)
KV2c meter event Voltage Out of Tolerance (Diagnostic flags = Phase A Voltage)
Over Voltage (Diagnostic 7) : Phase A.


# '3.26.38.150'

In [129]:
df_3_26_38_150 = pd.read_pickle(ids_with_paths_dict['3.26.38.150'])
#-----
print(f"shape[0] = {df_3_26_38_150.shape[0]}")
print(f"# Unique reasons = {df_3_26_38_150['reason'].nunique()}")
print(df_3_26_38_150['reason'].unique())

shape[0] = 390000
# Unique reasons = 20949
['Under Voltage (CA000400) for meter 00:13:50:05:ff:0b:88:ec. Phase  C Voltage out of tolerance.'
 'Under Voltage (CA000400) for meter 00:13:50:05:ff:0b:88:ec. Phase  A Voltage out of tolerance.'
 'Under Voltage (CA000400) occurred for meter 00:13:50:05:ff:0d:10:85: Phase A.'
 ...
 'Under Voltage (CA000400) occurred for meter 00:13:50:05:ff:1e:e6:d6: Phase A.'
 'Under Voltage (CA000400) occurred for meter 00:13:50:05:ff:1f:3b:3d: Phase C.'
 'Under Voltage (CA000400) for meter 00:13:50:05:ff:49:11:14. Phase  A Voltage out of tolerance.']


In [130]:
curated_reasons = []
for reason_i in df_3_26_38_150['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.26.38.150']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Under Voltage (CA000400)  Phase  C Voltage out of tolerance.
Under Voltage (CA000400)  Phase  A, B Voltage out of tolerance.
KV2c meter event Under Voltage  (Diagnostic flags = Phase A Voltage)
Diag6: Under Voltage, Element A
Under Voltage (CA000400)  Phase A.
Under Voltage (CA000400)  Phase  B, C Voltage out of tolerance.
Under Voltage (CA000400)  Phase  B Voltage out of tolerance.
Under Voltage (CA000400)  Phase  A Voltage out of tolerance.
Under Voltage (CA000400)  Phase C.
Under Voltage (CA000400)  Phase  A, B, C Voltage out of tolerance.
Under Voltage (CA000400)  Phase A and C.
Under Voltage (Diagnostic 6)  Phase A.
Under Voltage (CA000400)  Phase  A, C Voltage out of tolerance.


# '3.26.136.47'

In [131]:
df_3_26_136_47 = pd.read_pickle(ids_with_paths_dict['3.26.136.47'])
#-----
print(f"shape[0] = {df_3_26_136_47.shape[0]}")
print(f"# Unique reasons = {df_3_26_136_47['reason'].nunique()}")
print(df_3_26_136_47['reason'].unique())

shape[0] = 390000
# Unique reasons = 385739
['Last Gasp - NIC power lost for device: 00:13:50:05:ff:0b:83:e4, Reboot Count: 11900, NIC timestamp: 2020-01-05T15:26:48.000-05:00, Received timestamp: 2020-01-05T15:26:49.534-05:00, Fail Reason: [0x49] LG_ZERO_X_DETECTOR ,LG_DIRECT_NOTIFICATION'
 'Last Gasp - NIC power lost for device: 00:13:50:01:00:dc:37:9b, Reboot Count: 143, NIC timestamp: 2020-01-05T15:10:53.000-05:00, Received timestamp: 2020-01-05T15:10:54.921-05:00, Fail Reason: [0x40] LG_DIRECT_NOTIFICATION'
 'Last Gasp - NIC power lost for device: 00:13:50:05:ff:0b:83:e4, Reboot Count: 11879, NIC timestamp: 2020-01-05T03:26:46.000-05:00, Received timestamp: 2020-01-05T03:26:49.286-05:00, Fail Reason: [0x49] LG_ZERO_X_DETECTOR ,LG_DIRECT_NOTIFICATION'
 ...
 'Last Gasp - NIC power lost for device: 00:13:50:05:ff:39:6a:77, Reboot Count: 12, NIC timestamp: 2023-03-10T22:18:22.000-05:00, Received timestamp: 2023-03-10T22:18:23.416-05:00, Fail Reason: [0x50] LG_PF_DETECTOR ,LG_DIRECT_NO

In [132]:
curated_reasons = []
for reason_i in df_3_26_136_47['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.26.136.47']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Last Gasp - NIC power lost for device, Fail Reason: [0x58] LG_ZERO_X_DETECTOR ,LG_PF_DETECTOR ,LG_DIRECT_NOTIFICA
Last Gasp - NIC power lost for device, Fail Reason: [0x18] LG_ZERO_X_DETECTOR ,LG_PF_DETECTOR
Last Gasp - NIC power lost for device, Fail Reason: [0x58] LG_ZERO_X_DETECTOR ,LG_PF_DETECTOR ,LG_DIRECT_NOTIFICATIO
Last Gasp - NIC power lost for device, Fail Reason: [0x04]
Last Gasp - NIC power lost for device, Fail Reason: [0x5A] LG_FLAG_FLASH_ERR ,LG_ZERO_X_DETECTOR ,LG_PF_DETECTOR ,L
Last Gasp - NIC power lost for device, Fail Reason: [0x52] LG_FLAG_FLASH_ERR ,LG_PF_DETECTOR ,LG_DIRECT_NOTIFICATION
Last Gasp - NIC power lost for device, Fail Reason: NIC power fail
Last Gasp - NIC power lost for device, Fail Reason: [0x59] LG_ZERO_X_DETECTOR ,LG_PF_DETECTOR ,LG_DIRECT_NOTIFICATI
Last Gasp - NIC power lost for device, Fail Reason: [0x51] LG_PF_DETECTOR ,LG_DIRECT_NOTIFICATION
Last Gasp - NIC power lost for device, Fail Reason: Meter power fail
Last Gasp - NIC power lost for de

# '3.26.136.66'

In [133]:
df_3_26_136_66 = pd.read_pickle(ids_with_paths_dict['3.26.136.66'])
#-----
print(f"shape[0] = {df_3_26_136_66.shape[0]}")
print(f"# Unique reasons = {df_3_26_136_66['reason'].nunique()}")
print(df_3_26_136_66['reason'].unique())

shape[0] = 8502
# Unique reasons = 8461
['Device 00:13:50:05:ff:01:3c:23, Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_DISABLED, Detector State: EL_EVENT_POWER_FAIL_DETECT_METER_PF_DISABLED, Reboot Count: 206'
 'Device 00:13:50:03:ff:03:4a:ee, Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_DISABLED, Detector State: EL_EVENT_POWER_FAIL_DETECT_METER_PF_DISABLED, Reboot Count: 206'
 'Device 00:13:50:05:ff:24:8a:92, Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_DISABLED, Detector State: EL_EVENT_POWER_FAIL_DETECT_METER_PF_DISABLED, Reboot Count: 148'
 ...
 'Device 00:13:50:05:ff:44:b9:86, Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_DISABLED, Detector State: EL_EVENT_POWER_FAIL_DETECT_NIC_ZX_DISABLED, Reboot Count: 132'
 'Device 00:13:50:05:ff:44:bc:2c, Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_DISABLED, Detector State: EL_EVENT_POWER_FAIL_DETECT_NIC_ZX_DISABLED, Reboot Count: 107'
 'Device 00:13:50:05:ff:44:cb:5b, Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_DISABLED, Detector

In [134]:
curated_reasons = []
for reason_i in df_3_26_136_66['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.26.136.66']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_DISABLED, Detector State: EL_EVENT_POWER_FAIL_ENABLED
Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_ENABLED, Detector State: EL_EVENT_POWER_FAIL_ENABLED
Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_DISABLED, Detector State: EL_EVENT_POWER_FAIL_DETECT_METER_PF_DISABLED
Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_ENABLED, Detector State: EL_EVENT_POWER_FAIL_DETECT_METER_PF_DISABLED
Last Gasp State: EL_EVENT_POWER_FAIL_DETECT_LG_DISABLED, Detector State: EL_EVENT_POWER_FAIL_DETECT_NIC_ZX_DISABLED


# '3.26.136.216'

In [135]:
df_3_26_136_216 = pd.read_pickle(ids_with_paths_dict['3.26.136.216'])
#-----
print(f"shape[0] = {df_3_26_136_216.shape[0]}")
print(f"# Unique reasons = {df_3_26_136_216['reason'].nunique()}")
print(df_3_26_136_216['reason'].unique())

shape[0] = 379187
# Unique reasons = 376783
['NIC Power Restore Trap Received from device: 00:13:50:08:ff:00:01:e1, Reboot Count: 94, NIC timestamp: 2020-01-13T15:07:16.124-05:00, Received Timestamp: 2020-01-13T15:07:16.124-05:00, Power Restoration Timestamp: 2020-01-13T15:05:54.124-05:00, State: 4,p'
 'NIC Power Restore Trap Received from device: 00:13:50:03:00:4a:73:98, Reboot Count: 193, NIC timestamp: 2020-01-13T21:31:37.000-05:00, Received Timestamp: 2020-01-13T21:31:38.794-05:00, Power Restoration Timestamp: 2020-01-13T21:31:24.000-05:00, State: 4,'
 'NIC Power Restore Trap Received from device: 00:13:50:05:ff:0b:83:e4, Reboot Count: 12090, NIC timestamp: 2020-01-13T12:06:47.000-05:00, Received Timestamp: 2020-01-13T12:06:48.695-05:00, Power Restoration Timestamp: 2020-01-13T12:06:36.000-05:00, State:'
 ...
 'NIC Power Restore Trap Received from device: 00:13:50:05:ff:3b:da:72, Reboot Count: 30, NIC timestamp: 2023-03-10T10:26:26.000-05:00, Received Timestamp: 2023-03-10T10:26:30

In [136]:
curated_reasons = []
for reason_i in df_3_26_136_216['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.26.136.216']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

NIC Power Restore Trap Received from device


# '3.31.1.143'

In [137]:
df_3_31_1_143 = pd.read_pickle(ids_with_paths_dict['3.31.1.143'])
#-----
print(f"shape[0] = {df_3_31_1_143.shape[0]}")
print(f"# Unique reasons = {df_3_31_1_143['reason'].nunique()}")
print(df_3_31_1_143['reason'].unique())

shape[0] = 86691
# Unique reasons = 1
['Meter assumed to be disconnected has reported Load side voltage indicating a potential case of tamper.']


In [138]:
curated_reasons = []
for reason_i in df_3_31_1_143['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.31.1.143']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter assumed to be disconnected has reported Load side voltage indicating a potential case of tamper


# '3.33.1.219'

In [139]:
df_3_33_1_219 = pd.read_pickle(ids_with_paths_dict['3.33.1.219'])
#-----
print(f"shape[0] = {df_3_33_1_219.shape[0]}")
print(f"# Unique reasons = {df_3_33_1_219['reason'].nunique()}")
print(df_3_33_1_219['reason'].unique())

shape[0] = 20000
# Unique reasons = 2363
['Meter event Reverse Rotation Detected  Time event occurred on meter = 08/30/2021 04:25:38  Sequence number = 13124  User id = 0  Event argument = 00-00 '
 'Meter event Reverse Rotation Detected  Time event occurred on meter = 08/30/2021 04:13:29  Sequence number = 3467  User id = 0  Event argument = 00-00 '
 'Meter event Reverse Rotation Detected  Time event occurred on meter = 08/30/2021 04:27:33  Sequence number = 3468  User id = 0  Event argument = 00-00 '
 ...
 'Meter event Reverse Rotation Detected  Time event occurred on meter = 09/01/2021 18:21:05  Sequence number = 7992  User id = 0  Event argument = 00-00 '
 'Meter event Reverse Rotation Detected  Time event occurred on meter = 09/01/2021 18:31:07  Sequence number = 89  User id = 0  Event argument = 00-00 '
 'Meter event Reverse Rotation Detected  Time event occurred on meter = 09/01/2021 18:09:14  Sequence number = 452  User id = 0  Event argument = 00-00 ']


In [140]:
curated_reasons = []
for reason_i in df_3_33_1_219['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.33.1.219']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter event Reverse Rotation Detected


# '3.33.1.257'

In [141]:
df_3_33_1_257 = pd.read_pickle(ids_with_paths_dict['3.33.1.257'])
#-----
print(f"shape[0] = {df_3_33_1_257.shape[0]}")
print(f"# Unique reasons = {df_3_33_1_257['reason'].nunique()}")
print(df_3_33_1_257['reason'].unique())

shape[0] = 2147
# Unique reasons = 158
['Meter detected a Tamper Attempt.'
 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 08/26/2021 09:35:22  Sequence number = 5881  User id = 0  Event argument = 00-00 '
 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 08/26/2021 10:37:31  Sequence number = 676  User id = 0  Event argument = 00-00 '
 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 08/26/2021 10:32:31  Sequence number = 5883  User id = 0  Event argument = 00-00 '
 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 08/26/2021 11:01:42  Sequence number = 677  User id = 0  Event argument = 00-00 '
 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 08/26/2021 12:19:13  Sequence number = 680  User id = 0  Event argument = 00-00 '
 'Meter event Tamper Attempt Suspected  Time event occurred on meter = 08/26/2021 13:47:53  Sequence number = 4384  User id = 0  Event argument = 00-00 '
 'Me

In [142]:
curated_reasons = []
for reason_i in df_3_33_1_257['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.33.1.257']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter event Tamper Attempt Suspected


# '3.35.0.28'

In [143]:
df_3_35_0_28 = pd.read_pickle(ids_with_paths_dict['3.35.0.28'])
#-----
print(f"shape[0] = {df_3_35_0_28.shape[0]}")
print(f"# Unique reasons = {df_3_35_0_28['reason'].nunique()}")
print(df_3_35_0_28['reason'].unique())

shape[0] = 1556
# Unique reasons = 602
['Cleared: Meter 00:13:50:05:ff:1d:d8:4b detected a high temperature condition. (C1219 Table 3)'
 'Cleared: Meter 00:13:50:05:ff:11:69:bb detected a high temperature condition. (C1219 Table 3)'
 'Cleared: Meter 00:13:50:05:ff:11:62:2a detected a high temperature condition. (C1219 Table 3)'
 'Cleared: Meter 00:13:50:05:ff:25:b5:ca detected a high temperature condition. (C1219 Table 3)'
 'Cleared: Meter 00:13:50:05:ff:11:28:b4 detected a high temperature condition. (C1219 Table 3)'
 'Cleared: Meter 00:13:50:05:ff:11:ab:8e detected a high temperature condition. (C1219 Table 3)'
 'Cleared: Meter 00:13:50:05:ff:1e:ce:46 detected a high temperature condition. (C1219 Table 3)'
 'Cleared: Meter 00:13:50:05:ff:11:e8:cf detected a high temperature condition. (C1219 Table 3)'
 'Cleared: Meter 00:13:50:05:ff:1e:41:18 detected a high temperature condition. (C1219 Table 3)'
 'Cleared: Meter 00:13:50:05:ff:19:a6:f8 detected a high temperature condition. (C1219 T

In [144]:
curated_reasons = []
for reason_i in df_3_35_0_28['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.35.0.28']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Cleared: detected a high temperature condition. (C1219 Table 3)


# '3.35.0.40'

In [145]:
df_3_35_0_40_1 = pd.read_pickle(ids_with_paths_dict['3.35.0.40'])
df_3_35_0_40_2 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_TX\sample_dfs\df_3_35_0_40.pkl')
df_3_35_0_40 = pd.concat([df_3_35_0_40_1, df_3_35_0_40_2])
#-----
print(f"shape[0] = {df_3_35_0_40.shape[0]}")
print(f"# Unique reasons = {df_3_35_0_40['reason'].nunique()}")
print(df_3_35_0_40['reason'].unique())

shape[0] = 3723
# Unique reasons = 1694
['Meter 00:13:50:05:ff:11:62:2a detected a high temperature condition. (C1219 Table 3)'
 'Meter 00:13:50:05:ff:1d:d8:4b detected a high temperature condition. (C1219 Table 3)'
 'Meter 00:13:50:05:ff:11:69:bb detected a high temperature condition. (C1219 Table 3)'
 ...
 'Meter event S4TemperatureThreshold  Time event occurred on meter = 06/09/2023 18:13:50  Sequence number = 69  User id = 0  Event argument = 00-00-00-00-00-00 '
 'Meter event S4TemperatureThreshold  Time event occurred on meter = 06/09/2023 18:22:50  Sequence number = 647  User id = 0  Event argument = 00-00-00-00-00-00 '
 'Meter event S4TemperatureThreshold  Time event occurred on meter = 06/09/2023 18:22:50  Sequence number = 283  User id = 0  Event argument = 00-00-00-00-00-00 ']


In [146]:
curated_reasons = []
for reason_i in df_3_35_0_40['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.35.0.40']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter's temperature threshold exceeded
Meter event S4TemperatureThreshold
Meter detected a high temperature condition (C1219 Table 3)
Meter event AX Temp Threshold Exceeded


# '3.36.0.79'

In [147]:
df_3_36_0_79 = pd.read_pickle(ids_with_paths_dict['3.36.0.79'])
#-----
print(f"shape[0] = {df_3_36_0_79.shape[0]}")
print(f"# Unique reasons = {df_3_36_0_79['reason'].nunique()}")
print(df_3_36_0_79['reason'].unique())

shape[0] = 18573
# Unique reasons = 11
['Meter detected a clock error.'
 'Meter event Clock Error Detected  Time event occurred on meter = 04/18/2020 10:00:10  Sequence number = 332  User id = 0  Event argument = 00-00'
 'Meter event Clock Error Detected  Time event occurred on meter = 05/11/2020 21:48:35  Sequence number = 35792  User id = 0  Event argument = 00-00'
 'Meter event Clock Error Detected  Time event occurred on meter = 07/24/2021 01:57:34  Sequence number = 39646  User id = 0  Event argument = 00-00 '
 'Meter event Clock Error Detected  Time event occurred on meter = 07/24/2021 01:57:44  Sequence number = 39650  User id = 0  Event argument = 00-00 '
 'Meter event Clock Error Detected  Time event occurred on meter = 07/24/2021 01:57:53  Sequence number = 39654  User id = 0  Event argument = 00-00 '
 'Meter event Clock Error Detected  Time event occurred on meter = 07/24/2021 01:57:29  Sequence number = 39644  User id = 0  Event argument = 00-00 '
 'Meter event Clock Error 

In [148]:
curated_reasons = []
for reason_i in df_3_36_0_79['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.36.0.79']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter event Clock Error Detected


# '3.36.1.29'

In [149]:
df_3_36_1_29 = pd.read_pickle(ids_with_paths_dict['3.36.1.29'])
#-----
print(f"shape[0] = {df_3_36_1_29.shape[0]}")
print(f"# Unique reasons = {df_3_36_1_29['reason'].nunique()}")
print(df_3_36_1_29['reason'].unique())

shape[0] = 94
# Unique reasons = 1
['Meter detected a clock error.']


In [150]:
curated_reasons = []
for reason_i in df_3_36_1_29['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.36.1.29']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter detected a clock error


# '3.36.114.73'

In [151]:
df_3_36_114_73 = pd.read_pickle(ids_with_paths_dict['3.36.114.73'])
#-----
print(f"shape[0] = {df_3_36_114_73.shape[0]}")
print(f"# Unique reasons = {df_3_36_114_73['reason'].nunique()}")
print(df_3_36_114_73['reason'].unique())

shape[0] = 270000
# Unique reasons = 205447
['Meter 00:13:50:05:ff:2d:e1:c6, detected loss of time (C1219 Table 3)'
 'Meter 00:13:50:05:ff:1c:9b:0a, detected loss of time (C1219 Table 3)'
 'Meter 00:13:50:05:ff:21:3f:ae, detected loss of time (C1219 Table 3)'
 ...
 'Meter 00:13:50:05:ff:49:fe:25, detected loss of time (C1219 Table 3)'
 'Meter 00:13:50:05:ff:41:5b:20, detected loss of time (C1219 Table 3)'
 'Meter 00:13:50:05:ff:40:7c:81, detected loss of time (C1219 Table 3)']


In [152]:
curated_reasons = []
for reason_i in df_3_36_114_73['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.36.114.73']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Meter detected loss of time (C1219 Table 3)


# '3.36.114.159'

In [153]:
df_3_36_114_159 = pd.read_pickle(ids_with_paths_dict['3.36.114.159'])
#-----
print(f"shape[0] = {df_3_36_114_159.shape[0]}")
print(f"# Unique reasons = {df_3_36_114_159['reason'].nunique()}")
print(df_3_36_114_159['reason'].unique())

shape[0] = 270000
# Unique reasons = 185143
['Ignoring Interval Read data for device 00:13:50:05:ff:0c:80:da as it has time in the future 2021-01-18 03:15:00.0'
 'Ignoring Interval Read data for device 00:13:50:05:ff:0c:80:da as it has time in the future 2021-01-18 06:00:00.0'
 'Ignoring Interval Read data for device 00:13:50:05:ff:0c:80:da as it has time in the future 2021-01-18 15:45:00.0'
 ...
 'Ignoring Interval Read data for device 00:13:50:01:01:3d:7a:5d as it has time in the future 2066-01-22 11:34:00.0'
 'Ignoring Interval Read data for device 00:13:50:01:01:6b:56:43 as it has time in the future 2074-01-22 09:51:00.0'
 'Ignoring Interval Read data for device 00:13:50:01:01:03:4b:69 as it has time in the future 2063-01-23 08:02:00.0']


In [154]:
curated_reasons = []
for reason_i in df_3_36_114_159['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.36.114.159']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Ignoring Register Read data for device as it has time in the future
Ignoring Interval Read data for device as it has time in the future


# '3.36.136.73'

In [155]:
df_3_36_136_73 = pd.read_pickle(ids_with_paths_dict['3.36.136.73'])
#-----
print(f"shape[0] = {df_3_36_136_73.shape[0]}")
print(f"# Unique reasons = {df_3_36_136_73['reason'].nunique()}")
print(df_3_36_136_73['reason'].unique())

shape[0] = 270000
# Unique reasons = 266216
['Meter 00:13:50:05:ff:24:1b:21 needs explicit time sync. Drift: 1759020 s, Encountered Problems:  TS_ERR_LP_BX, TS_ERR_BIG_DRIFT, TS_ERR_DAY_BX [0x4C], Meter_Time: 14:54:04'
 'Meter 00:13:50:05:ff:2c:ed:eb needs explicit time sync. Drift: -3595 s, Encountered Problems:  TS_ERR_NEAR_LP_BND, TS_ERR_LP_BX, TS_ERR_BIG_DRIFT [0x46], Meter_Time: 04:45:14'
 'Meter 00:13:50:05:ff:23:aa:d8 needs explicit time sync. Drift: -3607 s, Encountered Problems:  TS_ERR_LP_BX, TS_ERR_BIG_DRIFT [0x44], Meter_Time: 01:21:54'
 ...
 'Meter 00:13:50:05:ff:45:92:ce needs explicit time sync. Drift: 3601 s, Encountered Problems:  TS_ERR_NEAR_LP_BND, TS_ERR_LP_BX, TS_ERR_BIG_DRIFT [0x46], Meter_Time: 01:28:20'
 'Meter 00:13:50:05:ff:45:8e:24 needs explicit time sync. Drift: 3601 s, Encountered Problems:  TS_ERR_LP_BX, TS_ERR_BIG_DRIFT [0x44], Meter_Time: 01:46:48'
 'Meter 00:13:50:05:ff:45:16:91 needs explicit time sync. Drift: 3603 s, Encountered Problems:  TS_ERR_NEA

In [156]:
curated_reasons = []
for reason_i in df_3_36_136_73['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.36.136.73']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*sorted(curated_reasons), sep='\n')

Meter needs explicit time sync. Encountered Problems:  TS_ERR_LP_BX, TS_ERR_BIG_DRIFT [0x44]
Meter needs explicit time sync. Encountered Problems:  TS_ERR_LP_BX, TS_ERR_BIG_DRIFT, TS_ERR_DAY_BX [0x4C]
Meter needs explicit time sync. Encountered Problems:  TS_ERR_LP_BX, TS_ERR_BIG_DRIFT, TS_ERR_DAY_BX, TS_ERR_DST_BX [0x6C]
Meter needs explicit time sync. Encountered Problems:  TS_ERR_LP_BX, TS_ERR_BIG_DRIFT, TS_ERR_DAY_BX, TS_ERR_NEAR_DST_BND [0x5C]
Meter needs explicit time sync. Encountered Problems:  TS_ERR_LP_BX, TS_ERR_BIG_DRIFT, TS_ERR_DST_BX [0x64]
Meter needs explicit time sync. Encountered Problems:  TS_ERR_LP_BX, TS_ERR_BIG_DRIFT, TS_ERR_NEAR_DST_BND [0x54]
Meter needs explicit time sync. Encountered Problems:  TS_ERR_NEAR_DAY_BND, TS_ERR_LP_BX, TS_ERR_BIG_DRIFT [0x45]
Meter needs explicit time sync. Encountered Problems:  TS_ERR_NEAR_DAY_BND, TS_ERR_LP_BX, TS_ERR_BIG_DRIFT, TS_ERR_DAY_BX [0x4D]
Meter needs explicit time sync. Encountered Problems:  TS_ERR_NEAR_DAY_BND, TS_ERR

# '3.36.136.79'

In [157]:
df_3_36_136_79 = pd.read_pickle(ids_with_paths_dict['3.36.136.79'])
#-----
print(f"shape[0] = {df_3_36_136_79.shape[0]}")
print(f"# Unique reasons = {df_3_36_136_79['reason'].nunique()}")
print(df_3_36_136_79['reason'].unique())

shape[0] = 270000
# Unique reasons = 12052
['Error occurred when attempting to synch meter time with NIC time for device 00:13:50:05:ff:29:e1:20'
 'Error occurred when attempting to synch meter time with NIC time for device 00:13:50:03:ff:04:74:f2'
 'Error occurred when attempting to synch meter time with NIC time for device 00:13:50:05:ff:04:60:b6'
 ...
 'Error occurred when attempting to synch meter time with NIC time for device 00:13:50:05:ff:3a:67:11'
 'Error occurred when attempting to synch meter time with NIC time for device 00:13:50:05:ff:3b:72:77'
 'Error occurred when attempting to synch meter time with NIC time for device 00:13:50:05:ff:44:a8:e1']


In [158]:
curated_reasons = []
for reason_i in df_3_36_136_79['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.36.136.79']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

Error occurred when attempting to synch meter time with NIC time for device


# '3.38.1.139'

In [159]:
df_3_38_1_139 = pd.read_pickle(ids_with_paths_dict['3.38.1.139'])
#-----
print(f"shape[0] = {df_3_38_1_139.shape[0]}")
print(f"# Unique reasons = {df_3_38_1_139['reason'].nunique()}")
print(df_3_38_1_139['reason'].unique())

shape[0] = 136
# Unique reasons = 9
['KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase C Current '
 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase C Voltage '
 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase A Current '
 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase B Current '
 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase B Voltage, Phase C Voltage '
 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase B Voltage '
 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase A Current, Phase C Current '
 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase B Current, Phase C Current, Phase B Voltage, Phase C Voltage '
 'KV2c meter event Polarity, Cross Phase, Reverse Energy Flow Diagnostic flags:Phase B Current, Phase B

In [160]:
curated_reasons = []
for reason_i in df_3_38_1_139['reason'].unique().tolist():
    curated_reason_i = AMIEndEvents.reduce_end_event_reason(
        reason=reason_i, 
        patterns=std_patterns_to_replace['3.38.1.139']
    )
    curated_reasons.append(curated_reason_i)
curated_reasons = list(set(curated_reasons))
print(*curated_reasons, sep='\n')

KV2c meter event Polarity, Cross Phase, Reverse Energy Flow
